Extended Stigmergic Search

Because databases like OpenAlex and Semantic Scholar have *millions* of results, I have added a `max_batches = 10` (300 papers) safety limit to these scripts. Without this, fetching 50,000 papers at 5-second intervals would lock up your Colab notebook for 70 hours! You can easily change this number in the parameters.

*(Note: CORE is the only one that requires a free API key to function. I have marked where to put it in that specific cell).*

In [1]:
# @title 1. Cell 1: bioRxiv / medRxiv (via Crossref)

import requests
import time
from IPython.display import Markdown, display

keywords_input = "animal communication"
batch_size = 30
max_batches = 10 # Set to a higher number if you want more than 300 results

def search_biorxiv():
    words = [w.strip().lower() for w in keywords_input.replace(",", " ").replace("_", " ").split() if w.strip()]
    query = "+".join(words)
    print(f"Searching bioRxiv for: {' AND '.join(words)}...\n")

    base_url = "https://api.crossref.org/works"
    all_results = []

    for batch_number in range(1, max_batches + 1):
        offset = (batch_number - 1) * batch_size
        # prefix:10.1101 restricts the search entirely to bioRxiv
        params = {"query": query, "filter": "prefix:10.1101", "rows": batch_size, "offset": offset}

        try:
            response = requests.get(base_url, params=params).json()
            items = response.get("message", {}).get("items", [])
        except Exception as e:
            print("Error reaching API.")
            break

        if not items:
            print("No more results found.")
            break

        print(f"\n--- BATCH {batch_number} (bioRxiv) ---")
        for i, res in enumerate(items, 1):
            title = res.get("title", ["Unknown Title"])[0]
            year = res.get("published", {}).get("date-parts", [[None]])[0][0] or "Unknown Year"
            print(f"{(batch_number - 1) * batch_size + i}. {title} ({year})")

            # Format authors
            authors_list = res.get("author", [])
            authors = ", ".join([f"{a.get('given', '')} {a.get('family', '')}".strip() for a in authors_list]) or "Unknown Authors"

            all_results.append({"title": title, "year": year, "authors": authors, "url": res.get("URL", "")})

        if len(items) < batch_size:
            break

        print("\nWaiting 5 seconds...")
        time.sleep(5)

    if all_results:
        md_content = f"### bioRxiv Results ({len(all_results)} fetched)\n\n"
        for r in all_results:
            md_content += f"* **[{r['title']}]({r['url']})** ({r['year']}) <br> 👤 *{r['authors']}*\n\n"
        display(Markdown(md_content))

search_biorxiv()

Searching bioRxiv for: animal AND communication...


--- BATCH 1 (bioRxiv) ---
1. Animal-to-Animal Variability in Neuromodulation and Circuit Function (2014)
2. Corrigendum: Animal-to-Animal Variability in Neuromodulation and Circuit Function: Figure 5. (2014)
3. What Is the Predictive Value of Animal Models for Vaccine Efficacy in Humans? (2017)
4. Animal treatment (2006)
5. Lysis Buffer for Animal Tissues (2017)
6. Apoptotic and Nonapoptotic Caspase Functions in Animal Development (2012)
7. Bacterial Influences on Animal Origins (2014)
8. Detergent Lysis of Animal Tissues for Immunoprecipitation (2017)
9. DEVELOPMENTAL CYCLES IN ANIMAL VIRUSES (1953)
10. Isolation of Mitochondria from Animal Tissue (2014)
11. A Guide to Neurotoxic Animal Models of Parkinson's Disease (2011)
12. Dissecting and Culturing Animal Cap Explants (2018)
13. Animal Models of Parkinson's Disease (2025)
14. Wnt signaling: a common theme in animal development (1997)
15. Animal Models of Alzheimer Disease (2012)


### bioRxiv Results (163 fetched)

* **[Animal-to-Animal Variability in Neuromodulation and Circuit Function](https://doi.org/10.1101/sqb.2014.79.024828)** (2014) <br> 👤 *Albert W. Hamood, Eve Marder*

* **[Corrigendum: Animal-to-Animal Variability in Neuromodulation and Circuit Function: Figure 5.](https://doi.org/10.1101/sqb.2014.79.026955)** (2014) <br> 👤 *Albert W. Hamood, Eve Marder*

* **[What Is the Predictive Value of Animal Models for Vaccine Efficacy in Humans?](https://doi.org/10.1101/cshperspect.a031583)** (2017) <br> 👤 *Ramin Sedaghat Herati, E. John Wherry*

* **[Animal treatment](https://doi.org/10.1101/pdb.caut158)** (2006) <br> 👤 *Unknown Authors*

* **[Lysis Buffer for Animal Tissues](https://doi.org/10.1101/pdb.rec101444)** (2017) <br> 👤 *Unknown Authors*

* **[Apoptotic and Nonapoptotic Caspase Functions in Animal Development](https://doi.org/10.1101/cshperspect.a008664)** (2012) <br> 👤 *M. Miura*

* **[Bacterial Influences on Animal Origins](https://doi.org/10.1101/cshperspect.a016162)** (2014) <br> 👤 *R. A. Alegado, N. King*

* **[Detergent Lysis of Animal Tissues for Immunoprecipitation](https://doi.org/10.1101/pdb.prot098566)** (2017) <br> 👤 *James DeCaprio, Thomas O. Kohl*

* **[DEVELOPMENTAL CYCLES IN ANIMAL VIRUSES](https://doi.org/10.1101/sqb.1953.018.01.009)** (1953) <br> 👤 *W. Henle*

* **[Isolation of Mitochondria from Animal Tissue](https://doi.org/10.1101/pdb.prot080010)** (2014) <br> 👤 *David A. Clayton, Gerald S. Shadel*

* **[A Guide to Neurotoxic Animal Models of Parkinson's Disease](https://doi.org/10.1101/cshperspect.a009316)** (2011) <br> 👤 *K. Tieu*

* **[Dissecting and Culturing Animal Cap Explants](https://doi.org/10.1101/pdb.prot097329)** (2018) <br> 👤 *Kevin S. Dingwell, James C. Smith*

* **[Animal Models of Parkinson's Disease](https://doi.org/10.1101/cshperspect.a041644)** (2025) <br> 👤 *Valina L. Dawson, Ted M. Dawson*

* **[Wnt signaling: a common theme in animal development](https://doi.org/10.1101/gad.11.24.3286)** (1997) <br> 👤 *Ken M. Cadigan, Roel Nusse*

* **[Animal Models of Alzheimer Disease](https://doi.org/10.1101/cshperspect.a006320)** (2012) <br> 👤 *F. M. LaFerla, K. N. Green*

* **[Pattern and Timing of Gene Duplication in Animal Genomes](https://doi.org/10.1101/gr.200601)** (2001) <br> 👤 *Robert Friedman, Austin L. Hughes*

* **[Evolution of the Animal Apoptosis Network](https://doi.org/10.1101/cshperspect.a008649)** (2013) <br> 👤 *C. M. Zmasek, A. Godzik*

* **[Animal Models of Hepatitis C Virus Infection](https://doi.org/10.1101/cshperspect.a036970)** (2019) <br> 👤 *Alexander Ploss, Amit Kapoor*

* **[Mechanical Forces and Growth in Animal Tissues](https://doi.org/10.1101/cshperspect.a019232)** (2015) <br> 👤 *Loïc LeGoff, Thomas Lecuit*

* **[Cytokinesis in Animal Cells](https://doi.org/10.1101/cshperspect.a015834)** (2015) <br> 👤 *Pier Paolo D’Avino, Maria Grazia Giansanti, Mark Petronczki*

* **[Epigenetic Reprogramming in Early Animal Development](https://doi.org/10.1101/cshperspect.a039677)** (2021) <br> 👤 *Zhenhai Du, Ke Zhang, Wei Xie*

* **[Animal Models and the Molecular Biology of Hepadnavirus Infection](https://doi.org/10.1101/cshperspect.a021352)** (2015) <br> 👤 *W. S. Mason*

* **[Hedgehog signaling in animal development: paradigms and principles](https://doi.org/10.1101/gad.938601)** (2001) <br> 👤 *Philip W. Ingham, Andrew P. McMahon*

* **[ELECTROLYTE BALANCE OF ANIMAL CELLS](https://doi.org/10.1101/sqb.1940.008.01.025)** (1940) <br> 👤 *H. B. Steinbach*

* **[Toward telomere-to-telomere cat genomes for precision medicine and conservation biology](https://doi.org/10.1101/gr.278546.123)** (2024) <br> 👤 *William J. Murphy, Andrew J. Harris*

* **[Some Aspects of Animal Photoperiodism](https://doi.org/10.1101/sqb.1960.025.01.028)** (1960) <br> 👤 *A. D. Lees*

* **[Animal Models of Tuberculosis: Guinea Pigs](https://doi.org/10.1101/cshperspect.a018572)** (2014) <br> 👤 *S. Clark, Y. Hall, A. Williams*

* **[Animal Studies of Addictive Behavior](https://doi.org/10.1101/cshperspect.a011932)** (2012) <br> 👤 *L. J. M. J. Vanderschuren, S. H. Ahmed*

* **[Small Animal Models of Hepatitis E Virus Infection](https://doi.org/10.1101/cshperspect.a032581)** (2018) <br> 👤 *Tian-Cheng Li, Takaji Wakita*

* **[Lysis of Cultured Animal Cells, Yeast, and Bacteria for Immunoblotting](https://doi.org/10.1101/pdb.prot5467)** (2010) <br> 👤 *Hong Ji*

* **[The role of the gustatory cortex in incidental experience-evoked enhancement of later taste learning](https://doi.org/10.1101/lm.048181.118)** (2018) <br> 👤 *Veronica L. Flores, Tamar Parmet, Narendra Mukherjee, Sacha Nelson, Donald B. Katz, David Levitan*

* **[Basic Mechanisms in the Biology of Animal Viruses](https://doi.org/10.1101/sqb.1962.027.001.049)** (1962) <br> 👤 *R. Dulbecco*

* **[Neural encoding of innate preference to gravity-defying motion](https://doi.org/10.1101/2025.11.24.690125)** (2025) <br> 👤 *Dmitry Kobylkov, Giorgio Vallortigara*

* **[Toxin-Induced Animal Models of Parkinson's Disease](https://doi.org/10.1101/cshperspect.a041643)** (2024) <br> 👤 *Kim Tieu, Said S. Salehe, Harry J. Brown*

* **[Retinal Degeneration Animal Models in Bardet–Biedl Syndrome and Related Ciliopathies](https://doi.org/10.1101/cshperspect.a041303)** (2023) <br> 👤 *Clarisse Delvallée, Hélène Dollfus*

* **[Animal Models of Parkinson's Disease: Vertebrate Genetics](https://doi.org/10.1101/cshperspect.a009324)** (2012) <br> 👤 *Y. Lee, V. L. Dawson, T. M. Dawson*

* **[Animal Models of Chemical Carcinogenesis: Driving Breakthroughs in Cancer Research for 100 Years](https://doi.org/10.1101/pdb.top069906)** (2015) <br> 👤 *Christopher J. Kemp*

* **[Experimental Autoimmune Encephalomyelitis (EAE) as Animal Models of Multiple Sclerosis (MS)](https://doi.org/10.1101/cshperspect.a028977)** (2018) <br> 👤 *Simon Glatigny, Estelle Bettelli*

* **[Analysis of transgenic animal pedigrees using the junction-polymerase chain reaction.](https://doi.org/10.1101/gr.1.3.208)** (1992) <br> 👤 *M Ward, W Abramow-Newerly, S Clapoff, J C Roder*

* **[On the Delimitation of Communities in Research on Animal Biocoenotics](https://doi.org/10.1101/sqb.1957.022.01.034)** (1957) <br> 👤 *P. Kontkanen*

* **[Exploring host–microbiota interactions in animal models and humans](https://doi.org/10.1101/gad.212522.112)** (2013) <br> 👤 *Aleksandar D. Kostic, Michael R. Howitt, Wendy S. Garrett*

* **[Animal Models of Tuberculosis: Zebrafish](https://doi.org/10.1101/cshperspect.a018580)** (2014) <br> 👤 *L. M. van Leeuwen, A. M. van der Sar, W. Bitter*

* **[Animal Maintenance Systems: <i>Xenopus tropicalis</i>](https://doi.org/10.1101/pdb.prot106146)** (2020) <br> 👤 *Nikko-Ideen Shaidani, Sean McNamara, Marcin Wlizla, Marko E. Horb*

* **[Animal Models of the Behavioral Symptoms of Substance Use Disorders](https://doi.org/10.1101/cshperspect.a040287)** (2020) <br> 👤 *Louk J.M.J. Vanderschuren, Serge H. Ahmed*

* **[Animal Maintenance Systems: <i>Xenopus laevis</i>](https://doi.org/10.1101/pdb.prot106138)** (2020) <br> 👤 *Nikko-Ideen Shaidani, Sean McNamara, Marcin Wlizla, Marko E. Horb*

* **[Understanding cognitive deficits in Parkinson’s disease: lessons from preclinical animal models](https://doi.org/10.1101/lm.032029.113)** (2013) <br> 👤 *Nicola Solari, Alessandra Bonito-Oliva, Gilberto Fisone, Riccardo Brambilla*

* **[NRF-1: a trans-activator of nuclear-encoded respiratory genes in animal cells.](https://doi.org/10.1101/gad.4.6.1023)** (1990) <br> 👤 *M J Evans, R C Scarpulla*

* **[Growing an Embryo from a Single Cell: A Hurdle in Animal Life: Figure 1.](https://doi.org/10.1101/cshperspect.a019042)** (2015) <br> 👤 *Patrick H. O’Farrell*

* **[Advancing Animal Models of Human Type 1 Diabetes](https://doi.org/10.1101/cshperspect.a041587)** (2024) <br> 👤 *David V. Serreze, Jennifer R. Dwyer, Jeremy J. Racine*

* **[Chromatin Immunoprecipitation with Fixed Animal Tissues and Preparation for High-Throughput Sequencing](https://doi.org/10.1101/pdb.err087585)** (2015) <br> 👤 *Justin L. Cotney, James P. Noonan*

* **[The role of RNAi and microRNAs in animal virus replication and antiviral immunity](https://doi.org/10.1101/gad.1793309)** (2009) <br> 👤 *Jennifer L. Umbach, Bryan R. Cullen*

* **[Chromatin Immunoprecipitation with Fixed Animal Tissues and Preparation for High-Throughput Sequencing](https://doi.org/10.1101/pdb.prot084848)** (2015) <br> 👤 *Justin L. Cotney, James P. Noonan*

* **[Applying Tensile and Compressive Force to <i>Xenopus</i> Animal Cap Tissue](https://doi.org/10.1101/pdb.prot105551)** (2019) <br> 👤 *Georgina K. Goddard, Nawseen Tarannum, Sarah Woolner*

* **[Electron Microscopic Studies on the Architecture of Animal Viruses](https://doi.org/10.1101/sqb.1962.027.001.006)** (1962) <br> 👤 *P. Wildy, D. H. Watson*

* **[The green yeast uses its plant-like clock to regulate its animal-like tail: Figure 1.](https://doi.org/10.1101/gad.1664508)** (2008) <br> 👤 *Michael Brunner, Martha Merrow*

* **[Use of Gene Transfer to Increase Animal Growth](https://doi.org/10.1101/sqb.1985.050.01.048)** (1985) <br> 👤 *R.E. Hammer, R.L. Brinster, R.D. Palmiter*

* **[In Vitro Induction of<i>Xenopus</i>Embryonic Organs Using Animal Cap Cells](https://doi.org/10.1101/pdb.prot097410)** (2017) <br> 👤 *Takashi Ariizumi, Tatsuo Michiue, Makoto Asashima*

* **[Physical and Chemical Properties and Infectivity of RNA from Animal Viruses](https://doi.org/10.1101/sqb.1962.027.001.012)** (1962) <br> 👤 *F. L. Schaffer*

* **[Purification of RNA from Animal Cells Using Trizol](https://doi.org/10.1101/pdb.prot4104)** (2006) <br> 👤 *Michael A. Connolly, Peter A. Clausen, James G. Lazar*

* **[Linkage and Expression of Foreign DNA in Cultured Animal Cells](https://doi.org/10.1101/sqb.1981.045.01.101)** (1981) <br> 👤 *M. Perucho, M. Wigler*

* **[A simple and rapid DNA microextraction method for plant, animal, and insect suitable for RAPD and other PCR analyses.](https://doi.org/10.1101/gr.3.1.69)** (1993) <br> 👤 *W Y Cheung, N Hubert, B S Landry*

* **[Animal model of methylphenidate's long-term memory-enhancing effects](https://doi.org/10.1101/lm.033613.113)** (2014) <br> 👤 *Stephanie A. Carmack, Kristin K. Howell, Kleou Rasaei, Emilie T. Reas, Stephan G. Anagnostaras*

* **[SOME PROBLEMS OF ANIMAL VIROLOGY AS STUDIED BY THE PLAQUE TECHNIQUE](https://doi.org/10.1101/sqb.1953.018.01.039)** (1953) <br> 👤 *R. Dulbecco, M. Vogt*

* **[Coordination between Cell Growth and Cell Cycle Transit in Animal Cells](https://doi.org/10.1101/sqb.1991.056.01.018)** (1991) <br> 👤 *A. Zetterberg, O. Larsson*

* **[Japanese Quail: An Efficient Animal Model for the Production of Transgenic Avians](https://doi.org/10.1101/pdb.emo112)** (2009) <br> 👤 *Greg Poynter, David Huss, Rusty Lansford*

* **[Harnessing brain–body communication to understand cancer](https://doi.org/10.1101/gad.352292.124)** (2024) <br> 👤 *Erica K. Sloan*

* **[Animal Cap Isolation from <i>Xenopus laevis</i>: Figure 1.](https://doi.org/10.1101/pdb.prot4744)** (2007) <br> 👤 *Hazel L. Sive, Robert M. Grainger, Richard M. Harland*

* **[Positional Identification of Structural and Regulatory Quantitative Trait Nucleotides in Domestic Animal Species](https://doi.org/10.1101/sqb.2003.68.179)** (2003) <br> 👤 *M. GEORGES, L. ANDERSSON*

* **[G1 Control in Yeast and Animal Cells](https://doi.org/10.1101/sqb.1991.056.01.009)** (1991) <br> 👤 *S.I. Reed, C. Wittenberg, D.J. Lew, V. Dulic, M. Henze*

* **[Responses to Novelty and Vulnerability to Cocaine Addiction: Contribution of a Multi-Symptomatic Animal Model](https://doi.org/10.1101/cshperspect.a011940)** (2012) <br> 👤 *D. Belin, V. Deroche-Gamonet*

* **[Dissociation and Reaggregation of <i>Xenopus laevis</i> Animal Caps](https://doi.org/10.1101/pdb.prot4745)** (2007) <br> 👤 *Hazel L. Sive, Robert M. Grainger, Richard M. Harland*

* **[Deep-Pose-Tracker: a unified model for behavioural studies of
                  <i>Caenorhabditis elegans</i>](https://doi.org/10.1101/2025.11.23.689997)** (2025) <br> 👤 *Debasish Saha, Shivam Chaudhary, Dhyey Vyas, Anindya Ghosh Roy, Rati Sharma*

* **[Intron length increases oscillatory periods of gene expression in animal cells](https://doi.org/10.1101/gad.1696108)** (2008) <br> 👤 *Ian A. Swinburne, David G. Miguez, Dirk Landgraf, Pamela A. Silver*

* **[THE USE OF TRACERS IN THE STUDY OF ACTIVE ION TRANSPORT ACROSS ANIMAL MEMBRANES](https://doi.org/10.1101/sqb.1948.013.01.026)** (1948) <br> 👤 *H. H. Ussing*

* **[Genomic architecture constrained placental mammal X Chromosome evolution](https://doi.org/10.1101/gr.275274.121)** (2021) <br> 👤 *Wesley A. Brashear, Kevin R. Bredemeyer, William J. Murphy*

* **[<i>Xenopus laevis</i> Animal Cap/Dorsal Mesoderm Conjugates: Figure 1.](https://doi.org/10.1101/pdb.prot4748)** (2007) <br> 👤 *Hazel L. Sive, Robert M. Grainger, Richard M. Harland*

* **[Homeo Box Sequences of the Antennapedia Class Are Conserved Only in Higher Animal Genomes](https://doi.org/10.1101/sqb.1985.050.01.033)** (1985) <br> 👤 *W. McGinnis*

* **[<i>Xenopus laevis</i> Animal Cap/Vegetal Endoderm Conjugates: Figure 1.](https://doi.org/10.1101/pdb.prot4747)** (2007) <br> 👤 *Hazel L. Sive, Robert M. Grainger, Richard M. Harland*

* **[Terri Grodzicker: 35 years of shaping scientific publishing and communication](https://doi.org/10.1101/gad.350465.123)** (2023) <br> 👤 *Rudolf Grosschedl*

* **[Gene Repression Paradigms in Animal Cells](https://doi.org/10.1101/sqb.2004.69.131)** (2004) <br> 👤 *L. LANDE-DINER, J. ZHANG, T. HASHIMSHONY, A. GOREN, I. KESHET, H. CEDAR*

* **[Convergence of Human Genetics and Animal Studies: Gene Therapy for X-Linked Retinoschisis](https://doi.org/10.1101/cshperspect.a017368)** (2015) <br> 👤 *Ronald A. Bush, Lisa L. Wei, Paul A. Sieving*

* **[APPLICATIONS OF STRAINS OF MAMMALIAN CELLS TO THE STUDY OF ANIMAL VIRUSES](https://doi.org/10.1101/sqb.1953.018.01.041)** (1953) <br> 👤 *J. T. Syverton, W. F. Scherer*

* **[RNA pseudoknot domain of tobacco mosaic virus can functionally substitute for a poly(A) tail in plant and animal cells.](https://doi.org/10.1101/gad.4.7.1149)** (1990) <br> 👤 *D R Gallie, V Walbot*

* **[A reminder of extinction reduces relapse in an animal model of voluntary behavior](https://doi.org/10.1101/lm.044495.116)** (2017) <br> 👤 *Javier Nieto, Metin Uengoer, Rodolfo Bernal-Gamboa*

* **[Influenza in High-Risk Hosts—Lessons Learned from Animal Models](https://doi.org/10.1101/cshperspect.a038604)** (2019) <br> 👤 *Rebekah Honce, Nicholas Wohlgemuth, Victoria A. Meliopoulos, Kirsty R. Short, Stacey Schultz-Cherry*

* **[Ectodermal (Animal Cap) Layer Separations in <i>Xenopus laevis</i>: Figure 1.](https://doi.org/10.1101/pdb.prot4746)** (2007) <br> 👤 *Hazel L. Sive, Robert M. Grainger, Richard M. Harland*

* **[Calcium Signaling in Synapse-to-Nucleus Communication](https://doi.org/10.1101/cshperspect.a004564)** (2011) <br> 👤 *A. M. Hagenston, H. Bading*

* **[Placental Extracellular Vesicles and Feto-Maternal Communication](https://doi.org/10.1101/cshperspect.a023028)** (2015) <br> 👤 *M. Tong, L. W. Chamley*

* **[Social modulation of associative fear learning by pheromone communication](https://doi.org/10.1101/lm.1226009)** (2008) <br> 👤 *Timothy W. Bredy, Mark Barad*

* **[Interaction between Tumor Viruses and Cells in Cysts and Tumors Induced by these Viruses in various Animal Species](https://doi.org/10.1101/sqb.1962.027.001.048)** (1962) <br> 👤 *L. A. Zilber*

* **[From humans to rats and back again: Bridging the divide between human and animal studies of recognition memory with receiver operating characteristics](https://doi.org/10.1101/lm.2214511)** (2011) <br> 👤 *Joshua D. Koen, Andrew P. Yonelinas*

* **[Demonstration of nondeclarative sequence learning in mice: Development of an animal analog of the human serial reaction time task: Figure 1.](https://doi.org/10.1101/lm.85404)** (2004) <br> 👤 *Michael A. Christie, Steven M. Hersch*

* **[Sex Differences in Major Depressive Disorder (MDD) and Preclinical Animal Models for the Study of Depression](https://doi.org/10.1101/cshperspect.a039198)** (2021) <br> 👤 *Elizabeth S. Williams, Michelle Mazei-Robison, A.J. Robison*

* **[The Functional Genomic Distribution of Protein Divergence in Two Animal Phyla: Coevolution, Genomic Conflict, and Constraint](https://doi.org/10.1101/gr.2195604)** (2004) <br> 👤 *Cristian I. Castillo-Davis, Fyodor A. Kondrashov, Daniel L. Hartl, Rob J. Kulathinal*

* **[Ribosome shunting in the cauliflower mosaic virus 35S RNA leader is a special case of reinitiation of translation functioning in plant and animal systems](https://doi.org/10.1101/gad.14.7.817)** (2000) <br> 👤 *Lyubov A. Ryabova, Thomas Hohn*

* **[Infectious Viral DNA in Rous Sarcoma Virus-transformed Nonproducer and Producer Animal Cells](https://doi.org/10.1101/sqb.1974.039.01.117)** (1974) <br> 👤 *M. Hill, J. Hillova, D. Dantchev, R. Mariage, G. Goubin*

* **[The Replication and Structure of Mitochondrial DNA in Animal Cells](https://doi.org/10.1101/sqb.1974.038.01.031)** (1974) <br> 👤 *H. Kasamatsu, L. I. Grossman, D. L. Robberson, R. Watson, J. Vinograd*

* **[Poetry of b1 Paramutation: cis- and trans-Chromatin Communication](https://doi.org/10.1101/sqb.2004.69.355)** (2004) <br> 👤 *V.L. CHANDLER*

* **[On the Structural Organization of the Transcriptional Unit in Animal Chromosomes](https://doi.org/10.1101/sqb.1974.038.01.089)** (1974) <br> 👤 *G. P. Georgiev, A. Ja. Varshavsky, A. P. Ryskov, R. B. Church*

* **[Probing Acoustic Communication during Fly Reproductive Behaviors](https://doi.org/10.1101/pdb.prot108107)** (2023) <br> 👤 *Anne C. von Philipsborn, Galit Shohat-Ophir, Carolina Rezaval*

* **[Studies on Early Intermediates in the Synthesis of DNA in Animal Cells](https://doi.org/10.1101/sqb.1979.043.01.032)** (1979) <br> 👤 *B. Y. Tseng, R. H. Grafstrom, D. Revie, W. Oertel, M. Goulian*

* **[Identification of a rat liver nuclear protein that binds to the enhancer core element of three animal viruses.](https://doi.org/10.1101/gad.1.2.133)** (1987) <br> 👤 *P F Johnson, W H Landschulz, B J Graves, S L McKnight*

* **[Evolutionary conservation of Y Chromosome ampliconic gene families despite extensive structural variation](https://doi.org/10.1101/gr.237586.118)** (2018) <br> 👤 *Wesley A. Brashear, Terje Raudsepp, William J. Murphy*

* **[Advancing Animal Models of Human Type 1 Diabetes by Engraftment of Functional Human Tissues in Immunodeficient Mice](https://doi.org/10.1101/cshperspect.a007757)** (2012) <br> 👤 *M. A. Brehm, A. C. Powers, L. D. Shultz, D. L. Greiner*

* **[What Is the Predictive Value of Animal Models for Vaccine Efficacy in Humans?](https://doi.org/10.1101/cshperspect.a029504)** (2017) <br> 👤 *Mauricio A. Martins, David I. Watkins*

* **[Structure and Function of Animal Cryptochromes](https://doi.org/10.1101/sqb.2007.72.015)** (2007) <br> 👤 *N. Öztürk, S.-H. Song, S. Özgür, C. P. Selby, L. Morrison, C. Partch, D. Zhong, A. Sancar*

* **[Endoplasmic Reticulum Architecture and Inter-Organelle Communication in Metabolic Health and Disease](https://doi.org/10.1101/cshperspect.a041261)** (2022) <br> 👤 *Ana Paula Arruda, Güneş Parlakgül*

* **[Rescue of mice homozygous for lethal albino deletions: implications for an animal model for the human liver disease tyrosinemia type 1.](https://doi.org/10.1101/gad.7.12a.2285)** (1993) <br> 👤 *G Kelsey, S Ruppert, F Beermann, C Grund, R M Tanguay, G Schutz*

* **[What Is the Predictive Value of Animal Models for Vaccine Efficacy in Humans?](https://doi.org/10.1101/cshperspect.a029132)** (2017) <br> 👤 *Stephen C. Jameson, David Masopust*

* **[Fear memory consolidation in sleep requires protein kinase A](https://doi.org/10.1101/lm.046458.117)** (2018) <br> 👤 *Jiyeon Cho, Krzysztof A. Sypniewski, Shoko Arai, Kazuo Yamada, Sonoko Ogawa, Constantine Pavlides*

* **[A plastidic ABC protein involved in intercompartmental communication of light signaling](https://doi.org/10.1101/gad.850101)** (2001) <br> 👤 *Simon Geir Møller, Tim Kunkel, Nam-Hai Chua*

* **[What Is the Predictive Value of Animal Models for Vaccine Efficacy in Humans?](https://doi.org/10.1101/cshperspect.a028902)** (2017) <br> 👤 *Hana Golding, Surender Khurana, Marina Zaitseva*

* **[Reduced U snRNP assembly causes motor axon degeneration in an animal model for spinal muscular atrophy](https://doi.org/10.1101/gad.342005)** (2005) <br> 👤 *Christoph Winkler, Christian Eggert, Dietmar Gradl, Gunter Meister, Marieke Giegerich, Doris Wedlich, Bernhard Laggerbauer, Utz Fischer*

* **[Revisiting models of enhancer–promoter communication in gene regulation](https://doi.org/10.1101/gr.278389.123)** (2025) <br> 👤 *Gilad Barshad, Charles G. Danko*

* **[Spindle-to-Cortex Communication in Cleaving Frog Eggs](https://doi.org/10.1101/sqb.2017.82.033654)** (2017) <br> 👤 *Timothy J. Mitchison, Christine M. Field*

* **[Phylogenomic evidence for ancient hybridization in the genomes of living cats (Felidae)](https://doi.org/10.1101/gr.186668.114)** (2015) <br> 👤 *Gang Li, Brian W. Davis, Eduardo Eizirik, William J. Murphy*

* **[Organization and dynamics of the Mu transpososome: recombination by communication between two active sites](https://doi.org/10.1101/gad.13.20.2725)** (1999) <br> 👤 *T. L. Williams, E. L. Jackson, A. Carritte, T. A. Baker*

* **[<i>WUSCHEL</i> signaling functions in interregional communication during <i>Arabidopsis</i> ovule development](https://doi.org/10.1101/gad.225202)** (2002) <br> 👤 *Rita Groß-Hardt, Michael Lenhard, Thomas Laux*

* **[Rapid communication of efforts to resolve differences or update variant interpretations in ClinVar through case-level data sharing](https://doi.org/10.1101/mcs.a003467)** (2018) <br> 👤 *Heidi Rehm*

* **[New Evidence for Genome-Wide Duplications at the Origin of Vertebrates Using an Amphioxus Gene Set and Completed Animal Genomes](https://doi.org/10.1101/gr.874803)** (2003) <br> 👤 *Georgia Panopoulou, Steffen Hennig, Detlef Groth, Antje Krause, Albert J. Poustka, Ralf Herwig, Martin Vingron, Hans Lehrach*

* **[Splicing of SV40 mRNAs: A Novel Mechanism for the Regulation of Gene Expression in Animal Cells](https://doi.org/10.1101/sqb.1978.042.01.058)** (1978) <br> 👤 *Y. Aloni, S. Bratosin, R. Dhar, O. Laub, M. Horowitz, G. Khoury*

* **[Rapid communication of efforts to resolve differences or update variant interpretations in ClinVar through case-level data sharing](https://doi.org/10.1101/mcs.a003467)** (2018) <br> 👤 *Heidi Rehm*

* **[Enhancer–promoter communication mediated by Chip during Pannier-driven proneural patterning is regulated by Osa](https://doi.org/10.1101/gad.255703)** (2003) <br> 👤 *Pascal Heitzler, Luc Vanolst, Inna Biryukova, Philippe Ramain*

* **[FETAL AND ADULT HEMOGLOBIN IN THE BLOOD OF THE HUMAN FETUS, A PRELIMINARY COMMUNICATION](https://doi.org/10.1101/sqb.1954.019.01.018)** (1954) <br> 👤 *J. Walker*

* **[Transcriptional regulation of the piRNA pathway by Ovo in animal ovarian germ cells](https://doi.org/10.1101/gad.352120.124)** (2024) <br> 👤 *Azad Alizada, Gregory J. Hannon, Benjamin Czech Nicholson*

* **[Arrangement and Characterization of Repetitive Sequence Elements in Animal DNAs](https://doi.org/10.1101/sqb.1974.038.01.033)** (1974) <br> 👤 *E. H. Davidson, D. E. Graham, B. R. Neufeld, M. E. Chamberlin, C. S. Amenson, B. R. Hough, R. J. Britten*

* **[Functional Consequences of Calcium-Dependent Synapse-to-Nucleus Communication: Focus on Transcription-Dependent Metabolic Plasticity](https://doi.org/10.1101/cshperspect.a035287)** (2019) <br> 👤 *Anna M. Hagenston, Hilmar Bading, Carlos Bas-Orth*

* **[Defining the transcriptional redundancy of early bodywall muscle development in
                    <i>C. elegans</i>
                    : evidence for a unified theory of animal muscle development](https://doi.org/10.1101/gad.1481706)** (2006) <br> 👤 *Tetsunari Fukushige, Thomas M. Brodigan, Lawrence A. Schriefer, Robert H. Waterston, Michael Krause*

* **[Mechanisms of Self-tolerance and Autoimmunity: From Whole-animal Phenotypes to Molecular Pathways](https://doi.org/10.1101/sqb.1999.64.313)** (1999) <br> 👤 *C.C. GOODNOW, R. GLYNNE, D. MACK, B. WEINTRAUB, J. RATHMELL, J.I. HEALY, S. CHAUDHRY, L. MIOSGE, A. LOY, L. WILSON*

* **[Communication between Hin recombinase and Fis regulatory subunits during coordinate activation of Hin-catalyzed site-specific DNA inversion](https://doi.org/10.1101/gad.12.17.2803)** (1998) <br> 👤 *Stacy K. Merickel, Michael J. Haykinson, Reid C. Johnson*

* **[Coordination and communication between the p53 and IGF-1–AKT–TOR signal transduction pathways](https://doi.org/10.1101/gad.1363206)** (2006) <br> 👤 *Arnold J. Levine, Zhaohui Feng, Tak W. Mak, Han You, Shengkan Jin*

* **[Juvenile female rats, but not male rats, show renewal, reinstatement, and spontaneous recovery following extinction of conditioned fear](https://doi.org/10.1101/lm.045831.117)** (2017) <br> 👤 *Chun Hui J. Park, Despina E. Ganella, Jee Hyun Kim*

* **[Brain region-specific effects of cGMP-dependent kinase II knockout on AMPA receptor trafficking and animal behavior](https://doi.org/10.1101/lm.042960.116)** (2016) <br> 👤 *Seonil Kim, Joseph E. Pick, Sinedu Abera, Latika Khatri, Danielle D.P. Ferreira, Matheus F. Sathler, Sage L. Morison, Franz Hofmann, Edward B. Ziff*

* **[Genetic basis of chemical communication in eusocial insects](https://doi.org/10.1101/gad.346965.120)** (2021) <br> 👤 *Hua Yan, Jürgen Liebig*

* **[Distal regulation, silencers, and a shared combinatorial syntax are hallmarks of animal embryogenesis](https://doi.org/10.1101/gr.275864.121)** (2022) <br> 👤 *Paola Cornejo-Páramo, Kathrein Roper, Sandie M. Degnan, Bernard M. Degnan, Emily S. Wong*

* **[A comprehensive survey of 3′ animal miRNA modification events and a possible role for 3′ adenylation in modulating miRNA targeting effectiveness](https://doi.org/10.1101/gr.106054.110)** (2010) <br> 👤 *A. Maxwell Burroughs, Yoshinari Ando, Michiel J.L. de Hoon, Yasuhiro Tomaru, Takahiro Nishibu, Ryo Ukekawa, Taku Funakoshi, Tsutomu Kurokawa, Harukazu Suzuki, Yoshihide Hayashizaki, Carsten O. Daub*

* **[Mechanisms of Receptor-mediated Transmembrane Communication](https://doi.org/10.1101/sqb.1986.051.01.090)** (1986) <br> 👤 *L. Ellis, D.O. Morgan, E. Clauser, M. Edery, S.-M. Jong, L.-H. Wang, R.A. Roth, W.J. Rutter*

* **[Communication between Target Organs and Nerve Cells: Retrograde Axonal Transport and Site of Action of Nerve Growth Factor](https://doi.org/10.1101/sqb.1982.046.01.016)** (1982) <br> 👤 *M. E. Schwab, R. Heumann, H. Thoenen*

* **[Interorganelle communication, aging, and neurodegeneration](https://doi.org/10.1101/gad.346759.120)** (2021) <br> 👤 *Maja Petkovic, Caitlin E. O'Brien, Yuh Nung Jan*

* **[Exploration of fine-scale recombination rate variation in the domestic horse](https://doi.org/10.1101/gr.243311.118)** (2019) <br> 👤 *Samantha K. Beeson, James R. Mickelson, Molly E. McCue*

* **[The Demosponge <i>Amphimedon queenslandica</i>: Reconstructing the Ancestral Metazoan Genome and Deciphering the Origin of Animal Multicellularity](https://doi.org/10.1101/pdb.emo108)** (2008) <br> 👤 *Bernard M. Degnan, Maja Adamska, Alina Craigie, Sandie M. Degnan, Bryony Fahey, Marie Gauthier, John N.A. Hooper, Claire Larroux, Sally P. Leys, Erica Lovas, Gemma S. Richards*

* **[Modulation of olfactory bulb network activity by serotonin: synchronous inhibition of mitral cells mediated by spatially localized GABAergic microcircuits](https://doi.org/10.1101/lm.035659.114)** (2014) <br> 👤 *Loren J. Schmidt, Ben W. Strowbridge*

* **[Recruitment of histone modifications to assist mRNA dosage maintenance after degeneration of cytosine DNA methylation during animal evolution](https://doi.org/10.1101/gr.221739.117)** (2017) <br> 👤 *Andrew Ying-Fei Chang, Ben-Yang Liao*

* **[Epigenetic analyses of planarian stem cells demonstrate conservation of bivalent histone modifications in animal stem cells](https://doi.org/10.1101/gr.239848.118)** (2018) <br> 👤 *Anish Dattani, Damian Kao, Yuliana Mihaylova, Prasad Abnave, Samantha Hughes, Alvina Lai, Sounak Sahu, A. Aziz Aboobaker*

* **[Mapping the regulatory landscape of auditory hair cells from single-cell multi-omics data](https://doi.org/10.1101/gr.271080.120)** (2021) <br> 👤 *Shuze Wang, Mary P. Lee, Scott Jones, Jie Liu, Joerg Waldhaus*

* **[Cellular projections from sensory hair cells form polarity-specific scaffolds during synaptogenesis](https://doi.org/10.1101/gad.259838.115)** (2015) <br> 👤 *Eliot Dow, Kimberly Siletti, Albert J. Hudspeth*

* **[Phylogeny-wide analysis of social amoeba genomes highlights ancient origins for complex intercellular communication](https://doi.org/10.1101/gr.121137.111)** (2011) <br> 👤 *Andrew J. Heidel, Hajara M. Lawal, Marius Felder, Christina Schilde, Nicholas R. Helps, Budi Tunggal, Francisco Rivero, Uwe John, Michael Schleicher, Ludwig Eichinger, Matthias Platzer, Angelika A. Noegel, Pauline Schaap, Gernot Glöckner*

* **[Inhibition of prefrontal glutamatergic neuron activity during the recovery period following chronic stress disrupts fear memory in male rats: potential role of the infralimbic cortex](https://doi.org/10.1101/lm.053957.124)** (2025) <br> 👤 *Jessica M. Judd, Dylan N. Peay, Jinah L. Kim, Elliot A. Smith, Megan E. Donnay, Joel Miller, Jean-Paul Klein, Erin K. Nagy, Amanda M. Acuña, M. Foster Olive, Cheryl D. Conrad*

* **[Role of the proline-rich disordered domain of DROSHA in intronic microRNA processing](https://doi.org/10.1101/gad.350275.122)** (2023) <br> 👤 *Soomin Son, Baekgyu Kim, Jihye Yang, V. Narry Kim*

* **[The transcription factor activity gradient (TAG) model: contemplating a contact-independent mechanism for enhancer–promoter communication](https://doi.org/10.1101/gad.349160.121)** (2021) <br> 👤 *Jonathan P. Karr, John J. Ferrie, Robert Tjian, Xavier Darzacq*

* **[Recent, full-length gene retrocopies are common in canids](https://doi.org/10.1101/gr.276828.122)** (2022) <br> 👤 *Kevin Batcher, Scarlett Varney, Daniel York, Matthew Blacksmith, Jeffrey M. Kidd, Robert Rebhun, Peter Dickinson, Danika Bannasch*

* **[Homozygous editing of multiple genes for accelerated generation of xenotransplantation pigs](https://doi.org/10.1101/gr.279709.124)** (2025) <br> 👤 *Xiaoyue Duan, Chaolei Chen, Chang Du, Liang Guo, Jun Liu, Naipeng Hou, Pan Li, Xiaolan Qi, Fei Gao, Xuguang Du, Jiangping Song, Sen Wu*

* **[Upgrading short-read animal genome assemblies to chromosome level using comparative genomics and a universal probe set](https://doi.org/10.1101/gr.213660.116)** (2016) <br> 👤 *Joana Damas, Rebecca O'Connor, Marta Farré, Vasileios Panagiotis E. Lenis, Henry J. Martell, Anjali Mandawala, Katie Fowler, Sunitha Joseph, Martin T. Swain, Darren K. Griffin, Denis M. Larkin*

* **[Single-cell transcriptomics of the human placenta: inferring the cell communication network of the maternal-fetal interface](https://doi.org/10.1101/gr.207597.116)** (2017) <br> 👤 *Mihaela Pavličev, Günter P. Wagner, Arun Rajendra Chavan, Kathryn Owens, Jamie Maziarz, Caitlin Dunn-Fletcher, Suhas G. Kallapur, Louis Muglia, Helen Jones*

* **[Using genome-wide measures of coancestry to maintain diversity and fitness in endangered and domestic pig populations](https://doi.org/10.1101/gr.187039.114)** (2015) <br> 👤 *Mirte Bosse, Hendrik-Jan Megens, Ole Madsen, Richard P.M.A. Crooijmans, Oliver A. Ryder, Frédéric Austerlitz, Martien A.M. Groenen, M. Angeles R. de Cara*

* **[Exome sequencing of lymphomas from three dog breeds reveals somatic mutation patterns reflecting genetic background](https://doi.org/10.1101/gr.194449.115)** (2015) <br> 👤 *Ingegerd Elvers, Jason Turner-Maier, Ross Swofford, Michele Koltookian, Jeremy Johnson, Chip Stewart, Cheng-Zhong Zhang, Steven E. Schumacher, Rameen Beroukhim, Mara Rosenberg, Rachael Thomas, Evan Mauceli, Gad Getz, Federica Di Palma, Jaime F. Modiano, Matthew Breen, Kerstin Lindblad-Toh, Jessica Alföldi*

* **[A cryptic Tudor domain links BRWD2/PHIP to COMPASS-mediated histone H3K4 methylation](https://doi.org/10.1101/gad.305201.117)** (2017) <br> 👤 *Marc A.J. Morgan, Ryan A. Rickels, Clayton K. Collings, Xiaolin He, Kaixiang Cao, Hans-Martin Herz, Kira A. Cozzolino, Nebiyu A. Abshiru, Stacy A. Marshall, Emily J. Rendleman, Christie C. Sze, Andrea Piunti, Neil L. Kelleher, Jeffrey N. Savas, Ali Shilatifard*

* **[ESICCC as a systematic computational framework for evaluation, selection, and integration of cell-cell communication inference methods](https://doi.org/10.1101/gr.278001.123)** (2023) <br> 👤 *Jiaxin Luo, Minghua Deng, Xuegong Zhang, Xiaoqiang Sun*

* **[Sleep strengthens resting-state functional communication between brain areas involved in the consolidation of problem-solving skills](https://doi.org/10.1101/lm.053638.122)** (2023) <br> 👤 *Nicholas H. van den Berg, Dylan Smith, Zhuo Fang, Alyssa Pozzobon, Balmeet Toor, Julia Al-Kuwatli, Laura Ray, Stuart M. Fogel*

* **[<i>Plagl1</i>
                    and
                    <i>Lrrc58</i>
                    control mammalian body size by triggering target-directed microRNA degradation of miR-322 and miR-503](https://doi.org/10.1101/gad.353138.125)** (2025) <br> 👤 *Collette A. LaVigne, Jaeil Han, He Zhang, Sihoon Cho, Minseon Kim, Komal Sethia, Bret M. Evers, Asha Acharya, Tsung-Cheng Chang, Joshua T. Mendell*

* **[Power-law behavior of transcriptional bursting regulated by enhancer–promoter communication](https://doi.org/10.1101/gr.278631.123)** (2024) <br> 👤 *Zihao Wang, Zhenquan Zhang, Songhao Luo, Tianshou Zhou, Jiajun Zhang*

* **[Comprehensive analyses of 723 transcriptomes enhance genetic and biological interpretations for complex traits in cattle](https://doi.org/10.1101/gr.250704.119)** (2020) <br> 👤 *Lingzhao Fang, Wentao Cai, Shuli Liu, Oriol Canela-Xandri, Yahui Gao, Jicai Jiang, Konrad Rawlik, Bingjie Li, Steven G. Schroeder, Benjamin D. Rosen, Cong-jun Li, Tad S. Sonstegard, Leeson J. Alexander, Curtis P. Van Tassell, Paul M. VanRaden, John B. Cole, Ying Yu, Shengli Zhang, Albert Tenesa, Li Ma, George E. Liu*

* **[GenomeMUSter mouse genetic variation service enables multitrait, multipopulation data integration and analysis](https://doi.org/10.1101/gr.278157.123)** (2024) <br> 👤 *Robyn L. Ball, Molly A. Bogue, Hongping Liang, Anuj Srivastava, David G. Ashbrook, Anna Lamoureux, Matthew W. Gerring, Alexander S. Hatoum, Matthew J. Kim, Hao He, Jake Emerson, Alexander K. Berger, David O. Walton, Keith Sheppard, Baha El Kassaby, Francisco Castellanos, Govindarajan Kunde-Ramamoorthy, Lu Lu, John Bluis, Sejal Desai, Beth A. Sundberg, Gary Peltz, Zhuoqing Fang, Gary A. Churchill, Robert W. Williams, Arpana Agrawal, Carol J. Bult, Vivek M. Philip, Elissa J. Chesler*



In [2]:
# @title 2. PMC Trace-Finder (Life Sciences & Biology.)
import requests
import time
from IPython.display import Markdown, display

keywords_input = "animal communication"
batch_size = 30
max_batches = 10

def search_europe_pmc():
    words = [w.strip().lower() for w in keywords_input.replace(",", " ").replace("_", " ").split() if w.strip()]
    query = " AND ".join(words) + " AND OPEN_ACCESS:Y"
    print(f"Searching Europe PMC for: {query}...\n")

    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    all_results = []
    cursor = "*"

    for batch_number in range(1, max_batches + 1):
        params = {"query": query, "format": "json", "pageSize": batch_size, "cursorMark": cursor}

        try:
            response = requests.get(base_url, params=params).json()
            items = response.get("resultList", {}).get("result", [])
            cursor = response.get("nextCursorMark")
        except:
            print("Error reaching API.")
            break

        if not items:
            break

        print(f"\n--- BATCH {batch_number} (Europe PMC) ---")
        for i, res in enumerate(items, 1):
            title = res.get("title", "Unknown Title")
            year = res.get("pubYear", "Unknown Year")
            print(f"{(batch_number - 1) * batch_size + i}. {title} ({year})")

            authors = res.get("authorString", "Unknown Authors")
            url = f"https://doi.org/{res.get('doi')}" if res.get("doi") else f"https://europepmc.org/article/MED/{res.get('pmid')}"

            all_results.append({"title": title, "year": year, "authors": authors, "url": url})

        if not cursor or len(items) < batch_size:
            break

        print("\nWaiting 5 seconds...")
        time.sleep(5)

    if all_results:
        md_content = f"### Europe PMC Results ({len(all_results)} fetched)\n\n"
        for r in all_results:
            md_content += f"* **[{r['title']}]({r['url']})** ({r['year']}) <br> 👤 *{r['authors']}*\n\n"
        display(Markdown(md_content))

search_europe_pmc()

Searching Europe PMC for: animal AND communication AND OPEN_ACCESS:Y...


--- BATCH 1 (Europe PMC) ---
1. Compositionality in human and animal communication. (2025)
2. Using Rhythmic Notation and Musical Analysis on Animal Communication: A Case Study on Sperm Whales. (2026)
3. Individual variation in animal communication: from species averages to unique voices. (2025)
4. Not your private tête-à-tête: leveraging the power of higher-order networks to study animal communication. (2024)
5. Overcoming bias in the comparison of human language and animal communication. (2023)
6. Modelling the emergence of synchrony from decentralized rhythmic interactions in animal communication. (2023)
7. Parasite effects on receivers in animal communication: Hidden impacts on behavior, ecology, and evolution. (2023)
8. Optionality in animal communication: a novel framework for examining the evolution of arbitrariness. (2022)
9. Commentary: Why Are no Animal Communication Systems Simple Languages? (2021)
10.

### Europe PMC Results (300 fetched)

* **[Compositionality in human and animal communication.](https://doi.org/10.1007/s10071-025-02010-8)** (2025) <br> 👤 *Klinedinst N.*

* **[Using Rhythmic Notation and Musical Analysis on Animal Communication: A Case Study on Sperm Whales.](https://doi.org/10.1111/nyas.70210)** (2026) <br> 👤 *Davitt M, Eckelberry M, Davitt M, Burchardt LS.*

* **[Individual variation in animal communication: from species averages to unique voices.](https://doi.org/10.1111/brv.70061)** (2025) <br> 👤 *Lombrey A, Fröhlich M.*

* **[Not your private tête-à-tête: leveraging the power of higher-order networks to study animal communication.](https://doi.org/10.1098/rstb.2023.0190)** (2024) <br> 👤 *Iacopini I, Foote JR, Fefferman NH, Derryberry EP, Silk MJ.*

* **[Overcoming bias in the comparison of human language and animal communication.](https://doi.org/10.1073/pnas.2218799120)** (2023) <br> 👤 *Cartmill EA.*

* **[Modelling the emergence of synchrony from decentralized rhythmic interactions in animal communication.](https://doi.org/10.1098/rspb.2023.0876)** (2023) <br> 👤 *Jadoul Y, Ravignani A.*

* **[Parasite effects on receivers in animal communication: Hidden impacts on behavior, ecology, and evolution.](https://doi.org/10.1073/pnas.2300186120)** (2023) <br> 👤 *Reichert MS, Bolek MG, McCullagh EA.*

* **[Optionality in animal communication: a novel framework for examining the evolution of arbitrariness.](https://doi.org/10.1111/brv.12882)** (2022) <br> 👤 *Watson SK, Filippi P, Gasparri L, Falk N, Tamer N, Widmer P, Manser M, Glock HJ.*

* **[Commentary: Why Are no Animal Communication Systems Simple Languages?](https://doi.org/10.3389/fpsyg.2021.763445)** (2021) <br> 👤 *Wacewicz S.*

* **[Commentary: Why Are No Animal Communication Systems Simple Languages?](https://doi.org/10.3389/fpsyg.2021.722685)** (2021) <br> 👤 *Penn DJ, Számadó S.*

* **[Why Are No Animal Communication Systems Simple Languages?](https://doi.org/10.3389/fpsyg.2021.602635)** (2021) <br> 👤 *Beecher MD.*

* **[Brevity is not a universal in animal communication: evidence for compression depends on the unit of analysis in small ape vocalizations.](https://doi.org/10.1098/rsos.200151)** (2020) <br> 👤 *Clink DJ, Ahmad AH, Klinck H.*

* **[The "Law of Brevity" in animal communication: Sex-specific signaling optimization is determined by call amplitude rather than duration.](https://doi.org/10.1002/evl3.147)** (2019) <br> 👤 *Demartsev V, Gordon N, Barocas A, Bar-Ziv E, Ilany T, Goll Y, Ilany A, Geffen E.*

* **[Taking turns: bridging the gap between human and animal communication.](https://doi.org/10.1098/rspb.2018.0598)** (2018) <br> 👤 *Pika S, Wilkinson R, Kendrick KH, Vernes SC.*

* **[Emotional and Interactional Prosody across Animal Communication Systems: A Comparative Approach to the Emergence of Language.](https://doi.org/10.3389/fpsyg.2016.01393)** (2016) <br> 👤 *Filippi P.*

* **[Animals as Communication Partners: Ethics and Challenges in Interspecies Language Research.](https://doi.org/10.3390/ani16030375)** (2026) <br> 👤 *Mamzer H, Kuchtar M, Grzegorzewski W.*

* **[Testing a key assumption in animal communication: between-individual variation in female visual systems alters perception of male signals.](https://doi.org/10.1242/bio.028282)** (2017) <br> 👤 *Ronald KL, Ensminger AL, Shawkey MD, Lucas JR, Fernández-Juricic E.*

* **[The Complexity of Communication in Mammals: From Social and Emotional Mechanisms to Human Influence and Multimodal Applications.](https://doi.org/10.3390/ani16020265)** (2026) <br> 👤 *Górski K, Kondracki S, Kępka-Borkowska K.*

* **[Helicobacter pylori and Non-Helicobacter pylori Helicobacter (NHPH) Zoonotic Infections: A Survey Among Greek Veterinarians Aiming to Enhance Communication](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Fragkiadaki E, Cortez Nunes F, Linou M, Martinez-Gonzalez B, Sgouras D, Mateus T.*

* **[Strengthening translational preclinical research through confirmatory multi-laboratory studies.](https://doi.org/10.3389/fmed.2025.1715361)** (2025) <br> 👤 *Rotter SC, Arroyo-Araujo M, Drude NI, Pellegrini P, Kobold S, Richter GHS, Müller OJ, Bruder D, Riecken LB, Gerlach B, Schuler L, Salbach-Hirsch J, Dalloul I, Kühn S, Wilcke JC, Toelch U.*

* **[Multi-model large-scale AI framework for avian influenza surveillance and preparedness: Harnessing large language models to enhance risk communication, real-time decision support, and public health response strategies.](https://doi.org/10.1016/j.onehlt.2026.101357)** (2026) <br> 👤 *Kong JD, Gillies M, Gardner E, Bragazzi NL.*

* **[Haptic Signals as a Communication Tool Between Handlers and Dogs: Review of a New Field.](https://doi.org/10.3390/ani16020323)** (2026) <br> 👤 *Jean-Joseph H, Bovet D.*

* **[Communication Skills Training in Veterinary Education: A Scoping Review of Programs and Practices.](https://doi.org/10.3390/vetsci13010063)** (2026) <br> 👤 *López-López V, Poblete Hormazábal M, Cofré González S, Sepúlveda Pérez C, Muñoz Pérez C, Zapata Lamana R.*

* **[Caretakers at the core: exploring worker perceptions of job satisfaction, training, cattlecare, and workplace community on feedyards in the United States.](https://doi.org/10.1093/jas/skaf400)** (2026) <br> 👤 *Sullivan PA, Bigler L, Cramer MC, Roman-Muniz IN, Edwards-Callaway LN.*

* **[ASAS-NANP SYMPOSIUM: mathematical modeling in animal nutrition: the importance of animal science flight simulators to enhance the competitiveness and sustainability of livestock production.](https://doi.org/10.1093/jas/skaf440)** (2026) <br> 👤 *Menendez Iii HM, Chang Y, Palmer E, La Manna F, Moreno ERV, Parsons I, Turner BL, Moradi Rekabdarkolaee H, Husmann AL, Tedeschi LO.*

* **[Applying Cognitive Stimulation Therapy (CST) on People with Concurrent Visual Impairment and Dementia: A Preliminary Study](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Tsang H, Luk C, Lo Y, Chiu A, Yip B, Wong W.*

* **[Communication and involvement of students and young researchers in animal science: an European perspective.](https://doi.org/10.1093/af/vfaf036)** (2025) <br> 👤 *Morosan S, Riva F, Grignaschi G, Chourbaji S.*

* **[Experimental Validation of a Battery-Free RFID-Powered Implantable Neural Sensor and Stimulator.](https://doi.org/10.3390/s26030954)** (2026) <br> 👤 *Pedigoni Bulisani LE, Herculano MA, Pauris CC, do Prado LRB, Sakai LJ, Júnior FMP, Marchi E.*

* **[Amniotic Mesenchymal Stromal/Stem Cell-Derived Extracellular Vesicles for Equine Chronic Degenerative Endometritis Treatment.](https://doi.org/10.1002/vms3.70685)** (2026) <br> 👤 *Gaspari G, Funghi F, Cantile C, Camillo F, Panzani D, Maltinti S, Fanelli D, Moroni R, Cremonesi F, Gagni P, Lange-Consiglio A.*

* **[Public perception of genetically modified organisms (GMOs) in Montenegro: insight for sustainable biotechnology and policy development.](https://doi.org/10.1080/21645698.2026.2620884)** (2026) <br> 👤 *Velimirović A, Jovović Z, Berjan S, El Bilali H, Allahyari MS.*

* **[Robust ISAC based framework for location estimation and target detection in 6G networks.](https://doi.org/10.1371/journal.pone.0337050)** (2026) <br> 👤 *Soni L, Taneja A, Alqahtani N, Alqahtani J.*

* **[Frequency-dependent cerebellar circuits independently gate social vocalizations and movement](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Unknown Authors*

* **[Building trust before the next crisis: lessons from the avian influenza front lines.](https://doi.org/10.3389/fpubh.2025.1735139)** (2025) <br> 👤 *Togami E, Guo EA, Liao J, Ostroff S, Sleeman JM, Mumford EL.*

* **[Beyond clinical skills: student-reported impacts of a veterinary public health externship in rural Alaska.](https://doi.org/10.3389/fvets.2025.1613867)** (2025) <br> 👤 *Meythaler-Mullins L, Lobdell AH, Kern-Allely C, Frey DM.*

* **[FELASA Congress 2025.](https://doi.org/10.1002/ame2.70133)** (2025) <br> 👤 *Wei L.*

* **[Sustainable Pet Diets: A Leading Effective Altruism Issue.](https://doi.org/10.3390/ani16030460)** (2026) <br> 👤 *Knight A.*

* **[The Zoonosis Council: establishing clarity on zoonotic diseases for the general public.](https://doi.org/10.1093/af/vfaf037)** (2025) <br> 👤 *Sanders L, Haydon MC, Messman R.*

* **[Parents' Perspectives on the Benefits of Animal-Assisted Intervention: A Systematic Review.](https://doi.org/10.3390/bs15121663)** (2025) <br> 👤 *González-Sala F, Llopiz-Guerra K, Ferri A, Martí-Vilar M.*

* **[From concept to action: Operationalising animal welfare strategies in zoos and aquaria.](https://doi.org/10.1017/awf.2025.10053)** (2025) <br> 👤 *Partoon JK, Sherwen S, Smith MFL, Riley L, Whittaker A.*

* **[The Tape of Life: A Musical Activity Engaging the Public with Aging Research](https://europepmc.org/article/MED/None)** (2025) <br> 👤 *Unknown Authors*

* **[Navigating the challenges in implementing financial incentives for behavior change at the intersection of human, animal, and ecosystem health: a case study.](https://doi.org/10.1016/j.soh.2025.100144)** (2026) <br> 👤 *Kerr JM, Delea MG, Chung M, Zhao J, Crawford J, Lapinski MK.*

* **[A survey on opinions, perceptions and attitudes of Swedish veterinarians-the Swedish veterinary disciplinary board.](https://doi.org/10.3389/fvets.2025.1732118)** (2025) <br> 👤 *Egenvall A, Enlund KB, Holmquist T, Rosenquist M.*

* **[SWOT analysis on veterinary telemedicine from pet owner and expert perspectives-a mixed-methods survey and interview study in Germany.](https://doi.org/10.3389/fvets.2026.1731577)** (2026) <br> 👤 *Karsch C, Tipold A, Kleinsorgen C.*

* **[Prioritization of zoonotic diseases in Bhutan through a One Health framework.](https://doi.org/10.1186/s42522-026-00198-4)** (2026) <br> 👤 *Penjor K, Dahal N, Tshering U, Dorji C, Lungten L, Badhur J, Yangzom D, Dorjee S, Dukpa K, Wangdi K, Aye YM, Tenzin T.*

* **[Tracking spread of carbapenemase-producing Enterobacterales between humans and companion animals: successes and challenges.](https://doi.org/10.3389/fcimb.2025.1730592)** (2025) <br> 👤 *Dietrich JM, Vagnone PMS, Dale JL, Beaudoin AL, Ruhland A, Kollmann L, Cole SD.*

* **[Tailored biosecurity training for veterinarians and farmers: bridging knowledge and practice gaps.](https://doi.org/10.3389/fvets.2025.1643029)** (2025) <br> 👤 *Mehmedi B, Niemi J, Saegerman C, De Meneghi D, Iatrou AM, Yildiz R, Chantziaras I, Allepuz A, Toppari I, Batikas G, Viltrop A, Niine T.*

* **[One Biology and the Status of Humans.](https://doi.org/10.3390/ani16010086)** (2025) <br> 👤 *Broom DM.*

* **[Passengers as Pathways: Behavioral Evidence on Travelers' Knowledge of African Swine Fever Introduction Through Pork Products.](https://doi.org/10.3390/vetsci13020194)** (2026) <br> 👤 *Mandas D, Murgia G, Usai K, Bazzardi R, Muroni G, Cappai S, Coccollone A, Loi F.*

* **[Participation of livestock farmers and community animal health workers in one health intervention to control zoonotic diseases in Rwanda: a qualitative study.](https://doi.org/10.1186/s12879-026-12696-x)** (2026) <br> 👤 *Hugor S, Izudi J, Shyaka A.*

* **[Colonial signature of the alarm pheromone and chemical differences between hornet workers.](https://doi.org/10.1371/journal.pone.0336261)** (2026) <br> 👤 *Haouzi M, Bastin F, Perdereau E, Humbert C, Play B, Beaugeard L, Darrouzet É.*

* **[The Vascular-Immune Cells Crosstalk and Microenvironment in Pulmonary Hypertension.](https://doi.org/10.1002/cph4.70110)** (2026) <br> 👤 *Navarro-Llinares L, García-León B, de la Bastida-Casero L, Oliver E.*

* **[The effect of cuproptosis-related gene Atox1 on cardiac hypertrophy in heart failure was investigated based on bioinformatics analysis.](https://doi.org/10.1038/s41598-025-28342-6)** (2025) <br> 👤 *Zhou H, Yang M, Jin H, Ye Z.*

* **[To treat or not to treat: Experiences and considerations of veterinarians in management of cats with diabetes mellitus.](https://doi.org/10.1371/journal.pone.0341762)** (2026) <br> 👤 *Rothlin-Zachrisson N, Ström Holst B, Öhlund M, Leksell J, Röcklinsberg H.*

* **[Canine-Assisted Speech Therapy for Children: Caregiver and Therapist Perspectives.](https://doi.org/10.3390/healthcare13243299)** (2025) <br> 👤 *Zhang ZY, Driscoll C, Hill J, Howell T, Ward G.*

* **[Veterinary clients value animal welfare and environmental sustainability in pet food choices.](https://doi.org/10.3389/fvets.2026.1735331)** (2026) <br> 👤 *Clark M, Scott D, Kern-Allely C, Kohen A, Puaa KK, Delcambre J, Duncan C.*

* **[Advancing the One Health Framework in EU Plant Protection Product Regulation: Challenges and Opportunities.](https://doi.org/10.3390/jox15060200)** (2025) <br> 👤 *Calliera M, Capri E, Suciu NA, Trevisan M.*

* **[A novel inhaled anesthesia technique for concurrent direct laryngoscopy in rats.](https://doi.org/10.1002/ame2.70116)** (2025) <br> 👤 *Negvesky C, Mirzaaghasi A, Smith EM, Kita AE.*

* **[Assessing positive selection in centromere-associated kinetochore proteins across Metazoan groups](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Unknown Authors*

* **[Gut Microbiota Dysbiosis in Depression: Pathological Correlations, Molecular Pathways, and Therapeutic Interventions.](https://doi.org/10.3390/ijms27031530)** (2026) <br> 👤 *Cao J, Ma J, Zha X, Bian X, Wang W, Liu X.*

* **[Computed Tomographic Grading of Ear Diseases in Guinea Pigs (Cavia porcellus).](https://doi.org/10.1111/vru.70143)** (2026) <br> 👤 *Brennan T, Beauvois M, Pignon C, Coutant T, Muffat-Es-Jacques P, Penel E, Gomes E, Huynh M, Hahn H.*

* **[Faces of time: a historical overview of rapid innovations in coding animal facial signals.](https://doi.org/10.3389/fvets.2025.1716633)** (2025) <br> 👤 *Lazebnik T, Florkiewicz B.*

* **[Fostering culture of care for early career researchers-Building a trustful environment: Insights from a German perspective.](https://doi.org/10.1371/journal.pone.0331675)** (2025) <br> 👤 *Gonzalez-Uarquin F, Ferrara F, Baumgart N, Baumgart J, Bischoff SJ.*

* **[Single-cell transcriptional profiling reveals developmental dynamics of longissimus dorsi muscle in adult and Juvenile bactrian camels.](https://doi.org/10.3389/fphys.2025.1697432)** (2025) <br> 👤 *De B, Liu Y, Li L, Meng F, Liu Y, Jia T, Chen Y, Liu C, Wang S, Li T, Xiao H, Wan F, Liu Y, Wang W, Zhou H, Zhang W, Wen X, Gong J, Naer A, Bao H, Song Q, Chaolu H, Long H, Zhang Y, Cao J.*

* **[Toward mass customization of animal trackers by design automation.](https://doi.org/10.1371/journal.pone.0342071)** (2026) <br> 👤 *Beutler P, Gregersen T, Roth J, Richter N, Meboldt M, Wikelski M, Wild TA.*

* **[Utilizing the Style Under Stress™ tool within Crucial Conversations© methodology to stimulate veterinary students' reflective practice of communication behaviors during client interactions.](https://doi.org/10.3389/fvets.2026.1746568)** (2026) <br> 👤 *Bagley RS, Mindthoff A.*

* **[Human emotional odours influence horses' behaviour and physiology.](https://doi.org/10.1371/journal.pone.0337948)** (2026) <br> 👤 *Jardat P, Destrez A, Damon F, Tanguy-Guillo N, Lainé AL, Parias C, Reigner F, Ferreira VHB, Calandreau L, Lansade L.*

* **[The Scientific Case for Animal Models: A Perspective From Musculoskeletal Researchers.](https://doi.org/10.1096/fba.2025-00313)** (2026) <br> 👤 *Hadjiargyrou M, Hildreth BE, Ko F, Sankar U, Yang T.*

* **[An open-source bio-logger for studying cetacean behavior and communication.](https://doi.org/10.1371/journal.pone.0337093)** (2025) <br> 👤 *Vogt DM, DelPreto J, Salino-Hugg M, Cummings MR, Bell MA, Kenny A, Malkin P, Hernandez AM, Wright AL, Duncan MA, Davidsen MR, Grewal K, Ahuja S, Ostrom K, Garcia M, Shahviri P, Weeraratna C, Zink H, Pagani S, Meyer G, Diamant R, Rus D, Gruber DF, Gero S, Wood RJ.*

* **[Facilitating Communication With Children and Young Adults With Special Health Care Needs Through a Web-Based Application: Qualitative Descriptive Study.](https://doi.org/10.2196/76512)** (2026) <br> 👤 *Hanks JR, Hughes AM, Sultana S, Klute R, Formella K, Flynn C, Wallenfang A, Krishnakumar D, Mourad M, Morse YC, Mischler MJ.*

* **[Editorial: Intercellular communication in chronic neuroinflammatory diseases.](https://doi.org/10.3389/fncel.2026.1791491)** (2026) <br> 👤 *Potokar M, Jorgačevski J, Matsuse D, Vicario N.*

* **[One health, one earth, one life: The overlooked role of veterinarians in the fight against COVID-19 and other public health emergencies in Italy.](https://doi.org/10.1016/j.onehlt.2025.101207)** (2025) <br> 👤 *Roperto S, Di Guardo G.*

* **[Research Priorities for Zoonotic and Pandemic Influenza Vaccines: Evidence and Recommendations from the WHO Public Health Research Agenda for Influenza (2024 Update).](https://doi.org/10.3390/vaccines13121206)** (2025) <br> 👤 *Zhang W, Cowling BJ, Tam JSL, Abraham T, Chen H, Duggal K, Khong WX, Maurer-Stroh S, Monto AS, Nikisins S, de Oliveira T, Shu Y, Viboud C, Webby R, van der Werf S, Wong J, Heraud JM.*

* **[Harnessing Medical Bioethics Mediation to Advance One Health Governance.](https://doi.org/10.3390/vetsci13010008)** (2025) <br> 👤 *Lioupi O, Kostoulas P, Monti G, Griva K, Billinis C, Tsiamis C.*

* **[Connecting the Dots: Systems Thinking Highlights the Role of the Environment Sector in One Health Operationalization.](https://doi.org/10.1002/gch2.202500256)** (2025) <br> 👤 *Hansen A, Chaber AL, Reid S, Milazzo A.*

* **[The Role of Swine Gut Microbiota and Its Metabolites in Maintaining Intestinal Barrier Integrity and Mitigating Stress via the Gut-Brain Axis.](https://doi.org/10.3390/ani15243653)** (2025) <br> 👤 *Woś K, Pachciński K, Wacko M, Koszła O, Sołek P, Czech A.*

* **[One health and community health workers: Bridging the gap in global health resilience.](https://doi.org/10.1016/j.onehlt.2026.101330)** (2026) <br> 👤 *Muvunyi CM, Gashema P, Siddig EE, de Dieu Harelimana J, Kaseya J.*

* **[Rabies post-exposure prophylaxis in India: a SWOT analysis.](https://doi.org/10.1177/25151355251410780)** (2026) <br> 👤 *Pardeshi G, Sharma P, Ittiel A.*

* **[Who, What, Where, When, Why, and How (5W1H)-Based Education of Informed Consent for Intravenous Thrombolysis in Acute Ischemic Stroke.](https://doi.org/10.7759/cureus.101137)** (2026) <br> 👤 *Wang Y.*

* **[Exosomes: A Comprehensive Exploration of their Beneficial, Detrimental, and Ambiguous Roles in Intercellular Communication and Pathophysiological Processes - Do They Justify the Enthusiasm?](https://doi.org/10.4103/ijd.ijd_407_25)** (2026) <br> 👤 *Kapoor R, Bellani D, Patil R, Gold M, Shome D.*

* **[Spatiotemporal dynamics of neuronal subtypes and their interactions with glia following intracortical electrode implantation.](https://doi.org/10.1186/s13062-025-00719-7)** (2025) <br> 👤 *Zhao Z, Duan X, Luo J, He Z, Zhang Y, Wang M, Qin J, Lin S, Chen H.*

* **[Single-nucleus and spatial transcriptomics reveal intestinal cellular heterogeneity, differentiation, and cell communication mechanisms in SAP-induced intestinal injury.](https://doi.org/10.3389/fimmu.2026.1719902)** (2026) <br> 👤 *Zhang L, Xu C, Su T, Xiao T, Yi X, Feng Y, Wang J, Zhao S.*

* **[Meows encode less individual information than purrs and show greater variability in domestic than in wild cats.](https://doi.org/10.1038/s41598-025-31536-7)** (2025) <br> 👤 *Russo D, Schild AB, Knörnschild M.*

* **[Gandouling Inhibits the Sinusoid Capillarization Associated with Liver Fibrosis in Wilson’s Disease by Blocking the Communication Between Hepatic Stellate Cells and Liver Sinusoidal Endothelial Cells](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Cai Y, Jin Q, Yuan M, Zhou X, Wu Y, Song Y, Wang B, Miao C, Wu P.*

* **[Cuproptosis-driven reprogramming of fibroblast communication by GK is associated with the immune microenvironment in diabetic foot ulcers.](https://doi.org/10.3389/fimmu.2025.1687806)** (2025) <br> 👤 *Li T, Wang J, Gao L.*

* **[Single-cell transcriptomics reveals diversity and conservation of chicken lymphocytes.](https://doi.org/10.1016/j.psj.2026.106475)** (2026) <br> 👤 *Liu H, Tu J, He X, Ouyang Q, Zhang H.*

* **[The role of prefrontal-hippocampal functional connectivity in schizophrenia-related cognitive dysfunction and the thalamic ventral midline involvement: &lt;i&gt;in vivo&lt;/i&gt; and &lt;i&gt;silico&lt;/i&gt; evidence.](https://doi.org/10.3389/fnins.2025.1653828)** (2025) <br> 👤 *Černousova A, Patrono E.*

* **[A Regional Audit on Venous Thromboembolism (VTE) Prophylaxis Compliance and Cultural Considerations in Prescribing Porcine-Derived Low-Molecular-Weight Heparins.](https://doi.org/10.7759/cureus.100617)** (2026) <br> 👤 *Mujahid M, Ahmed H, Ahmad S, Mujahid I, Ur-Rehman J, Maheshwari G, Norouzi M, Prudon B.*

* **[What's in a Name? Not All Mesopredators Are Mesocarnivores.](https://doi.org/10.1002/ece3.72768)** (2025) <br> 👤 *Madsen EK, Eckersley L, Linden JF, Lai S, Hare D, Macdonald DW, Kimaili D, Kulunge S, Mutinhuma Y, Petracca L, Rono BJ, Sibanda L, Sillero-Zubiri C, Tacey J, Broekhuis F.*

* **[Implementing Culture of Care in Germany.](https://doi.org/10.3390/ani15192918)** (2025) <br> 👤 *Ameli K, Krämer S.*

* **[In vitro culture of cumulus oocyte-complexes from early antral follicles in prepubertal and adult ewes.](https://doi.org/10.1530/raf-25-0102)** (2025) <br> 👤 *Ebrahimi M, Mara L, Succu S, Gadau SD, Chessa F, Dattena M, Luciano AM, Berlinguer F.*

* **[&lt;i&gt;The Botucatu Document&lt;/i&gt;: One health antifungal resistance policies - A call for action.](https://doi.org/10.1016/j.onehlt.2026.101319)** (2026) <br> 👤 *Ceresini PC, de Jesus Junior WC, Firmino AC, Silva BPC, Stipp DT, Furtado EL, Mingotte FLC, Costa KA, Lopes PRM, Peruquetti RL.*

* **[Integrative single-cell and machine-learning analysis identifies ac&lt;sup&gt;4&lt;/sup&gt;C-related S100A13 as a causal risk gene in cholangiocarcinoma.](https://doi.org/10.1186/s12885-026-15602-5)** (2026) <br> 👤 *Zheng Y, Lin Y, Wang Z, Wu F.*

* **[Healthcare Providers' Perspectives on Patient-Healthcare Worker Communication in the Emergency Department: A Cross-Sectional Survey Study in Riyadh, Saudi Arabia.](https://doi.org/10.7759/cureus.101434)** (2026) <br> 👤 *Aburiyah A, Alsaeedi A, Aljehaiman TA, Alotaibi G, Alshehri R.*

* **[Challenges and new opportunities in deciphering the meaning of corvid call sequences.](https://doi.org/10.1007/s10071-025-02015-3)** (2025) <br> 👤 *Salis A, Martin K, Girard-Buttoz C.*

* **[The 2025 FELASA Congress in Athens: From Aristotle to AI.](https://doi.org/10.1038/s41684-025-01651-z)** (2026) <br> 👤 *Franco NH, Kalliokoski O, Voikar V.*

* **[Non-prescription dispensing of veterinary medicines for treating mastitis in dairy cattle among non-veterinary personnel in selected districts of Zambia.](https://doi.org/10.3389/fvets.2025.1707434)** (2025) <br> 👤 *Bwalya A, M'kandawire E, Pyatt A, Prince C, Gilbert C, Muma JB, Mudenda S.*

* **[G. T. Fechner (1848): Plants as sentient living beings.](https://doi.org/10.1080/15592324.2026.2632571)** (2026) <br> 👤 *Parovel G.*

* **[Prevention as a Pillar of Communicable Disease Control: Strategies for Equity, Surveillance, and One Health Integration.](https://doi.org/10.3390/epidemiologia7010019)** (2026) <br> 👤 *Genovese G, Rizzo CE, Bartucciotto L, Calderone SM, Loddo F, Leonforte F, Mistretta A, Squeri R, Genovese C.*

* **[Cultural‐linguistic divergences in animal fluency performance between Korean‐ and English‐speaking older adults with Alzheimer's disease](https://europepmc.org/article/MED/None)** (2025) <br> 👤 *Shin J, Garcia A, Scimeca M, Kiran S, Sung J.*

* **[Quality Improvement Project for the Management of per Vaginal (PV) Bleeding in Early Pregnancy: Evaluating the Effectiveness of Patient Information Leaflets.](https://doi.org/10.7759/cureus.98922)** (2025) <br> 👤 *Hussain S, Kittappa K.*

* **[Distinct oxytocin populations support seismic social communication and militant behaviors in a naturally solitary wild rodent.](https://doi.org/10.1016/j.isci.2025.114151)** (2025) <br> 👤 *Zilkha N, Hamade R, Chuartzman SG, Sofer Y, Kashash Y, Meir R, Fishman R, Biton IE, Ben-Dor S, Kimchi T.*

* **[Integrative transcriptomic analysis reveals microglial metabolic-inflammatory crosstalk of HK2-HSPA5-TNF axis after intracerebral hemorrhage.](https://doi.org/10.3389/fbinf.2025.1740715)** (2025) <br> 👤 *Zhang Y, Liu Y, Meng W, Yu X, Xu X.*

* **[Trauma-informed veterinary practice: linking emotional labor, moral distress, and occupational well-being.](https://doi.org/10.3389/fvets.2026.1746939)** (2026) <br> 👤 *Serim-Yıldız B, Onaylı S, Ilgın HE.*

* **[Potential role of exosomes derived from the tumor microenvironment in tumor progression and treatment.](https://doi.org/10.3389/fonc.2025.1583295)** (2025) <br> 👤 *Yu Y, Zhang W, Wang K.*

* **[Ebola Virus Disease Preparedness in Subnational Health Systems: A Readiness Assessment of Jinja District, Uganda.](https://doi.org/10.1155/jotm/5519966)** (2026) <br> 👤 *Oposhia J, Kungu JM, Nantamu PD, Namayanja J, Okuyo CAB, Mulowoza M, Kabali K, Edson K, Olupot-Olupot P.*

* **[Hippocampal-Prefrontal Communication Subspaces Align with Behavioral and Network Patterns in a Spatial Memory Task.](https://doi.org/10.1523/eneuro.0336-24.2025)** (2025) <br> 👤 *Young RA, Shin JD, Guo Z, Jadhav SP.*

* **[Advancing one health implementation in the Eastern Mediterranean region: insights from the EMPHNET 8th biennial roundtable.](https://doi.org/10.3389/fpubh.2025.1652846)** (2025) <br> 👤 *Himatt S, Aziz MA, ElDesouki B, Muehlen M, Malkawi M, Pinto J, Al Nsour M, Khader Y, Bashier H, Adam S, Al Faouri I.*

* **[The Bone–Brain Axis: Novel Insights into the Bidirectional Crosstalk in Depression and Osteoporosis](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Li P, Gao Y, Zhao X.*

* **[Consumer Perspectives on Antibiotic-Free Animal Products: A Systematic Review Identifying Critical Gaps in Non-Pharmaceutical Intervention Research.](https://doi.org/10.3390/ani16010070)** (2025) <br> 👤 *Hussain SA, Abbas SR, Lee SW.*

* **[Single-cell atlas reveals cellular heterogeneity and BMP5-mediated regulation of adipogenic differentiation in sheep adipose tissue.](https://doi.org/10.1038/s42003-026-09581-3)** (2026) <br> 👤 *Cheng J, Han K, Xu D, Tian H, Zhang X, Zhao L, Yang X, Zhang D, Huang K, Zhang Y, Zhao Y, Li X, Xu Q, Ma Z, Wu W, Wang J, Li F, Wang W.*

* **[A Novel Interprofessional Educational Experience for Internal Medicine Residents Within the Clinical Learning Environment.](https://doi.org/10.7759/cureus.100207)** (2025) <br> 👤 *Donato J, Montane B, Luo C, Foshee C.*

* **[Epidemiological indicators of accidental laboratory-origin outbreaks.](https://doi.org/10.1017/s0950268825100915)** (2026) <br> 👤 *Dhawan S, Pan-Ngum W, MacIntyre CR, Blacksell SD.*

* **[Artificial Intelligence in Rhinoplasty Recovery: Linguistic Intelligence and Machine Learning-Driven Insights](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Aliyeva A, Azizli E, Snyder V, Muradova A, Ahmadov N, Muderris T, Hashimli R, Erbek S, Hepkarsi S, Dalgic A.*

* **[The 2025-2030 US Dietary Guidelines: an analysis of scientific integrity and global health governance.](https://doi.org/10.1016/j.lana.2026.101402)** (2026) <br> 👤 *Neves FS, Nilson EAF, Mendes LL, Khandpur N, Nestle M.*

* **[Single-Cell Transcriptomic Characterization of DNCB-Induced Mouse Model Reveals Atopic Dermatitis-Associated Skin Lesions in Skin Microenvironment.](https://doi.org/10.1007/s10753-025-02391-5)** (2026) <br> 👤 *Liu W, Qiu Z, Fu J, Hou L, Ding X, Du H, Zhai Y, Cao Z, Wang P, Wang C.*

* **[Maintaining sexual identity in adult flies is an ongoing commitment.](https://doi.org/10.1371/journal.pbio.3003530)** (2025) <br> 👤 *Kopp A.*

* **[Patients' Perception Regarding the Communicative Capabilities of Portuguese Family Doctors: A Cross-Sectional Study.](https://doi.org/10.7759/cureus.100336)** (2025) <br> 👤 *Gonçalves CA, Alves CF, Alves CS, Coutinho R, Rocha L, Moreira B, Seabra P.*

* **[Single-cell analysis reveals cellular heterogeneity and molecular features during postnatal pig testis development.](https://doi.org/10.1186/s12864-025-12280-8)** (2025) <br> 👤 *Wang L, Ao H, Liu H, Zhao Q, Ma F, Yang J, Wang J, Wang X, Liu G, Niu X, Zheng Z, Wang C, Xing K.*

* **[Examination of the Communication Environment When Applying Removable Dentures as Wearable Devices.](https://doi.org/10.7759/cureus.97531)** (2025) <br> 👤 *Iwata Y, Yanagizono Y, Iida T, Ashida Y, Yamada Y, Asano T, Suzuki H, Tsuge T, Komiyama O, Niitsu K.*

* **['PULSE FICTION': Development of Slightly Processed Pulse-Based Foods and Recipes to Meet the Needs of Consumers and the Agricultural Sector and Improve Food Sustainability.](https://doi.org/10.1111/nbu.70041)** (2026) <br> 👤 *Arvisenet G, Chambaron S, Hernandez-Casiano Z, Gerard-Simonin H, Tanguy C, Hugol-Gential C, Saint-Eve A, Salles C.*

* **[PTSD assistance dogs: concerns for animal well-being, rights, and justice.](https://doi.org/10.3389/fvets.2025.1658857)** (2025) <br> 👤 *Kiiroja L, Gadbois S, Fenton A.*

* **[A Nationwide Cross-Sectional Survey of Knowledge, Attitudes, and Practices on Rabies in Saudi Arabia.](https://doi.org/10.3390/tropicalmed11020055)** (2026) <br> 👤 *Bakhsh E, Doumi R, Alqahtani N, Althubiti S, Hagr J, Alnujide A, Alobaid S, Allaboon J, Alotaibi S, Aljuhaymi D, Alotaibi M, Assiri A.*

* **[Bridging gaps in veterinary care: restructuring a community clinic for cultural inclusion in Phoenix, Arizona.](https://doi.org/10.3389/fvets.2025.1595312)** (2025) <br> 👤 *Master CD, Vincent MR, Twomey BA, Vazquez C, Robertson JV.*

* **[Unpacking PCOS Inflammation: From Misconceptions to Immune Networks.](https://doi.org/10.1210/endocr/bqaf166)** (2025) <br> 👤 *De Robles G, Wiggins KD, Del Mundo Z, Ujagar N, Nguyen CM, Seldin MM, Pacheco-Sanchez G, Nicholas DA.*

* **[Deciphering immune features and cellular heterogeneity in PRRSV infection via single-cell RNA sequencing.](https://doi.org/10.1128/jvi.01828-25)** (2026) <br> 👤 *Li J, Liang Y, Zhang Y, Xu Y, Liu F, Ding L, Wang Y, Zhang Z, Chen Z, Sun W, Yu J, Wu J.*

* **[Advances in Animal Disease Surveillance and Information Systems and Their Role in Disease Control and Prevention: Implications in Ethiopia.](https://doi.org/10.1002/vms3.70701)** (2025) <br> 👤 *Engdawork A, Negussie H.*

* **[Osteosarcopenia: key molecular mechanisms and translational perspectives.](https://doi.org/10.3389/fphys.2025.1723522)** (2025) <br> 👤 *Pu Y, Teng Y, Li Y, Zhou Y, Gao M, Yan Z, Teng Z.*

* **[Clinical Reasoning Uncertainty in Veterinary Medical Encounters with a Clinical Example.](https://doi.org/10.3390/vetsci12121203)** (2025) <br> 👤 *Petrovski KR, Kirkwood RN.*

* **[Single-cell transcriptomics reveals mechanisms of &lt;i&gt;Galt&lt;/i&gt; gene editing-induced liver injury involving HGF-VEGF-mediated intercellular signaling in mice.](https://doi.org/10.3389/fcell.2025.1729321)** (2025) <br> 👤 *Li Z, Wang N, Ruan H, Li Q, Zhang X, Liu N, Li Y, Gu L, Yue P, Yu H.*

* **[Race and Patient-Physician Communication on Blood Pressure Management: Helping Improve Care.](https://doi.org/10.1007/s11886-025-02294-9)** (2026) <br> 👤 *Nasser SA, Pender A, Ali A, Tobb K, Ferdinand KC.*

* **[Capturing vocal communication in a free-living corvid: high-resolution data from low-impact miniaturized tags.](https://doi.org/10.1007/s10071-025-02018-0)** (2025) <br> 👤 *Baglione V, Canestrari D, Cusimano M, Hoffman B, Moreno V, Trapote E.*

* **[Hypo-glycosylated FSH enhances the ovarian microenvironment for follicular development compared to fully glycosylated FSH.](https://doi.org/10.1186/s12964-025-02576-3)** (2025) <br> 👤 *Tang J, Wu Z, Liu M, Xu L, Cheng J, Wang C, Zhu X, Zhou X, Yang L, Davis JS, Hua G.*

* **[German Cat and Dog Owners' Views on Veterinary Error Handling: Communication and Transparency Concerns from Qualitative Interviews.](https://doi.org/10.3390/ani15202981)** (2025) <br> 👤 *Johann VK, Busse C, Volk HA, Kleinsorgen C.*

* **[Revitalizing GIP: Therapeutic Potential in Metabolic and Neurodegenerative Disorders.](https://doi.org/10.2147/dmso.s559587)** (2026) <br> 👤 *Qiao Y, Zhou F, Mao T, Gao L.*

* **[IL-18 receptor axis in allergic conjunctivitis: a multi-omics study.](https://doi.org/10.3389/fimmu.2025.1704591)** (2025) <br> 👤 *Yuan L, Su W, Li L, Xiang Y, Xu Z, Wang Y, Hua X, Yuan X.*

* **[Construction and validation of a novel diagnostic model for esophageal squamous cell carcinoma: an integrated analysis of multi-omics data](https://europepmc.org/article/MED/None)** (Unknown Year) <br> 👤 *Cui Y, Li S, Wu Z, Jin Y, Yu J, Chen Y, Chen J, Chang J, Yan Y, Li X, Li N, Hu S, Zhu C, Feng L.*

* **[Supporting One Health policies to manage antibiotic resistance in Senegal: a systems analysis using group model building.](https://doi.org/10.3389/fpubh.2025.1689609)** (2025) <br> 👤 *Sow MM, Ciss M, Diouf ND, Fall AG, Dia NM, Penney T, Bordier M, Clifford Astbury C.*

* **[Progress and challenges in pet management in disasters: recommended strategies for developing countries - a scoping review.](https://doi.org/10.1186/s12917-025-05028-9)** (2025) <br> 👤 *Moslehi S, Tavan A, Narimani S, Soleimanpour S.*

* **[Design and development of an affordable multi-mode small animal ventilator.](https://doi.org/10.1038/s41598-025-28241-w)** (2025) <br> 👤 *Dzierzawski P, Flamm B, Hegele V, Spassov S, Wenzel C, Schumann S, Lozano-Zahonero S.*

* **[Donkey-assisted therapy in mental health conditions: a systematic review.](https://doi.org/10.3389/fpsyt.2025.1680983)** (2025) <br> 👤 *Fulgenzi A, Raffa A, De Domenico C, Di Cara M, Leonardi S, Piccolo A, Giambò FM, Marafioti G, Alito A, Leonardi G, Turriziani L, Panzera M, Quartarone A, Cucinotta F.*

* **[Insights into speckled eggs provided by single-cell transcriptomics in high-yield laying hens.](https://doi.org/10.1016/j.psj.2026.106581)** (2026) <br> 👤 *Sun Y, Zhang J, Chen Z, Liu W, Han H, Hao D, Li D, Wang J, Liu J, Lei Q, Zhou Y, Cao D, Wang Y, Li G, Chen F, Li F.*

* **[Comparing training methods to promote better animal welfare outcomes for animals in the research and teaching industry.](https://doi.org/10.1017/awf.2026.10064)** (2026) <br> 👤 *Cohen S, Lomax S, Hemsworth L, Tripovich JS, Slavich E, Brandis KJ, Becker JA.*

* **[Seeking interdisciplinary solutions at the interface of animal, human, plant, and environmental health](https://europepmc.org/article/MED/None)** (2025) <br> 👤 *Unknown Authors*

* **[Perceptions of assisted reproductive technologies in wildlife conservation: Public expectations and ethical implications across three EU countries](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Biasetti P, Hildebrandt T, Seet S, Stejskal J, Giardullo P, Göritz F, Holtze S, Galli C, Šťastný M, de Mori B.*

* **[Encephalitozoonosis in Pet Rabbits: Epidemiology, Pathogenesis, Immunology and Consensus on Clinical Management.](https://doi.org/10.3390/ani16020346)** (2026) <br> 👤 *Keeble E, Kϋnzel F, Montiani-Ferreira F, Graham J, Jeklová E, Kanfer S, Lennox A, Desoubeaux G, Biswell E, Cray C, Joachim A.*

* **[From commitment to action: lessons from 18 transdisciplinary projects addressing health risks in wildlife trade.](https://doi.org/10.1186/s42522-025-00190-4)** (2026) <br> 👤 *Li H, Henrichfreise B, Keil J, Nagel M, Stephen C, Reaser JK.*

* **[Editorial: The growing problem of free-roaming dogs: a one health perspective on public and animal health.](https://doi.org/10.3389/fvets.2025.1743386)** (2025) <br> 👤 *Viozzi G, Flores V, Sánchez Thevenet P.*

* **[A bounded hierarchy framework for the evolution of syntax.](https://doi.org/10.1007/s10539-025-09998-w)** (2025) <br> 👤 *Palazzolo G.*

* **[Strengthening intersectoral cooperation for control and integrated surveillance of rabies in Uganda: A multistakeholder engagement workshop.](https://doi.org/10.1016/j.onehlt.2025.101206)** (2025) <br> 👤 *Ghimire R, Okech SG, Fahrion AS, Odoch T, Matern J, Bohnes T, Vielle NJ, Muturi M, Wiethoff JP, Nakanjako MF, Lohr F, Ung P, Tann C, Alubbe SP, Ademun AR, Opolot J, Monje F, Semanda JN, Akankwatsa D, Wambua L, Dürr S, Hartnack S.*

* **[Design and evaluation of a remote damage control surgery real-time guidance system based on HoloLens 2 in low-speed network environments.](https://doi.org/10.1038/s41598-025-34705-w)** (2026) <br> 👤 *Chen C, Kang N, Zhong X, Jia Y, Jiang R, Dai C, Yang H, Chen L, Du W, Zong Z.*

* **[Paracetamol, pregnancy and law: What the Texas Tylenol case teaches SA doctors.](https://doi.org/10.4102/safp.v68i1.6262)** (2026) <br> 👤 *Bhamjee S.*

* **[Cognitive Communication, Voice and Swallowing Difficulties Experienced by Adults With Long-COVID: A Scoping Review.](https://doi.org/10.1111/hex.70595)** (2026) <br> 👤 *McTiernan K, Hughes C, Gilheaney Ó.*

* **[Challenges and recommendations for embracing digital tools for one health surveillance in LMICs: Viewpoints from a special session of 17th ISVEE.](https://doi.org/10.1016/j.onehlt.2025.101242)** (2025) <br> 👤 *Nguyen-Tien T, Unger F, Yano T, Mutua F, Cook EAJ, Lee HS, Bett B, Nguyen-Viet H.*

* **[Assessment of the Awareness and Use of Quality of Life Tools in Small Animal Practices in Germany.](https://doi.org/10.3390/ani15243617)** (2025) <br> 👤 *Rhein FF, Klee R, Albrecht B, Krämer S.*

* **[Extracellular vesicle-mediated microRNA signaling: expanding insights into intercellular communication and disease modulation.](https://doi.org/10.1097/ms9.0000000000004579)** (2026) <br> 👤 *Batool ST, Ali U, Khan MI, Mahato RK.*

* **[Enhanced Sensitivity to Elevated Plasma p‐Tau 217 with Proper Name Delayed Recall in the Preclinical Alzheimer's Cognitive Composite](https://europepmc.org/article/MED/None)** (2025) <br> 👤 *He D, Langhough R, Jonaitis E, Wilson R, Hermann B, Johnson S, Mueller K.*

* **[Insights and Lessons from Chilean Salmon Aquaculture on Antimicrobial Use.](https://doi.org/10.3390/antibiotics14121177)** (2025) <br> 👤 *Farias DR, Ibarra R, Tucca F, Jaramillo-Torres A, Cornejo J, Ibieta P, Mardones FO, Avendaño-Herrera R.*

* **[Single-cell transcriptomics uncover RNF130-mediated TNF-α pathway activation and worenine synergy with paclitaxel in breast cancer.](https://doi.org/10.1186/s13148-026-02057-5)** (2026) <br> 👤 *Hu M, Huang L, Deng H, Chen Z, Cheng G, Liu X.*

* **[Chimpanzee mothers, but not fathers, influence offspring vocal-visual communicative behavior.](https://doi.org/10.1371/journal.pbio.3003270)** (2025) <br> 👤 *Mine JG, Dees LC, Wilke C, Willems EP, Machanda ZP, Muller MN, Emery Thompson M, Wrangham RW, Scully EJ, Langergraber K, Stoll S, Slocombe KE, Townsend SW.*

* **[Why Humans Prefer Phylogenetically Closer Species: An Evolutionary, Neurocognitive, and Cultural Synthesis.](https://doi.org/10.3390/biology14101438)** (2025) <br> 👤 *Ragusa A.*

* **[Single-cell transcriptome revealed the aberrant keratinocytes activation in antigen presentation in atopic dermatitis.](https://doi.org/10.1080/07853890.2026.2627742)** (2026) <br> 👤 *Liu WX, Cao Y, Chen YF, Lu YF, Xu CY, Zhai YH, Wang C, Cao Z.*

* **[On the origin of Earth's Universal BioConsciousness.](https://doi.org/10.3389/fpsyg.2025.1636473)** (2025) <br> 👤 *Tleuberdinova A.*

* **[Chronic cold stress induced transcriptomic alterations in multi-metabolically active tissues of pigs.](https://doi.org/10.1038/s42003-025-09226-x)** (2025) <br> 👤 *Huang B, Bai Y, Zhou X, Tian X, Wang X, Li J, Hua L, Fan X, Li M.*

* **[Bioethics for Medical Students Using a Novel Concept for Patient Workshops.](https://doi.org/10.7759/cureus.96910)** (2025) <br> 👤 *Toma C, Petca A, Visan IG, Munteanu A, Jinga V.*

* **[VetEd Asia 2024: a symposium to promote veterinary education and collaboration.](https://doi.org/10.4142/jvs.25110)** (2025) <br> 👤 *Matsuwaki T, Sato K, Urabe M, Wijayawardhane N.*

* **[Development of a wireless bioelectronic actuator for wound healing in a porcine model.](https://doi.org/10.1038/s41598-025-26161-3)** (2025) <br> 👤 *Baniya P, Tebyani M, Hee WS, Li H, Asefifeyzabadi N, Schorger K, Yang HY, Gallegos A, Zhu K, Lu F, Keller G, Franco C, Devarajan K, Barbee A, Hernandez C, Nguyen T, Recendez C, Aslankoohi E, Isseroff RR, Zhao M, Gomez M, Rolandi M, Teodorescu M.*

* **[In sickness and health: A reflexive thematic analysis of ethical considerations and experiences of owners of cats treated for diabetes mellitus.](https://doi.org/10.1371/journal.pone.0341759)** (2026) <br> 👤 *Rothlin-Zachrisson N, Ström Holst B, Öhlund M, Leksell J, Röcklinsberg H.*

* **[Feline chronic pain management: the importance of a team approach for optimal outcomes.](https://doi.org/10.1177/1098612x251374161)** (2025) <br> 👤 *Monteiro BP, Taylor S, Dowgray N, Eyre K, Gruen M.*

* **[Beyond counting the numbers - focused outbreaks response strategies are needed to control the mpox outbreak in Africa.](https://doi.org/10.1186/s42522-026-00195-7)** (2026) <br> 👤 *Olu OO, Lubajo R, Wayengera M, Sabiiti W, Muwanguzi EN, Maleghemi S, Bosa HK.*

* **[Scissor-CIBERSORTx Deconvolution Reveals Functional Heterogeneity of CTAL/aTAL Cells and Associated Biomarkers in Renal Fibrosis.](https://doi.org/10.3390/cimb48020215)** (2026) <br> 👤 *Wang H, Zhang Y, Li J, Fu Y, Wang H.*

* **[Dog-assisted therapy on Hong Kong children with autism spectrum disorder: an exploratory randomized controlled trial.](https://doi.org/10.1007/s00431-025-06720-6)** (2026) <br> 👤 *Wong WHS, Chen C, Tso A, So HK, Wong JPY, Tinsley H, Chung CHY, Luk RKW, Ip P.*

* **[Attitudes of Family Medicine Program Directors on Including Race in Clinical Presentations: A Cross-Sectional Survey Study.](https://doi.org/10.7759/cureus.100767)** (2026) <br> 👤 *McLean I, Rosenberg R, Phelan M, Lopez A, Chen PH.*

* **[Establishing One Health Surveillance Platform for Electronic Integrated Disease Surveillance and Response in Malawi: Action Design Research Study.](https://doi.org/10.2196/72029)** (2026) <br> 👤 *Wu TJ, Mvula MV, Chado EKK, Kamanga BN, Mapemba DD, Billy RE, Nyirongo L, Mwale AC, Banda EC, Kagoli M, Manda TD, Bjune GA, Kaasbøll JJ.*

* **[Communication increases cooperation among students in a coordination game.](https://doi.org/10.1073/pnas.2517669122)** (2025) <br> 👤 *Hallers-Haalboom ET, De Jaegher KJM, Sterck EHM, Brosnan SF, Vermande MM.*

* **[Program Evaluation of the Family Adoption Program for Medical Graduates in India: A Strengths, Weaknesses, Opportunities and Challenges (SWOC) and Stakeholder Analysis in a Medical College in Assam.](https://doi.org/10.7759/cureus.100365)** (2025) <br> 👤 *Sharma S, Choudhari SG, Das D, Habib A, Ojah J.*

* **[Vocalization behaviors in captive bottlenose dolphins (Tursiops truncatus) and rough-toothed dolphins (Steno bredanensis).](https://doi.org/10.1371/journal.pone.0336138)** (2025) <br> 👤 *Fu W, Song Z, Xiang W, Zhang C, Yang W, Xu X, Zhang Y.*

* **[Chagas disease in Florida: An emerging one health challenge in the United States.](https://doi.org/10.1016/j.onehlt.2026.101360)** (2026) <br> 👤 *Ramírez JD, Monaghan J, Gutierrez S, Lichtenberger P, Beatty NL.*

* **[Testing Motivational Appeals to Promote Legume-Enriched Foods](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Unknown Authors*

* **[Single-cell aggrephagy landscape delineates intercellular communication of tumor microenvironment associated with ovarian cancer progression and immunotherapy.](https://doi.org/10.1007/s12672-025-04060-3)** (2025) <br> 👤 *Fang F, Chen J, Li X, Zhou C, Dai Y, Chen Y, Yang X, Xie Y, Qin F, Fu H.*

* **[Single-Cell Atlas of Fetal Immune Development Across Lung, Spleen, and Umbilical Cord Blood in Nonhuman Primates](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Unknown Authors*

* **[Transformative Learning in Digital Bioethics Education: An Interdisciplinary Lecture Series on Human-Animal Relationships.](https://doi.org/10.3390/ani15233398)** (2025) <br> 👤 *Widmann K, Busse TS, Ehlers JP, Nitsche J.*

* **[A Novel Cell-Cell Communication Structure: Tanycyte and Cilia Hypothalamic Unifying Glia-cilia Structure (HUGS).](https://doi.org/10.17912/micropub.biology.001961)** (2026) <br> 👤 *Schwantz KR, Boone JG, Brewer KM, Berbari NF.*

* **[A Structured Quality Improvement Framework for Managing Serratia marcescens Outbreaks in a Resource-Limited Neonatal Intensive Care Unit.](https://doi.org/10.7759/cureus.102156)** (2026) <br> 👤 *Khaild M, Al Shafouri N, Al Nassri A.*

* **[The Challenge of Lyme Borreliosis: Knowledge, Attitudes, and Practices in France.](https://doi.org/10.3390/biology14091286)** (2025) <br> 👤 *Philibert C, Ferrolho J, Gibson MS, Ramalho de Sousa S, Vilhena H, Anastácio S.*

* **[A Prospective Study on the Impact of the Communication Skills Module on Medication Adherence: Healing Through Dialogue.](https://doi.org/10.7759/cureus.97315)** (2025) <br> 👤 *Bhatti N, Joseph G, Abhilasha P, Badyal DK, Ansari MWF, Oberoi A, Kumar A.*

* **[Cultural barriers influencing virtual community of practice: a scopic review.](https://doi.org/10.1186/s40359-025-03841-3)** (2026) <br> 👤 *Manzoor S, Atiq N.*

* **[Single-Cell Dissection of Dimethomorph-Induced Neurotoxicity in the Gestational Brain: Metabolic Disruption and Blood-Brain Barrier Dysfunction.](https://doi.org/10.1021/acs.est.5c11595)** (2026) <br> 👤 *Zhang T, Qi Z, Teng Y, Duan Z, Dai H, Cheng D, Han J, Liu M.*

* **[Acoustic analysis of bottlenose dolphin vocalizations for behavioral classification in controlled settings.](https://doi.org/10.1371/journal.pone.0336419)** (2025) <br> 👤 *Screpanti L, Di Nardo F, De Marco R, Furlati S, Bucci G, Lucchetti A, Scaradozzi D.*

* **[Bridging Communication Gaps to Enhance Patient Safety: A Quality Improvement (QI) Project on the Role of Abbreviations, Their Risks, and Pathways to Change.](https://doi.org/10.7759/cureus.95843)** (2025) <br> 👤 *Talha S, Smith B, Khan A, Gaddoura Z.*

* **[Advances and Applications of Organ-on-a-Chip and Tissue-on-a-Chip Technology.](https://doi.org/10.3390/bioengineering13010009)** (2025) <br> 👤 *Moore M, Sriram S, Ku J, Li Y.*

* **[Identifying vascular stiffening-sensitive macrophages through integration of single-cell transcriptomics and imaging flow cytometry.](https://doi.org/10.52601/bpr.2025.240063)** (2025) <br> 👤 *Wang J, Wang J, Zhang Y, Xiong C, Liu SA, Kong Y, Zhou J, Wang X.*

* **[An Interventional Study to Evaluate the Effect of Communication Skill Training and Its Perception Among Second-Year Undergraduate Medical Students.](https://doi.org/10.7759/cureus.95819)** (2025) <br> 👤 *Acharya TA, Chhaiya SB, Kakasaniya GG, Vaghasia YA, Master RJ, Singh US.*

* **[Dynamic crosstalk between Tspan4&lt;sup&gt;+&lt;/sup&gt; macrophage subsets and MSCs via migrasomes orchestrates fracture repair.](https://doi.org/10.3389/fcell.2025.1666465)** (2025) <br> 👤 *Zhang S, Wang M, Kutibiding A, Liu D, Manafu T, Zhao W, Yang Y.*

* **[Emerging technologies in poultry genomics: Unlocking innovation for the future of sustainable production.](https://doi.org/10.1016/j.psj.2025.106240)** (2026) <br> 👤 *Zhou H, McCarthy FM, Kim TH, Warren W, Zhang G.*

* **[One Health Approaches to Ethical, Secure, and Sustainable Food Systems and Ecosystems: Plant-Based Diets and Livestock in the African Context.](https://doi.org/10.3390/foods15010085)** (2025) <br> 👤 *Hosseini E, Tsegay ZT, Smaoui S, Elfalleh W, Antoniadou M, Varzakas T, Caraher M.*

* **[Advancing pet care through technology: An evaluation of the LINE-based VET PROMPT notification system.](https://doi.org/10.5455/javar.2025.l947)** (2025) <br> 👤 *Somrup S, Sirimalaisuwan A, Chaisowwong W, Na-Lampang K.*

* **[LPS-Stressed Bovine Endometrial Cells upon Morulae in a Transwell Model of Embryo-Maternal Talk.](https://doi.org/10.3390/ani16010038)** (2025) <br> 👤 *Lange-Consiglio A, Gaspari G, Gagni P, Bosi G, Riccaboni P, Cremonesi F.*

* **[Extracellular vesicles in reproductive medicine: from "animal-led" to "plant-enabled".](https://doi.org/10.3389/fcell.2025.1718643)** (2025) <br> 👤 *Pan K, Ju W, Wang Y, Zhang Q, Wang R, Jiang X, Zhao S.*

* **[More than Dysbiosis: Imbalance in Humoral and Neuronal Bidirectional Crosstalk Between Gut and Brain in Alzheimer's Disease.](https://doi.org/10.3390/ijms27010369)** (2025) <br> 👤 *Tassibekova G, Zholdassova M, Novosolova N, Malm T, Giniatullin R, Kustubayeva A.*

* **[Developing a measurement tool for assessing animal-assisted activity effectiveness on children with special educational needs' socialization: A pilot study.](https://doi.org/10.1177/20503121251363015)** (2025) <br> 👤 *Hong L, Shen X, Hui TK, Luk RK, So HK, Wong WH.*

* **[Exploring the potential mechanisms of OSBPL3 in metabolic dysfunction-associated steatotic liver disease by integrating bulk and single-cell RNA sequencing data.](https://doi.org/10.1097/md.0000000000046273)** (2025) <br> 👤 *Wang Q, Zhu C, Xiao Y, Wang X, Song W, Li S, Jiang F, Wei L, Hua F.*

* **[The 3D revolution: organoids and spheroids reshape parasitology research.](https://doi.org/10.1017/s0031182025101273)** (2025) <br> 👤 *Cantacessi C.*

* **[Unveiling the IL-1β/CXCL2 axis: a shared therapeutic target in periodontitis and inflammatory bowel disease.](https://doi.org/10.3389/fimmu.2025.1728773)** (2025) <br> 👤 *Gu Z, Gao A, Ma X, Wang X, Zhang Y, Wang Y, Qiu C.*

* **[Measuring the Effectiveness of Both Cognitive and Emotional Forms of Instructional Videos Related to the Beef Industry.](https://doi.org/10.3390/ani15172584)** (2025) <br> 👤 *Locke S, Hiltbrand K, Corbitt K, Richburg D, Johnson G, Shannon D, Rodning S, Sawyer J, Mulvaney D.*

* **[Exploring veterinarians' perceptions of decision-making with clients in the context of providing access to veterinary care.](https://doi.org/10.1371/journal.pone.0342564)** (2026) <br> 👤 *Groves CNH, Coe JB, Bauman C, Grant LE.*

* **[Application of Artificial Intelligence Tools for Social and Psychological Enhancement of Students with Autism Spectrum Disorder: A Systematic Review.](https://doi.org/10.3390/brainsci16010056)** (2025) <br> 👤 *Tsapanou A, Bouka A, Papadopoulou A, Vamvatsikou C, Mikrouli D, Theofila E, Dionysopoulou K, Kortseli K, Lytaki P, Spyridonidi TM, Plotas P.*

* **[Impact of a Spanish for Specific Purposes in Agriculture Curriculum for Animal Science and Veterinary Students on Animal Welfare Knowledge of Spanish-Speaking Livestock Workers.](https://doi.org/10.3390/ani15172506)** (2025) <br> 👤 *Martinez Aguiriano AJ, Salazar L, Pietrosemoli S, Awosile B, Lamino P, Garcia A.*

* **[Shared Decision-Making in Veterinary Clinics: A Cross-Sectional Study Examining Effects on Appropriate Antibiotic Prescribing and Value-Driven Factors Influencing It.](https://doi.org/10.3390/antibiotics14111139)** (2025) <br> 👤 *Guo H, Ong LS, Hildon ZJ, Chua T, Teo BH, Chow A.*

* **[Animal models of autism spectrum disorder: Insights into genetic, structural and environmental models.](https://doi.org/10.17221/87/2024-vetmed)** (2025) <br> 👤 *Chivchibashi-Pavlova D, Bratoeva K.*

* **[Integrative omics and experimental validation reveal METTL17 and SLC27A1 as biomarkers and potential therapeutic targets in chronic kidney disease](https://europepmc.org/article/MED/None)** (Unknown Year) <br> 👤 *Unknown Authors*

* **[Successful Rescue of a Juvenile Humpback Whale (&lt;i&gt;Megaptera novaeangliae&lt;/i&gt;) Trapped Upstream of the Rance Tidal Power Station, Brittany, France.](https://doi.org/10.3390/ani15233503)** (2025) <br> 👤 *Olhasque O, Carpentier L, Duchemin M, Jung JL, Dars C, Boucard F, Labrut S, De Weerdt J.*

* **[Decoding the distinct immune landscape and possible regulatory mechanisms of autoimmune hepatitis through integrated single-cell and bulk RNA sequencing.](https://doi.org/10.1371/journal.pone.0335605)** (2025) <br> 👤 *Chi G, Che Y, Pei J, Li X, Wang J.*

* **[Enhancing Preoperative Orthopaedic Communication: A Comparative Analysis of Large Language Model- and Clinician-Generated Clinic Letters.](https://doi.org/10.7759/cureus.101413)** (2026) <br> 👤 *Saunders WC, Glendenning AC, Gamble C, Roberts R.*

* **[Developmental and early-life stress-induced effects on 5-HT&lt;sub&gt;3&lt;/sub&gt;R-expressing interneurons within auditory cortex.](https://doi.org/10.1016/j.ynstr.2026.100780)** (2026) <br> 👤 *Moore JT, Sunthimer MJ, White E, Mellott JG, Rosen MJ.*

* **[Testing methodological and population-specific influences on the detection of Zipf’s law of brevity in chimpanzee gestures](https://doi.org/10.1101/2025.11.21.689682)** (2025) <br> 👤 *Safryghin A, Badihi G, Ferrer-i-Cancho R, Grund C, Hayashi M, Mielke A, Mine J, Rodrigues E, Soldati A, Zuberbühler K, Zulberti C, Hobaiter C.*

* **[From Pen to Portal: Using Artificial Intelligence to Evaluate the Impact of Electronic Patient Information and Communication (EPIC) on the Quality of Urology Clinic Letters at a Major London Hospital.](https://doi.org/10.7759/cureus.97486)** (2025) <br> 👤 *Patel T, Khan N, Huntley J, Raison N.*

* **[Perceptions of Intraoperative Music Among Surgical Trainees and Operating Room Staff: A Cross-Sectional Study in an Accreditation Council for Graduate Medical Education (ACGME)-Accredited Program.](https://doi.org/10.7759/cureus.101331)** (2026) <br> 👤 *Medina-Morell JR, Ramirez-Rivera F, Zapata-Valentin M, Bolanos-Rios A, Santini-Dominguez R.*

* **[Bottleneck and enabler evaluation of avian influenza health event - Guatemala, January-February 2023.](https://doi.org/10.1371/journal.pgph.0005443)** (2025) <br> 👤 *Bastani P, Leonardo EB, Fuentes JCM, Pereira CC, Zielinski Gutierrez E, Suchdev PS.*

* **[Rider education at Swedish riding schools: Comparing teachers' and pupils' perspectives](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Nyberg L, Blokhuis M, McLean A, Hartmann E.*

* **[One health rapid qualitative assessment: Exploring local governance gaps in Tanzania.](https://doi.org/10.1016/j.onehlt.2026.101353)** (2026) <br> 👤 *Rubin O, Nyanda SS, Norström M.*

* **[Challenging the recessive paradigm of Mahvash disease: heterozygous phenotypes from a novel splice-site variant.](https://doi.org/10.20945/2359-4292-2026-0005)** (2026) <br> 👤 *Bugallo FM, de GG, Prieto-Morín C.*

* **[Practical Strategies for Integrating Ad Hoc Simulated Participants in Emergency Medicine Education: A Faculty Guide.](https://doi.org/10.7759/cureus.100025)** (2025) <br> 👤 *Wong KU, Lei C, Hock SM, Chen TH.*

* **[Perinatal hyperandrogenization and immune activation in rodents model subtypes of autism.](https://doi.org/10.1038/s41398-026-03821-0)** (2026) <br> 👤 *Burke FF, Randell AM, Sparkes KM, Salia S, Hinks M, Martin Y, Porter Z, Hynes C, Maekawa AS, Sled J, Wilson DR, Kaushik DK, Kakumani PK, Walling SG, Cahill LS, Swift-Gallant A.*

* **[Evidence of prevailing practice of home slaughter in Iran revealed by bioeconomic modeling of small ruminants slaughtered in and outside registered abattoirs.](https://doi.org/10.1371/journal.pone.0337839)** (2026) <br> 👤 *Ebrahimipour M, Borhani M, Dayani O, Fasihi Harandi M.*

* **[Evaluating the Impact of the Saudi Model of Care ST-Elevation Myocardial Infarction (STEMI) Pathway on Interfacility Communication Between a PCI-Capable Center and Referring Hospitals in the Qassim Region, Saudi Arabia.](https://doi.org/10.7759/cureus.101290)** (2026) <br> 👤 *Aldakheel MA, Bhat YA, Alribdi F, Almeman AA, Alanazi MA, Alfawzan A, Alradhi SA, Almesned A, Alsultan SA, Alanazi A, Camba AV, Rimando QAD, Alkhalaf K, Alharbi MM, Alqwaee A.*

* **[Bridging biology and therapy: translational advances of extracellular vesicles in veterinary clinical practice.](https://doi.org/10.1007/s11259-025-10917-3)** (2025) <br> 👤 *Gad WA, Ibrahim S, Nagdy H, Elsawy BSM, Aboelsoued D, Abdel-Ghany HSM, Abdel-Wareth AAA, Abd El-Razik KA, Mahmoud KGM, Soliman WTM, Taqi MO.*

* **[PANoptosis-Related Diagnostic Biomarkers in Non-Neovascular Age-Related Macular Degeneration: An Integrative Transcriptomic and Experimental Study.](https://doi.org/10.1155/genr/8903808)** (2026) <br> 👤 *Li J, Ma Y, Hu M, Zhang Q, Wang A, Tang Q, Guo Q, Huang B.*

* **[Underwater Pressure Lumen Expansion: A Novel Method to Overcome Lumen Collapse in Submucosal Endoscopy and Animal Endoscopic Full-thickness Resection Models.](https://doi.org/10.1002/deo2.70195)** (2026) <br> 👤 *Shiwaku H, Shiwaku A, Sato S, Yamauchi K, Hasegawa S.*

* **[The Conundrum of XenomiRs and Human Health.](https://doi.org/10.1016/j.advnut.2025.100510)** (2025) <br> 👤 *Tambaro F, Gallicchio C, Orlando S, Carnevale S, Muscaritoli M.*

* **[Autism Spectrum Disorders and Purinergic Signaling: A Systematic Review of Emerging Insights from Preclinical Studies.](https://doi.org/10.1007/s12031-026-02479-z)** (2026) <br> 👤 *Guha S, Elisha DH, Eshraghi RS, Mittal R, Deth RC, Eshraghi AA.*

* **[Development of language composites for enhanced sensitivity to multiple plasma biomarkers.](https://doi.org/10.1002/alz.70975)** (2025) <br> 👤 *He D, Langhough RE, Van Hulle C, Basche K, Jonaitis E, Wilson R, Hermann B, Zetterberg H, Johnson S, Mueller KD.*

* **[Effects of gestational perfluorohexanesulfonic acid exposure at human realistic dose on social communication deficit in mouse offspring.](https://doi.org/10.1016/j.ebiom.2026.106160)** (2026) <br> 👤 *Zhang S, Gao C, Li R, Lv J, Xu J, Miao M, Liang H, Xu DX, Wang B, Huang Y.*

* **[Role of Monocyte/Macrophage-Associated Gene Ccl9 in Acute Myocardial Infarction Analysis: Based on Bulk and Single Cell RNA Sequencing Data.](https://doi.org/10.2147/jir.s516092)** (2025) <br> 👤 *Yang Q, Luo Z, Cheng X, Wang T, Hu P.*

* **[The eNAMPT-Integrin α5β1 Axis Mediates Neutrophil-Endothelial Cell Interactions Driving Inflammation in Ulcerative Colitis.](https://doi.org/10.2147/jir.s554975)** (2025) <br> 👤 *Di Y, Ji W, Xiong W, Song W, Chen G, Li D, Qi F.*

* **[Correction: Cao et al. Evidence for Semantic Communication in Alarm Calls of Wild Sichuan Snub-Nosed Monkeys. &lt;i&gt;Biology&lt;/i&gt; 2025, &lt;i&gt;14&lt;/i&gt;, 1028.](https://doi.org/10.3390/biology14091139)** (2025) <br> 👤 *Cao FJ, Anderson JR, Fu WW, Gou NN, Shen JN, Cen FS, Tu YR, Mao M, Wang KF, Yang B, Li BG.*

* **[Effects of sheep follicular fluid-derived exosomes and miR-148a on theca cell function in vitro.](https://doi.org/10.1186/s40659-026-00668-z)** (2026) <br> 👤 *Liu K, Chi R, Zhang R, Liu Q, Xing F, Liu Y, Chu M.*

* **[A Multi-omics Exploration Revealing SLIT2 as a Prime Therapeutic Target for Peripheral Facial Paralysis: Integrating Single-Cell Transcriptomics and Plasma Proteome Data.](https://doi.org/10.1007/s10571-025-01607-4)** (2025) <br> 👤 *Liu Y, Li C, Yao L, Tian M, Tan Y, Shi L, Chen P.*

* **[Weight optimization of MIMO-UWB distributed beamforming for implant communications.](https://doi.org/10.1038/s41598-026-36694-w)** (2026) <br> 👤 *Kobayashi T, Hyry J, Fujimoto M, Anzai D.*

* **[First detection of West Nile virus in Belgium through wild bird surveillance, Belgium, 2025.](https://doi.org/10.2807/1560-7917.es.2026.31.4.2600049)** (2026) <br> 👤 *Sohier C, Breman FC, Vervaeke M, Sikkema R, Boter M, Oude Munnink B, Koopmans M, Linden A, Duran Illan L, Romero JR, Lernout T, De Regge N.*

* **[Examining the Effectiveness of a Clinical Skills Curriculum: A Critical Analysis.](https://doi.org/10.7759/cureus.99479)** (2025) <br> 👤 *Raphael C, HajAli T, Sarkis S.*

* **[Analyzing countries' needs to improve multisectoral collaboration against health threats at the human-animal-environment interface: a study of 51 National Bridging Workshop Roadmaps.](https://doi.org/10.1186/s44263-025-00193-3)** (2025) <br> 👤 *Alfonso-Dilley CS, Belot G, Alessandrini B, Algüerno MI, Caya F, Dorjee S, Dukpa K, Furco A, Gongal G, Guigma Y, Sousa Hoejskov P, Kayamori Lopes J, Lafia B, Lasley JN, Mahrous H, Monagin C, Montabord D, Morgan D, Pinto J, Saidouni A, Skrypnyk A, Traore T, Wambua L, Weber-Vintzel L, de la Rocque S.*

* **[Reply: Seemingly Reverse Wenckebach Phenomenon, Truly Inverse Decremental Conduction.](https://doi.org/10.1016/j.jaccas.2025.106718)** (2026) <br> 👤 *Chen F, Jiao ZZ, Guo XG.*

* **[Toward a "green" brain health agenda: establishing short- and long-term goals for the field of neurology in a changing climate.](https://doi.org/10.3389/fneur.2025.1618365)** (2025) <br> 👤 *Trahair ED, Dalapati T, Levinson D, Linares AR, Reuben A, Bedlack R.*

* **[E-health Literacy and the Impact of Telehealth on Treatment Compliance and Accessibility.](https://doi.org/10.7759/cureus.103718)** (2026) <br> 👤 *Abdellatif N, Qais IO, Hussain SR, AlJabal HA, Alhimairi A, Naji H.*

* **[Is it feasible to develop a one health surveillance system in a fragile state? The case of Guinea-Bissau.](https://doi.org/10.1016/j.onehlt.2025.101269)** (2025) <br> 👤 *Fernandes FOL, Crima S, Ferrinho P.*

* **[Strengthening Patient Safety in Respiratory Wards by Developing a Weekend Continuity Checklist.](https://doi.org/10.7759/cureus.97546)** (2025) <br> 👤 *Iftikhar Z.*

* **[KLF4 orchestrates a Ccl2+ fibroblast-mediated inflammatory network driving preterm birth.](https://doi.org/10.3389/fmed.2025.1700275)** (2025) <br> 👤 *Gao J, Chang Y, Su Y, Sun L, Wang F, Xin H.*

* **[A concise mathematical description of signal transformations across the hippocampal apical CA3 to CA1 dendritic response](https://europepmc.org/article/MED/None)** (Unknown Year) <br> 👤 *Gattas S, Le A, Karimi Abadchi J, Pruess B, Amba R, Shen Y, Swindlehurst A, Yassa M, Lynch G.*

* **[A multi-tissue single-cell expression atlas in cattle.](https://doi.org/10.1038/s41588-025-02329-5)** (2025) <br> 👤 *Han B, Li H, Zheng W, Zhang Q, Chen A, Zhu S, Shi T, Wang F, Zou D, Song Y, Ye W, Du A, Fu Y, Jia M, Bai Z, Yuan Z, Liu W, Tuo W, Hope JC, MacHugh DE, O'Grady JF, Madsen O, Sahana G, Luo Y, Lin L, Li C, Cai Z, Li B, Huang J, Liu L, Zhang Z, Ma Z, Hou Y, Liu GE, Jiang Y, Sun HZ, Fang L, Sun D.*

* **[Identifying What's Vital in Shift Handover: The ABC-VITAL Model for ED Transitions.](https://doi.org/10.7759/cureus.96860)** (2025) <br> 👤 *Peñaloza AJ, Garcés J, Salazar C, Cadavid A.*

* **[A Digital Patient Engagement and Monitoring System to Improve Quality, Safety, and Continuity of Care for Pediatric Sickle Cell Anemia: A Cross-Sectional Study of Caregiver Perceptions.](https://doi.org/10.7759/cureus.101590)** (2026) <br> 👤 *Hanafy E, Alasmari BG, Elmugadam AA, Hassanien SS, Alsafy AS, Alpakra M.*

* **[Combining Spatial Transcriptomics and Machine Learning Unravels Fibroblast-Related Biomarkers in Ulcerative Colitis.](https://doi.org/10.1007/s10753-025-02387-1)** (2026) <br> 👤 *Kang K, Li X, Wang S, Huang C, Wu Z.*

* **[Communicating Antimicrobial Resistance on Instagram: Content Analysis of #AntibioticResistance.](https://doi.org/10.2196/67825)** (2025) <br> 👤 *Nilsson E, Oljans E, Nordvall AC, Ancillotti M.*

* **[Functional analysis and identification of miRNAs associated with lipid metabolism from milk-derived exosomes.](https://doi.org/10.1186/s40104-025-01331-5)** (2026) <br> 👤 *Lu X, Deng T, Liu Y, Zhang X, Bai X, Fang X, Yang R.*

* **[Macrophage Phospholipase D3 promotes atherosclerosis via exacerbating foam cell formation and inducing inflammatory responses.](https://doi.org/10.1016/j.ijcha.2025.101834)** (2025) <br> 👤 *Li T, Gu X, Yin X, Zhai P, Yan H, Wang Z, Li Y, Wang B.*

* **[Improving Discharge Summary Follow-Up Documentation in a Major Trauma Center: A Two-Cycle Quality Improvement Project.](https://doi.org/10.7759/cureus.99842)** (2025) <br> 👤 *Ahmed H, Mazhar F, Yousaf Z, Akram U, Mazhar S, Rizwan M.*

* **[Closed, pumpless microphysiological system with unidirectional flow for co-culture and focal irradiation.](https://doi.org/10.1039/d5ra06702b)** (2026) <br> 👤 *Ueno H, Hanamura K, Aoki Y, Yamamura M, Shirao T, Suzuki T.*

* **[Bridging the Gap in Medical Education: A Qualitative Study on the Perspectives of Japanese Medical Students and Patients on Outcomes-Based Education.](https://doi.org/10.7759/cureus.95527)** (2025) <br> 👤 *Son D, Homma M, Pickering J.*

* **[Inclusion of Diagnostic Imaging in Orthopaedic Discharge Summaries: A Quality Improvement Project.](https://doi.org/10.7759/cureus.95992)** (2025) <br> 👤 *Parkar N, Alduhoky M, Wignadasan W.*

* **[Single-Cell Sequencing Reveals the Crosstalk Between MuSCs and FAPs in Ruminant Skeletal Muscle Development.](https://doi.org/10.3390/cells15020206)** (2026) <br> 👤 *Chen Y, Gong Y, Xu X, Song M, Sun X, Luo J, Guo J, Li L, Zhang H.*

* **[Leveraging disease outbreak news to strengthen the global response to antimicrobial resistance: a call for action.](https://doi.org/10.3389/fpubh.2025.1710596)** (2025) <br> 👤 *Kiggundu R, Waswa JP, Mwanja H, Hope M, Kambugu A, Kakooza F, Byonanebye DM.*

* **[Changes in Nucleus Pulposus Cell Atlas and the Role of SPP1 During Intervertebral Disc Degeneration: Single-Cell Sequencing Analysis.](https://doi.org/10.1155/mi/5593429)** (2025) <br> 👤 *Liu C, Jiao K, Li X, Deng Z, Wang S, Cheng Y, Li M, Zhou X, Wei X.*

* **[Enhanced sparrow search algorithm with DV-Hop for high-precision fire sensor localization in underground spaces.](https://doi.org/10.1371/journal.pone.0338706)** (2025) <br> 👤 *Li J, Dong L, Xu L, Luo F, Lu Z, Huang Y, Wei S.*

* **[Evaluating the Correct Documentation of Physician Details and Time in General Surgery Ward Round Notes: An Audit and Re-Audit.](https://doi.org/10.7759/cureus.98900)** (2025) <br> 👤 *Al-Wizni A.*

* **[Examining bhutan's zoonotic disease policies through a One Health lens: a qualitative policy analysis using the policy triangle framework.](https://doi.org/10.1186/s42522-025-00159-3)** (2025) <br> 👤 *Rai BD, Tenzin T, Tshering D, Dahal N, Tessema GA, Fritschi L, Nyadanu SD, Pereira G.*

* **[Can nursing strengthen one health initiatives?](https://doi.org/10.1016/j.onehlt.2025.101288)** (2025) <br> 👤 *Fernandes JB, Vareta D.*

* **[Inflammation associated microglial expansion disrupts hippocampal glial network communication, driving postoperative neurocognitive impairment.](https://doi.org/10.1186/s12974-025-03566-5)** (2025) <br> 👤 *Zheng Y, Xu W, Suo Z, Qu Y, Lin L, Zhang E, Han D, Wu M, Li H, Zheng H, Xiao T, Ni C.*

* **[Poultry production under threat: Framework to counter misinformation and protect food security.](https://doi.org/10.1016/j.psj.2025.106050)** (2025) <br> 👤 *Nassar FS.*

* **[LSD1 maintains oocyte quality by epigenetically repressing CAMK2β to control granulosa cell-oocyte communication.](https://doi.org/10.1186/s12967-026-07730-0)** (2026) <br> 👤 *Zhang J, Yu Y, Zhu Z, Zhang H, Hu Z, Sheng Z, He M, Zhang T.*

* **[A qualitative risk assessment of Crimean-Congo haemorrhagic fever in Nigeria: implications for One Health response.](https://doi.org/10.1093/inthealth/ihaf056)** (2026) <br> 👤 *Egwuenu A, Makama S, Ahmed N, Igah O, Nwiyi G, Abioye Y, Kolade D, Okpala C, Emelife C, Okunromade O, Ilori E, Pantuvo J, Agogo E, Aruna O, Lucia Ochu C, Adetifa IMO.*

* **[Dopamine and the Gut Microbiota: Interactions Within the Microbiota-Gut-Brain Axis and Therapeutic Perspectives.](https://doi.org/10.3390/ijms27010271)** (2025) <br> 👤 *Barbu AC, Stoleru S, Zugravu A, Poenaru E, Dragomir A, Costescu M, Aurelian SM, Shhab Y, Stoleru CM, Coman OA, Fulga I.*

* **[Deep learning-aided inter-species-comparison reveals shared and distinct molecular patterns in cynomolgus monkey and humans following non-specific T cell activation.](https://doi.org/10.3389/fimmu.2025.1603716)** (2025) <br> 👤 *Friedrich VD, Neier K, Müller K, Fogal B, Loncova Z, Rade M, Shoaib M, Köhl U, Hoyt K, Pande P, Blanchard L, Raymond E, Scholz M, Reiche K, Kirsten H.*

* **[OASIS evaluation of the French laboratory diagnostic surveillance system: right people, right techniques but imperfect use.](https://doi.org/10.3389/fvets.2025.1419034)** (2025) <br> 👤 *Chikh M, Ngwa-Mbot D, Morignat E, Memeteau S, Amat JP.*

* **[Sympathetic nerve inhibition enhances calvarial bone repair via senescent macrophage-induced osteogenesis and angiogenesis.](https://doi.org/10.1038/s41420-025-02886-y)** (2025) <br> 👤 *Zhao L, Xu Z, Zhao P, Pang Z, You Y, Wu C, Zhu D, Su M, Zhang N, Luo J, Wang Y.*

* **[Research on the influencing factors of user engagement in tourism short video from the perspective of configuration.](https://doi.org/10.1371/journal.pone.0337406)** (2025) <br> 👤 *Li L, An H.*

* **[A framework for facilitating sustainable One Health collaboration across sectors at the national level in the European Union/European Economic Area.](https://doi.org/10.2807/1560-7917.es.2025.30.43.2500218)** (2025) <br> 👤 *Hoefle-Bénard J, Varela Santos C, Heuer O, Kinsman J.*

* **[Undergraduate Medical Teachers' Perceptions Toward Video-Based Feedback in Teaching Communication Skills: A Qualitative Study Protocol.](https://doi.org/10.7759/cureus.95023)** (2025) <br> 👤 *Lim WW, Paris Li Min C.*

* **[Single-cell RNA sequencing reveals T and B cell-related immune features in foot and mouth disease virus-infected mice.](https://doi.org/10.1080/21505594.2025.2580141)** (2025) <br> 👤 *Wang Y, Zhang L, Zhang Z, Chen Z, Xu J, Zhang H, Zheng H, Pei J.*

* **[Protein undernutrition alters the colonic bacteriome, disrupts intestinal immune homeostasis, and impairs control of &lt;i&gt;Leishmania infantum&lt;/i&gt; infection in a murine model of visceral leishmaniasis.](https://doi.org/10.3389/fnut.2025.1733703)** (2025) <br> 👤 *Azevedo R, Losada-Barragán M, Costa EM, Pauer H, Hardoim CCP, Lima F, Pinho N, Durães J, Gaitán-Albarracín F, Cuervo-Escobar S, Arcila-Barrera S, Umaña-Pérez A, Antunes LCM, Cuervo P.*

* **[Building resilience in animal research: a two-year mixed methods survey of organizational support programs.](https://doi.org/10.3389/fvets.2025.1736974)** (2025) <br> 👤 *Young L, Thompson-Iritani S, Bellanca RU, Ferrara F, Bell K, Martin T, LaFollette MR.*

* **[Interspecies Relational Theory: A Framework for Compassionate Interspecies Interactions.](https://doi.org/10.3390/vetsci12060586)** (2025) <br> 👤 *Kieson E.*

* **[VDzSL: A synthetic video dataset for Algerian sign language using 3D avatars.](https://doi.org/10.1016/j.dib.2026.112551)** (2026) <br> 👤 *Ouargani Y, Khattabi NE.*

* **[Moving from reactive response to proactive prevention of emerging infectious diseases: Socio-ecological systems mapping in the Democratic Republic of the Congo.](https://doi.org/10.1371/journal.pgph.0005400)** (2025) <br> 👤 *Yambayamba MK, Metena M, Paku R, Lutonda C, Ngolole F, Bongono EF, Makiala-Mandanda S, Masumu J, Rüegg SR.*

* **[Challenges Experienced by Novice Undergraduate Nursing Students in Their Clinical Learning Environment: A Qualitative Study.](https://doi.org/10.7759/cureus.97322)** (2025) <br> 👤 *S K PR.*

* **[Evaluating Public Awareness and Understanding of Anesthesia Practices in Saudi Arabia: A Cross-Sectional Study on Patients' Confidence in and Perceptions of Anesthesiologists.](https://doi.org/10.7759/cureus.102374)** (2026) <br> 👤 *Nabri A, Alaboudi ZA, Al Dhaif NI, Alshalawi KS, Aljasser DA, Alrazoog AA, Alanazi FK, Alshaikh SA, Alanazi MF, Alanazi AF, Alaboudi ZA.*

* **[Alpha-synuclein overexpression does not cause vocalization deficits in a mouse model of parkinsonism.](https://doi.org/10.1242/bio.062120)** (2025) <br> 👤 *Rodgers B, Schaser A.*

* **[Conservatively Managed Iatrogenic Coronary Fistula to the Coronary Sinus During Primary Percutaneous Intervention.](https://doi.org/10.1016/j.jaccas.2025.106429)** (2026) <br> 👤 *Mahdy T, Khedr M, Hoey E, Bhatia G, Ment J, Pulikal G.*

* **[Neonatal social communication and single genes predict the variability of post-pubertal social behavior in a mouse model of paternal 15q11-13 duplication](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Unknown Authors*

* **[Identification of immunometabolism-associated genes and immune infiltration in sepsis-induced cardiomyopathy using integrated bioinformatics and machine learning approaches.](https://doi.org/10.1177/03000605251395584)** (2025) <br> 👤 *Pei C, Zheng H, Yang K, Song N.*

* **[Fostering learning among the next generation of veterinarians: incorporating one health and antimicrobial stewardship into veterinary medicine training curricula in Ethiopia.](https://doi.org/10.3389/fvets.2025.1690392)** (2025) <br> 👤 *Nyokabi N, Phelan L, Berg S, Lindahl J, Korir L, Bett B, Muunda E, Mihret A, Moore H, Wood J.*

* **[Ethical considerations of artificial intelligence in veterinary medicine decision-making](https://europepmc.org/article/MED/None)** (Unknown Year) <br> 👤 *Heinlein M.*

* **[Understanding the Preferences of Genetic Tools and Extension Services for the Northern Australia Beef Industry.](https://doi.org/10.3390/ani16010132)** (2026) <br> 👤 *Menchon P, Cosby A, Swain DL, Manning JK.*

* **[Single-Cell Transcriptomic Analysis Reveals an Inflammatory Antigen-Presenting Macrophages Subtype Drive Vitiligo Pathogenesis Through STAT1-Mediated Dual Mechanisms.](https://doi.org/10.1155/mi/8878698)** (2025) <br> 👤 *Qi R, Huang M, Lin Z, Deng H, Sa R, Chen Y, Chen G, Duan X, Jinhui L.*

* **[Unveiling critical signaling pathways in the murine salivary gland and the role of midkine.](https://doi.org/10.1016/j.isci.2026.114852)** (2026) <br> 👤 *Wrynn T, Osinski J, Sinha S, Romano RA.*

* **[Real-World Practices in Educating Patients and Caregivers About Sudden Unexpected Death in Epilepsy: A Scoping Review.](https://doi.org/10.4103/sjmms.sjmms_114_25)** (2026) <br> 👤 *Alzahrani LA, Aldweesh AF, Alotaibi AN, Zafar AS, Bansal MB.*

* **[Associations Between Self‐reported History of Traumatic Brain Injury on Longitudinal Cognitive and Speech Changes Linked to Alzheimer's Disease Risk Factors](https://europepmc.org/article/MED/None)** (2025) <br> 👤 *Goulette O, He D, Basche K, Langhough R, Betthauser T, Hermann B, Johnson S, Mueller K.*

* **[Quantifying Information Exchange Between Cells in Inflammaging](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Ibrahim I, Pidaparti R.*

* **[Key aspects of One Health governance in the European Union.](https://doi.org/10.1016/j.onehlt.2026.101325)** (2026) <br> 👤 *Donnelly F, Stilianakis NI.*

* **[The Fibro-Immune Landscape Across Organs: A Single-Cell Comparative Study of Human Fibrotic Diseases](https://europepmc.org/article/MED/None)** (2026) <br> 👤 *Deng G, Luo Y, Lin X, Zhang Y, Lin Y, Pan Y, Ruan Y, Mo X, Fang S.*

* **[Gut dysbiosis in neurodevelopmental disorders: linking microbiota signatures to cognitive rigidity in autism spectrum disorder.](https://doi.org/10.3389/fmicb.2026.1760635)** (2026) <br> 👤 *Sivamaruthi BS, Kesika P, Chaiyasut C, Ragu Varman D.*



In [3]:
# @title 3. OpenAlex Trace-Finder (Open-Source Aggregator).
import requests
import time
from IPython.display import Markdown, display

keywords_input = "animal communication"
batch_size = 30
max_batches = 10

def search_openalex():
    words = [w.strip().lower() for w in keywords_input.replace(",", " ").replace("_", " ").split() if w.strip()]
    query = " ".join(words)
    print(f"Searching OpenAlex for: {query}...\n")

    base_url = "https://api.openalex.org/works"
    all_results = []

    for batch_number in range(1, max_batches + 1):
        params = {"search": query, "filter": "is_oa:true", "per-page": batch_size, "page": batch_number}

        try:
            response = requests.get(base_url, params=params).json()
            items = response.get("results", [])
        except:
            print("Error reaching API.")
            break

        if not items:
            break

        print(f"\n--- BATCH {batch_number} (OpenAlex) ---")
        for i, res in enumerate(items, 1):
            title = res.get("title", "Unknown Title")
            year = res.get("publication_year", "Unknown Year")
            print(f"{(batch_number - 1) * batch_size + i}. {title} ({year})")

            authors = ", ".join([a.get("author", {}).get("display_name", "") for a in res.get("authorships", [])]) or "Unknown"
            url = res.get("doi") or res.get("id")

            all_results.append({"title": title, "year": year, "authors": authors, "url": url})

        if len(items) < batch_size:
            break

        print("\nWaiting 5 seconds...")
        time.sleep(5)

    if all_results:
        md_content = f"### OpenAlex Results ({len(all_results)} fetched)\n\n"
        for r in all_results:
            md_content += f"* **[{r['title']}]({r['url']})** ({r['year']}) <br> 👤 *{r['authors']}*\n\n"
        display(Markdown(md_content))

search_openalex()

Searching OpenAlex for: animal communication...


--- BATCH 1 (OpenAlex) ---
1. Animal Communication: When I’m Calling You, Will You Answer Too? (2017)
2. Principles of Animal Communication (1999)
3. The evolution of animal communication (2013)
4. SPECIES RECOGNITION AND SEXUAL SELECTION AS A UNITARY PROBLEM IN ANIMAL COMMUNICATION (1993)
5. A systems approach to animal communication (2016)
6. Optical Signals: Animal Communication and Light (1979)
7. Rough‐and‐tumble play as a window on animal communication (2015)
8. Optical Signals. Animal Communication and Light (1978)
9. C. elegans Ciliated Sensory Neurons Release Extracellular Vesicles that Function in Animal Communication (2014)
10. Syntax and compositionality in animal communication (2019)
11. The Evolution of Animal Communication: Reliability and Deception in Signaling Systems (2006)
12. Taking turns: bridging the gap between human and animal communication (2018)
13. Alert signals enhance animal communication in “noisy” environm

### OpenAlex Results (300 fetched)

* **[Animal Communication: When I’m Calling You, Will You Answer Too?](https://doi.org/10.1016/j.cub.2017.05.064)** (2017) <br> 👤 *Neil J. Vickers*

* **[Principles of Animal Communication](https://doi.org/10.1093/ae/45.2.126)** (1999) <br> 👤 *Ralph W. Howard*

* **[The evolution of animal communication](https://doi.org/10.1163/1568539x-00003098)** (2013) <br> 👤 *Marc Naguib, J. Jordan Price*

* **[SPECIES RECOGNITION AND SEXUAL SELECTION AS A UNITARY PROBLEM IN ANIMAL COMMUNICATION](https://doi.org/10.1111/j.1558-5646.1993.tb02118.x)** (1993) <br> 👤 *Michael J. Ryan, A. Stanley Rand*

* **[A systems approach to animal communication](https://doi.org/10.1098/rspb.2015.2889)** (2016) <br> 👤 *Eileen A. Hebets, Andrew B. Barron, Christopher N. Balakrishnan, Márk E. Hauber, Paul H. Mason, Kim L. Hoke*

* **[Optical Signals: Animal Communication and Light](https://doi.org/10.2307/3494073)** (1979) <br> 👤 *John E. Lloyd, Jack P. Hailman*

* **[Rough‐and‐tumble play as a window on animal communication](https://doi.org/10.1111/brv.12172)** (2015) <br> 👤 *Elisabetta Palagi, Gordon M. Burghardt, Barbara Smuts, Giada Cordoni, Stefania Dall’Olio, Hillary N. Fouts, Milada Řeháková‐Petrů, Stephen M. Siviy, Sergio M. Pellis*

* **[Optical Signals. Animal Communication and Light](https://doi.org/10.2307/4085533)** (1978) <br> 👤 *W. J. Bernhard Smith*

* **[C. elegans Ciliated Sensory Neurons Release Extracellular Vesicles that Function in Animal Communication](https://doi.org/10.1016/j.cub.2014.01.002)** (2014) <br> 👤 *Juan Wang, Malan Silva, Leonard A. Haas, Natalia S. Morsci, Ken C. Q. Nguyen, David H. Hall, Maureen M. Barr*

* **[Syntax and compositionality in animal communication](https://doi.org/10.1098/rstb.2019.0062)** (2019) <br> 👤 *Klaus Zuberbühler*

* **[The Evolution of Animal Communication: Reliability and Deception in Signaling Systems](https://doi.org/10.1650/0010-5422(2006)108[989:br]2.0.co;2)** (2006) <br> 👤 *Timothy F. Wright*

* **[Taking turns: bridging the gap between human and animal communication](https://doi.org/10.1098/rspb.2018.0598)** (2018) <br> 👤 *Simone Pika, Ray Wilkinson, Kobin H. Kendrick, Sonja C. Vernes*

* **[Alert signals enhance animal communication in “noisy” environments](https://doi.org/10.1073/pnas.0807657105)** (2008) <br> 👤 *Terry J. Ord, Judy A. Stamps*

* **[Why Are No Animal Communication Systems Simple Languages?](https://doi.org/10.3389/fpsyg.2021.602635)** (2021) <br> 👤 *Michael D. Beecher*

* **[Animal Communication Networks](https://doi.org/10.1650/0010-5422(2006)108[485:acn]2.0.co;2)** (2006) <br> 👤 *Lauren P. Reed*

* **[Anthropogenic Noise and its Effect on Animal Communication: An Interface Between Comparative Psychology and Conservation Biology](https://doi.org/10.46867/c4f59p)** (2003) <br> 👤 *Lawrence A. Rabin, Brenda McCowan, Stacie L. Hooper, Donald H. Owings*

* **[Animal Communication: Techniques of Study and Results of Research](https://doi.org/10.2979/animalcommunicationt)** (2021) <br> 👤 *Thomas Sebeok*

* **[ADAPTATION AND PLASTICITY OF ANIMAL COMMUNICATION IN FLUCTUATING ENVIRONMENTS](https://doi.org/10.1111/j.1558-5646.2010.01056.x)** (2010) <br> 👤 *Terry J. Ord, Judy A. Stamps, Jonathan B. Losos*

* **[Complex spatial group patterns result from different animal communication mechanisms](https://doi.org/10.1073/pnas.0611483104)** (2007) <br> 👤 *Raluca Eftimie, Gerda de Vries, Mark A. Lewis*

* **[Vibration and Animal Communication: A Review](https://doi.org/10.1093/icb/41.5.1135)** (2001) <br> 👤 *Peggy S. M. Hill*

* **[Animal Communication: Techniques of Study and Results of Research](https://doi.org/10.1093/besa/15.4.379)** (1969) <br> 👤 *Tayyab Walker*

* **[Categorical perception in animal communication and decision-making](https://doi.org/10.1093/beheco/araa004)** (2020) <br> 👤 *Patrick A. Green, Nicholas C. Brandley, Stephen Nowicki*

* **[Song type matching is an honest early threat signal in a hierarchical animal communication system](https://doi.org/10.1098/rspb.2012.2517)** (2013) <br> 👤 *Çağlar Akçay, Mari E. Tom, S. Elizabeth Campbell, Michael D. Beecher*

* **[Living in a noisy world: indirect effects of noise on animal communication](https://doi.org/10.1163/1568539x-00003058)** (2013) <br> 👤 *Marc Naguib*

* **[Animal Sounds and Communication](https://doi.org/10.1093/besa/7.2.109)** (1961) <br> 👤 *Mable Frings, M. Frings*

* **[Optical Signals: Animal Communication and Light](https://doi.org/10.2979/opticalsignalsanimal)** (2021) <br> 👤 *Jack Hailman*

* **[Animal Communication: Timing Counts](https://doi.org/10.1016/j.cub.2007.04.053)** (2007) <br> 👤 *Henrik Brumm, P.J.B. Slater*

* **[Emotional and Interactional Prosody across Animal Communication Systems: A Comparative Approach to the Emergence of Language](https://doi.org/10.3389/fpsyg.2016.01393)** (2016) <br> 👤 *Piera Filippi*

* **[The use of network analysis to study complex animal communication systems: a study on nightingale song](https://doi.org/10.1098/rspb.2014.0460)** (2014) <br> 👤 *Michael N. Weiss, Henrike Hultsch, Iris Adam, Constance Scharff, Silke Kipper*

* **[What can animal communication teach us about human language?](https://doi.org/10.1098/rstb.2019.0042)** (2019) <br> 👤 *Adam R. Fishbein, Jonathan B. Fritz, William J. Idsardi, Gerald S. Wilkinson*

* **[Animal Communication: City Birds Have Changed Their Tune](https://doi.org/10.1016/j.cub.2006.10.043)** (2006) <br> 👤 *Henrik Brumm*

* **[Integrating multiple disciplines to understand effects of anthropogenic noise on animal communication](https://doi.org/10.1002/ecs2.2127)** (2018) <br> 👤 *Patrícia Rosa, Nicola Koper*

* **[Signalling in groups: New tools for the integration of animal communication and collective movement](https://doi.org/10.1111/2041-210x.13939)** (2022) <br> 👤 *Vlad Demartsev, Andrew S. Gersick, Frants H. Jensen, Mara Thomas, Marie A. Roch, Marta B. Manser, Ariana Strandburg‐Peshkin*

* **[Animal Communication in Linguistic and Cognitive Perspective](https://doi.org/10.1146/annurev-linguistics-030421-061233)** (2022) <br> 👤 *Thom Scott‐Phillips, Christophe Heintz*

* **[You talkin’ to me? Interactive playback is a powerful yet underused tool in animal communication research](https://doi.org/10.1098/rsbl.2015.0403)** (2015) <br> 👤 *Stephanie L. King*

* **[Animal Communication: Do Dolphins Have Names?](https://doi.org/10.1016/j.cub.2006.07.002)** (2006) <br> 👤 *Robert A. Barton*

* **[How Signaling Geometry Shapes the Efficacy and Evolution of Animal Communication Systems](https://doi.org/10.1093/icb/icab090)** (2021) <br> 👤 *Sebastian Echeverri, Audrey Miller, Jason Chen, Eden W. McQueen, Melissa S Plakke, Michelle Elise Spicer, Kim L. Hoke, Mary Caswell Stoddard, Nathan I. Morehouse*

* **[Animal Communication and Human Language: An overview](https://doi.org/10.46867/ijcp.2016.29.00.07)** (2016) <br> 👤 *Leonardo Barón Birchenall*

* **[Automated Interactive Video Playback for Studies of Animal Communication](https://doi.org/10.3791/2374)** (2011) <br> 👤 *Trisha Butkowski, Wei Yan, Aaron Gray, Rongfeng Cui, Machteld N. Verzijden, Gil G. Rosenthal*

* **[Optionality in animal communication: a novel framework for examining the evolution of arbitrariness](https://doi.org/10.1111/brv.12882)** (2022) <br> 👤 *Stuart K. Watson, Piera Filippi, Luca Gasparri, Nikola Falk, Nicole Tamer, Paul Widmer, Marta B. Manser, Hans‐Johann Glock*

* **[Dynamic changes in display architecture and function across environments revealed by a systems approach to animal communication*](https://doi.org/10.1111/evo.13448)** (2018) <br> 👤 *Malcolm F. Rosenthal, Matthew R. Wilkins, Daizaburo Shizuka, Eileen A. Hebets*

* **[A Study on Economic Models of Animal Communication Methods](https://doi.org/10.46632/jitl/3/2/2)** (2024) <br> 👤 *Unknown*

* **[Animal Communication: Hidden Complexity](https://doi.org/10.1016/j.cub.2013.10.001)** (2013) <br> 👤 *Jessica C. Flack*

* **[Cybernetics, or Control and Communication in the Animal and the Machine](https://doi.org/10.2307/1266498)** (1963) <br> 👤 *Enders A. Robinson, Norbert Wiener*

* **[Signaling in a Polluted World: Oxidative Stress as an Overlooked Mechanism Linking Contaminants to Animal Communication](https://doi.org/10.3389/fevo.2016.00095)** (2016) <br> 👤 *Valeria Marasco, David Costantini*

* **[Meaning in animal and human communication](https://doi.org/10.1007/s10071-015-0845-5)** (2015) <br> 👤 *Thomas C. Scott‐Phillips*

* **[Brevity is not a universal in animal communication: evidence for compression depends on the unit of analysis in small ape vocalizations](https://doi.org/10.1098/rsos.200151)** (2020) <br> 👤 *Dena J. Clink, Abdul Hamid Ahmad, Holger Klinck*

* **[Overcoming bias in the comparison of human language and animal communication](https://doi.org/10.1073/pnas.2218799120)** (2023) <br> 👤 *Erica A. Cartmill*

* **[Animal Communication and Human Language: An overview](https://openalex.org/W4300434843)** (2016) <br> 👤 *Leonardo Barón Birchenall*

* **[The Evolution of Animal Communication Systems: Questions of Function Examined through Simulation](https://openalex.org/W1562514527)** (1998) <br> 👤 *Jason Noble*

* **[Quantification of Information in a One-Way Plant-to-Animal Communication System](https://doi.org/10.3390/e110300431)** (2009) <br> 👤 *Laurance R. Doyle*

* **[New Synthesis: Animal Communication Mediated by Microbes: Fact or Fantasy?](https://doi.org/10.1007/s10886-013-0343-7)** (2013) <br> 👤 *Angela E. Douglas, A. Dobson*

* **[The Evolution of Animal Communication: Reliability and Deception in Signaling Systems](https://doi.org/10.1641/0006-3568(2006)56[849:waash]2.0.co;2)** (2006) <br> 👤 *H. Jane Brockmann*

* **[New Dimensions for Animal Communication Networks:Space and Time](https://doi.org/10.1093/icb/icab013)** (2021) <br> 👤 *Michael S. Reichert, Maya Enriquez, Nora V. Carlson*

* **[Not your private tête-à-tête: leveraging the power of higher-order networks to study animal communication](https://doi.org/10.1098/rstb.2023.0190)** (2024) <br> 👤 *Iacopo Iacopini, Jennifer R. Foote, Nina H. Fefferman, Elizabeth P. Derryberry, Matthew J. Silk*

* **[Prosocial behaviour in animals: the influence of social relationships, communication and rewards](https://doi.org/10.1016/j.anbehav.2012.08.009)** (2012) <br> 👤 *Katherine A. Cronin*

* **[Animal cognition and the evolution of human language: why we cannot focus solely on communication](https://doi.org/10.1098/rstb.2019.0046)** (2019) <br> 👤 *W. Tecumseh Fitch*

* **[The “Law of Brevity” in animal communication: Sex-specific signaling optimization is determined by call amplitude rather than duration](https://doi.org/10.1002/evl3.147)** (2019) <br> 👤 *Vlad Demartsev, Naomi Gordon, Adi Barocas, Einat Bar‐Ziv, Tchia Ilany, Yael Goll, Amiyaal Ilany, Eli Geffen*

* **[Automated Interactive Video Playback for Studies of Animal Communication](https://doi.org/10.3791/2374-v)** (2011) <br> 👤 *Trisha Butkowski, Wei Yan, Aaron Gray, Rongfeng Cui, Machteld N. Verzijden, Gil G. Rosenthal*

* **[Short Communication: Renal Tubular Vacuolation in Animals Treated with Polyethylene-Glycol-Conjugated Proteins](https://doi.org/10.1093/toxsci/42.2.152)** (1998) <br> 👤 *Alison M. Bendele, Jim Seely, Carl Richey, Gina Sennello, George M. Shopp*

* **[Plant–animal communication](https://doi.org/10.1093/aob/mcs273)** (2012) <br> 👤 *Jarmo K. Holopainen*

* **[Parasite effects on receivers in animal communication: Hidden impacts on behavior, ecology, and evolution](https://doi.org/10.1073/pnas.2300186120)** (2023) <br> 👤 *Michael S. Reichert, Matthew G. Bolek, Elizabeth A. McCullagh*

* **[Bird sounds in nature writing: Human perspective on animal communication](https://doi.org/10.12697/sss.2009.37.3-4.11)** (2009) <br> 👤 *Kadri Tüür*

* **[Animal communication and sentience](https://doi.org/10.51291/2377-7478.1819)** (2023) <br> 👤 *Catia Correia‐Caeiro, Katja Liebal*

* **[Testing a key assumption in animal communication: between-individual variation in female visual systems alters perception of male signals](https://doi.org/10.1242/bio.028282)** (2017) <br> 👤 *Kelly L. Ronald, Amanda L. Ensminger, Matthew D. Shawkey, Jeffrey R. Lucas, Esteban Fernández‐Juricic*

* **[Animal Communication: Laughter Is the Shortest Distance between Two Apes](https://doi.org/10.1016/j.cub.2009.05.007)** (2009) <br> 👤 *David A. Leavens*

* **[Animal Communication: What Makes a Dog Able to Understand its Master?](https://doi.org/10.1016/s0960-9822(03)00271-9)** (2003) <br> 👤 *Richard W. Byrne*

* **[Animal Communication: Learning by Listening about Danger](https://doi.org/10.1016/j.cub.2018.07.039)** (2018) <br> 👤 *Christopher N. Templeton*

* **[Introduction: A primer on information and influence in animal communication](https://doi.org/10.1017/cbo9781139003551.002)** (2013) <br> 👤 *Ulrich E. Stegmann*

* **[AN ANIMAL MODEL OF THE INTERPERSONAL COMMUNICATION OF INTEROCEPTIVE (PRIVATE) STATES](https://doi.org/10.1901/jeab.1987.48-1)** (1987) <br> 👤 *David Lubinski, Travis Thompson*

* **[The syntax–semantics interface in animal vocal communication](https://doi.org/10.1098/rstb.2018.0405)** (2019) <br> 👤 *Toshitaka N. Suzuki, David Wheatcroft, Michael Griesser*

* **[Complexity in Animal Communication: Estimating the Size of N-Gram Structures](https://doi.org/10.3390/e16010526)** (2014) <br> 👤 *Reginald D. Smith*

* **[MicroRNAs from plants to animals, do they define a new messenger for communication?](https://doi.org/10.1186/s12986-018-0305-8)** (2018) <br> 👤 *Zhiqing Li, Ruodan Xu, Ning Li*

* **[Overlap not gap: Understanding the relationship between animal communication and language with Prototype Theory](https://doi.org/10.1016/j.lingua.2022.103332)** (2022) <br> 👤 *Jenny Amphaeris, Graeme Shannon, Thora Tenbrink*

* **[Veterinarian-client-patient communication patterns used during clinical appointments in companion animal practice](https://doi.org/10.2460/javma.228.5.714)** (2006) <br> 👤 *Jane R. Shaw, Brenda N. Bonnett, Cindy L. Adams, Debra Roter*

* **[Animal Communication](https://doi.org/10.2307/3493908)** (1977) <br> 👤 *Thomas Walker, Hubert Frings, Mable Frings*

* **[Biology, environment, and culture: From animal communication to human language and cognition](https://doi.org/10.21638/spbu17.2020.113)** (2020) <br> 👤 *Tatiana V. Chernigovskaya*

* **[Plant and Animal microRNAs (miRNAs) and Their Potential for Inter-kingdom Communication](https://doi.org/10.1007/s10571-017-0547-4)** (2017) <br> 👤 *Yuhai Zhao, Lin Cong, Walter J. Lukiw*

* **[Animal Communication Overview](https://doi.org/10.1017/9781108564113.003)** (2021) <br> 👤 *Federico Rossano, Stephan P. Kaufhold*

* **[The species Severe acute respiratory syndrome-related coronavirus: classifying 2019-nCoV and naming it SARS-CoV-2](https://doi.org/10.1038/s41564-020-0695-z)** (2020) <br> 👤 *Alexander E. Gorbalenya, Susan C. Baker, Ralph S. Baric, Raoul J. de Groot, Christian Drosten, Anastasia Gulyaeva, Bart L. Haagmans, Chris Lauber, A. M. Leontovich, Benjamin W. Neuman, Dmitry Penzar, Stanley Perlman, Leo L. M. Poon, Dmitry V. Samborskiy, Igor A. Sidorov, Isabel Sola, John Ziebuhr*

* **[Using cross-disciplinary knowledge to facilitate advancements in animal communication and science communication research](https://doi.org/10.1242/jeb.179978)** (2018) <br> 👤 *Eileen A. Hebets, Alissa G. Anderson*

* **[Bound for Specific Sounds: Vocal Predisposition in Animal Communication](https://doi.org/10.1016/j.tics.2020.05.013)** (2020) <br> 👤 *Maxime Garcia, Marta B. Manser*

* **[Deep‐time convergent evolution in animal communication presented by shared adaptations for coping with noise in lizards and other animals](https://doi.org/10.1111/ele.13773)** (2021) <br> 👤 *Terry J. Ord, Danielle A. Klomp, Thomas C. Summers, Arvin C. Diesmos, Norhayati Ahmad, Indraneil Das*

* **[Animal Communication: Big Talkers and Small Talk](https://doi.org/10.1016/j.cub.2007.02.007)** (2007) <br> 👤 *Kevin G. Munhall, Stacie K. Byrne*

* **[Animal communication in linguistic &amp;amp; cognitive perspective](https://doi.org/10.31234/osf.io/vnc5m)** (2022) <br> 👤 *Thom Scott‐Phillips, Christophe Heintz*

* **[Citizens’ and Farmers’ Framing of ‘Positive Animal Welfare’ and the Implications for Framing Positive Welfare in Communication](https://doi.org/10.3390/ani9040147)** (2019) <br> 👤 *Belinda Vigors*

* **[Animal Communication: Sniffing Is About More Than Just Smell](https://doi.org/10.1016/j.cub.2013.02.030)** (2013) <br> 👤 *Bennett G. Galef*

* **[Information Theory Applied to Animal Communication Systems and Its Possible Application to SETI](https://doi.org/10.1017/s0074180900193817)** (2004) <br> 👤 *Sean F. Hanser, Laurance R. Doyle, Brenda McCowan, Jon M. Jenkins*

* **[Animal Models of Speech and Vocal Communication Deficits Associated With Psychiatric Disorders](https://doi.org/10.1016/j.biopsych.2015.07.001)** (2015) <br> 👤 *Geneviève Konopka, Todd F. Roberts*

* **[How Animals Communicate](https://doi.org/10.2979/howanimalscommunicat)** (2021) <br> 👤 *Thomas A. Sebeok*

* **[Short communication: On recognizing the proper experimental unit in animal studies in the dairy sciences](https://doi.org/10.3168/jds.2016-11516)** (2016) <br> 👤 *Nora M. Bello, Matthew Kramer, Robert J. Tempelman, Walter W. Stroup, N.R. St-Pierre, Bruce Α. Craig, Linda J. Young, Edward E. Gbur*

* **[Political Communication with Animals](https://doi.org/10.52537/humanimalia.9966)** (2013) <br> 👤 *Eva Meijer*

* **[Individual variation in animal communication: from species averages to unique voices](https://doi.org/10.1111/brv.70061)** (2025) <br> 👤 *Angèle Lombrey, Marlen Fröhlich*

* **[Communication is key: extracellular vesicles as mediators of infection and defence during host–microbe interactions in animals and plants](https://doi.org/10.1093/femsre/fuab044)** (2021) <br> 👤 *Henrik U. Stotz, Dominik Brotherton, Jameel M. Inal*

* **[Use of the Roter interaction analysis system to analyze veterinarian-client-patient communication in companion animal practice](https://doi.org/10.2460/javma.2004.225.222)** (2004) <br> 👤 *Jane R. Shaw, Cindy L. Adams, Brenda N. Bonnett, Susan Larson, Debra Roter*

* **[Spatio-temporal Dynamics in Animal Communication: A Special Issue Arising from a Unique Workshop-Symposium Model](https://doi.org/10.1093/icb/icab151)** (2021) <br> 👤 *Kim L. Hoke, Nicholai M. Hensley, Jessleen K Kanwal, Sara Wasserman, Nathan I. Morehouse*

* **[Animal Communication: Origins of Sequential Structure in Birdsong](https://doi.org/10.1016/j.cub.2017.10.041)** (2017) <br> 👤 *Ofer Tchernichovski, Dina Lipkind*

* **[The representation of animal communication and language evolution in introductory linguistics textbooks](https://doi.org/10.1093/jole/lzac010)** (2022) <br> 👤 *Sławomir Wacewicz, Michael Pleyer, Aleksandra Szczepańska, Aleksandra Ewa Poniewierska, Przemysław Żywiczyński*

* **[Evolving grounded communication for robots](https://doi.org/10.1016/s1364-6613(03)00129-3)** (2003) <br> 👤 *Luc Steels*

* **[A Theory of Unsupervised Translation Motivated by Understanding Animal Communication](https://doi.org/10.48550/arxiv.2211.11081)** (2022) <br> 👤 *Shafi Goldwasser, David F. Gruber, Adam Tauman Kalai, Orr Paradise*

* **[Songs, Roars, and Rituals: Communication in Birds, Mammals, and Other Animals](https://doi.org/10.1093/ae/46.4.266a)** (2000) <br> 👤 *Unknown*

* **[Animal Communication: Flies' Ears Are Tuned In](https://doi.org/10.1016/j.cub.2011.03.034)** (2011) <br> 👤 *Elina Immonen, Michael G. Ritchie*

* **[Animal communication](https://doi.org/10.1038/275778a0)** (1978) <br> 👤 *Patricia Slater*

* **[Animal communication in ethological research](https://doi.org/10.1038/270120a0)** (1977) <br> 👤 *John Krebs*

* **[Communication with self, friends and foes in active-sensing animals](https://doi.org/10.1242/jeb.242637)** (2021) <br> 👤 *Te K. Jones, Kathryne M. Allen, Cynthia F. Moss*

* **[Veterinarian-client-patient communication during wellness appointments versus appointments related to a health problem in companion animal practice](https://doi.org/10.2460/javma.233.10.1576)** (2008) <br> 👤 *Jane R. Shaw, Cindy L. Adams, Brenda N. Bonnett, Susan Larson, Debra Roter*

* **[Evolution of animal chemical communication: Insights from non-model species and phylogenetic comparative methods](https://doi.org/10.26496/bjz.2019.31)** (2019) <br> 👤 *Simon Baeckens*

* **[Animal Vocal Communication: Say What?](https://doi.org/10.7551/mitpress/1885.003.0043)** (2002) <br> 👤 *Drew Rendall, Michael J. Owren*

* **[Individual variation in animal communication: from species averages to unique voices](https://doi.org/10.32942/x2j63s)** (2025) <br> 👤 *Angèle Lombrey, Marlen Fröhlich*

* **[Animal communication in a human-dominated world: a comment on Radford et al.](https://doi.org/10.1093/beheco/aru139)** (2014) <br> 👤 *Bob B. M. Wong*

* **[Ingestion of <i>Lactobacillus</i> strain regulates emotional behavior and central GABA receptor expression in a mouse via the vagus nerve](https://doi.org/10.1073/pnas.1102999108)** (2011) <br> 👤 *Javier A. Bravo, Paul Forsythe, Marianne V. Chew, Emily Escaravage, Hélène M. Savignac, Timothy G. Dinan, John Bienenstock, John F. Cryan*

* **[Syntax in animal communication: its study in songbirds and other taxa](https://doi.org/10.1111/jav.03258)** (2024) <br> 👤 *Heather Wolverton, Rindy C. Anderson*

* **[Team-Based Learning Increases Active Engagement and Enhances Development of Teamwork and Communication Skills in a First-Year Course for Veterinary and Animal Science Undergraduates](https://doi.org/10.3138/jvme.0213-034r1)** (2013) <br> 👤 *Susan Hazel, Nicole Heberle, Margaret-Mary McEwen, Karen E. Adams*

* **[Know thine enemy: fighting fish gather information from observing conspecific interactions](https://doi.org/10.1098/rspb.1998.0397)** (1998) <br> 👤 *Rui F. Oliveira, Peter K. McGregor, Claire Latruffe*

* **[Microbially Mediated Chemical Ecology of Animals: A Review of Its Role in Conspecific Communication, Parasitism and Predation](https://doi.org/10.3390/biology10040274)** (2021) <br> 👤 *Mónica Mazorra-Alonso, Gustavo Tomás, Juan José Soler*

* **[Dairy 4.0: Intelligent Communication Ecosystem for the Cattle Animal Welfare with Blockchain and IoT Enabled Technologies](https://doi.org/10.3390/app12147316)** (2022) <br> 👤 *Anita Gehlot, Praveen Kumar Malik, Rajesh Singh, Shaik Vaseem Akram, Turki Alsuwian*

* **[A minimum‐impact, flexible tool to study vocal communication of small animals with precise individual‐level resolution](https://doi.org/10.1111/2041-210x.12610)** (2016) <br> 👤 *Lisa Gill, Pietro B. D’Amelio, Nicolás M. Adreani, Hannes Sagunsky, Manfred C. Gahr, Andries Ter Maat*

* **[Arguments for the Integration of the Non-Zero-Sum Logic of Complex Animal Communication with Information Theory](https://doi.org/10.3390/e12010127)** (2010) <br> 👤 *Vincenzo Penteriani*

* **[Commentary: Why Are No Animal Communication Systems Simple Languages?](https://doi.org/10.3389/fpsyg.2021.722685)** (2021) <br> 👤 *Dustin J. Penn, Szabolcs Számadó*

* **[Using animal communication strategy (ACS) for MANET routing](https://doi.org/10.4038/jnsfsr.v43i3.7948)** (2015) <br> 👤 *C. T. Manimegalai, C. Jayakumar, G. Gunasekaran*

* **[Rhythm and synchrony in animal movement and communication](https://doi.org/10.1093/cz/zoy087)** (2018) <br> 👤 *Andrea Ravignani*

* **[Facial Displays Are Tools for Social Influence](https://doi.org/10.1016/j.tics.2018.02.006)** (2018) <br> 👤 *Carlos Crivelli, Alan J. Fridlund*

* **[Meaning in Animal Communication: Varieties of meaning and their roles in explaining communication](https://doi.org/10.25911/5d5148b321836)** (2018) <br> 👤 *David Kalkman*

* **[The Evolution of Rhythm Processing](https://doi.org/10.1016/j.tics.2018.08.002)** (2018) <br> 👤 *Sonja A. Kotz, Andrea Ravignani, W. Tecumseh Fitch*

* **[Carotenoid limitation of sexual coloration along an environmental gradient in guppies](https://doi.org/10.1098/rspb.1999.0781)** (1999) <br> 👤 *Gregory F. Grether, Jocelyn Hudon, David F. Millie*

* **[Evolving views on cognition in animal vocal communication: Contributions from scream research](https://doi.org/10.26451/abc.07.02.12.2020)** (2020) <br> 👤 *Jay W. Schwartz, Jonathan Engelberg, Harold Gouzoules*

* **[Animal-Assisted Therapy Improves Communication and Mobility among Institutionalized People with Cognitive Impairment](https://doi.org/10.3390/ijerph17165899)** (2020) <br> 👤 *María Dolores Rodrigo Claverol, Belén Malla-Clua, Carme Marquilles-Bonet, Joaquím Sol, Júlia Jové-Naval, Meritxell Sole-Pujol, Marta Ortega*

* **[Red deer stags use formants as assessment cues during intrasexual agonistic interactions](https://doi.org/10.1098/rspb.2004.2954)** (2005) <br> 👤 *David Reby, Karen McComb, Bruno Cargnelutti, C. J. Darwin, W. Tecumseh Fitch, Tim Clutton‐Brock*

* **[A one health glossary to support communication and information exchange between the human health, animal health and food safety sectors](https://doi.org/10.1016/j.onehlt.2021.100263)** (2021) <br> 👤 *Tasja Buschhardt, Taras Günther, Taran Skjerdal, Mia Torpdahl, Jörn Gethmann, Maria‐Eleni Filippitzi, C.B.M. Maassen, Solveig Jore, Johanne Ellis‐Iversen, Matthias Filter*

* **[What evidence exists on the effects of anthropogenic noise on acoustic communication in animals? A systematic map protocol](https://doi.org/10.1186/s13750-019-0165-3)** (2019) <br> 👤 *Oded Berger‐Tal, Bob B. M. Wong, Ulrika Candolin, Jesse R. Barber*

* **[The effects of forest gaps on animal communication networks](https://openalex.org/W2950424053)** (2019) <br> 👤 *Cedar Arran Mathers-Winn*

* **[Finding, visualizing, and quantifying latent structure across diverse animal vocal repertoires](https://doi.org/10.1371/journal.pcbi.1008228)** (2020) <br> 👤 *Tim Sainburg, Marvin Thielk, Timothy Q. Gentner*

* **[Animal Communication: Chit-Chat in Meerkats](https://doi.org/10.1016/j.cub.2018.09.038)** (2018) <br> 👤 *Daniel Y. Takahashi*

* **[A functional data analysis approach for modelling frequency‐modulated tonal sounds in animal communication](https://doi.org/10.1111/2041-210x.14446)** (2025) <br> 👤 *Maria Silvia Labriola, Petra Oswine Pammer, Giulia Pedrazzi, Giancarlo Giacomini, Giovanna Jona Lasinio, Daniela Silvia Pace*

* **[Rhythm interaction in animal groups: selective attention in communication networks](https://doi.org/10.1098/rstb.2020.0338)** (2021) <br> 👤 *Michael D. Greenfield, Ikkyu Aihara, Guy Amichay, Marianna Anichini, Vivek Nityananda*

* **[Data warehouse for assessing animal health, welfare, risk management and –communication](https://doi.org/10.1186/1751-0147-53-s1-s3)** (2011) <br> 👤 *Annette Cleveland Nielsen*

* **[What is it Like to be a ‘Batty Telepathic Woman’? Considering Animal Communication in the Anthropocene](https://doi.org/10.14220/trns.2022.1.1.85)** (2022) <br> 👤 *Mary Trachsel*

* **[Animal board invited review: Animal agriculture and alternative meats – learning from past science communication failures](https://doi.org/10.1016/j.animal.2021.100360)** (2021) <br> 👤 *Alison L. Van Eenennaam, Samantha J. Werth*

* **[Maps and streams in the auditory cortex: nonhuman primates illuminate human speech processing](https://doi.org/10.1038/nn.2331)** (2009) <br> 👤 *Josef P. Rauschecker, Sophie K. Scott*

* **[Body size and vocalization in primates and carnivores](https://doi.org/10.1038/srep41070)** (2017) <br> 👤 *Daniel L. Bowling, Maxime Garcia, Jacob C. Dunn, R. Ruprecht, A. Stewart, Karl‐Heinz Frommolt, W. Tecumseh Fitch*

* **[Modelling Animal Interactive Rhythms in Communication](https://doi.org/10.1177/1176934318823558)** (2019) <br> 👤 *Andrea Ravignani, Koen de Reus*

* **[<scp>A</scp>strocyte disruption of neurovascular communication is linked to cortical damage in an animal model of multiple sclerosis](https://doi.org/10.1002/glia.23304)** (2018) <br> 👤 *Raya Eilam, Menahem Segal, Rafael Malach, Michael Sela, Ruth Arnon, Rina Aharoni*

* **[The impact of environmental noise on animal communication: pattern formation in a class of deterministic and stochastic hyperbolic models for self-organised biological aggregations](https://doi.org/10.11145/j.biomath.2018.07.217)** (2018) <br> 👤 *Raluca Eftimie*

* **[The Meanings of Chimpanzee Gestures](https://doi.org/10.1016/j.cub.2014.05.066)** (2014) <br> 👤 *Catherine Hobaiter, Richard W. Byrne*

* **[Vibrational and Acoustic Communication in Animals](https://doi.org/10.1007/978-3-030-97540-1_11)** (2022) <br> 👤 *Rebecca A. Dunlop, William L. Gannon, Marthe Kiley-Worthington, Peggy S. M. Hill, Andreas Wessel, Jeànette A. Thomas*

* **[Commentary: Why Are no Animal Communication Systems Simple Languages?](https://doi.org/10.3389/fpsyg.2021.763445)** (2021) <br> 👤 *Sławomir Wacewicz*

* **[Mitochondria and Chloroplasts Shared in Animal and Plant Tissues: Significance of Communication](https://doi.org/10.12659/msm.894481)** (2015) <br> 👤 *George B. Stefano*

* **[Animal Communication: Reading Lizards' Body Language in Context](https://doi.org/10.1016/j.cub.2007.07.006)** (2007) <br> 👤 *Johannes M. Zanker*

* **[The gut-brain axis: interactions between enteric microbiota, central and enteric nervous systems.](https://openalex.org/W1663623302)** (2015) <br> 👤 *Marilia Carabotti, Annunziata Scirocco, M.A. Maselli, Carola Severi*

* **[What If We Understood What Animals Are Saying? The Legal Impact Of Ai-assisted Studies Of Animal Communication](https://doi.org/10.2139/ssrn.5165527)** (2025) <br> 👤 *César Rodríguez-Garavito, David F. Gruber*

* **[Campbell's monkeys concatenate vocalizations into context-specific call sequences](https://doi.org/10.1073/pnas.0908118106)** (2009) <br> 👤 *Karim Ouattara, Alban Lemasson, Klaus Zuberbühler*

* **[The emerging significance of bioacoustics in animal species conservation](https://doi.org/10.1016/j.biocon.2010.03.025)** (2010) <br> 👤 *Paola Laiolo*

* **[Intercellular communication and the organization of simple multicellular animals](https://doi.org/10.1016/j.cdev.2021.203726)** (2021) <br> 👤 *Pablo Japón, Francisco J. Morales, Fernando Casares*

* **[Hunting at the highway: traffic noise reduces foraging efficiency in acoustic predators](https://doi.org/10.1098/rspb.2010.2262)** (2010) <br> 👤 *Björn M. Siemers, Andrea Schaub*

* **[The Language Myth: Why Language Is Not an Instinct](https://doi.org/10.1017/cbo9781107358300)** (2014) <br> 👤 *Vyvyan Evans*

* **[Neuroscience-Inspired Artificial Intelligence](https://doi.org/10.1016/j.neuron.2017.06.011)** (2017) <br> 👤 *Demis Hassabis, Dharshan Kumaran, Christopher Summerfield, Matthew Botvinick*

* **[Bottlenose dolphins can use learned vocal labels to address each other](https://doi.org/10.1073/pnas.1304459110)** (2013) <br> 👤 *Stephanie L. King, Vincent M. Janik*

* **[Colour variation in cichlid fish: Developmental mechanisms, selective pressures and evolutionary consequences](https://doi.org/10.1016/j.semcdb.2013.05.003)** (2013) <br> 👤 *Martine E. Maan, Kristina M. Sefc*

* **[Communication: Animal Steps on the Road to Syntax?](https://doi.org/10.1016/j.cub.2017.06.066)** (2017) <br> 👤 *Andrew F. Russell, Simon W. Townsend*

* **[The Central Nervous System and the Gut Microbiome](https://doi.org/10.1016/j.cell.2016.10.027)** (2016) <br> 👤 *Gil Sharon, Timothy R. Sampson, Daniel H. Geschwind, Sarkis K. Mazmanian*

* **[How spatial constraints on efficacy and dynamic signaling alignment shape animal communication](https://openalex.org/W3112818985)** (2020) <br> 👤 *Sebastian Echeverri*

* **[Host marking behavior in phytophagous insects and parasitoids](https://doi.org/10.1046/j.1570-7458.2001.00827.x)** (2001) <br> 👤 *César R. Nufio, Daniel R. Papaj*

* **[Animal visual illusion and confusion: the importance of a perceptual perspective](https://doi.org/10.1093/beheco/art118)** (2013) <br> 👤 *Laura A. Kelley, Jennifer L. Kelley*

* **[Semantic communication in birds: evidence from field research over the past two decades](https://doi.org/10.1007/s11284-016-1339-x)** (2016) <br> 👤 *Toshitaka N. Suzuki*

* **[The Communication Between Intestinal Microbiota and Ulcerative Colitis: An Exploration of Pathogenesis, Animal Models, and Potential Therapeutic Strategies](https://doi.org/10.3389/fmed.2021.766126)** (2021) <br> 👤 *Yu Hu, Zhen Ye, Mingquan Wu, Yingqi She, Linzhen Li, Yu-Jie Xu, Kaihua Qin, Zhipeng Hu, Maoyi Yang, Fating Lu, Qiaobo Ye*

* **[Acoustic Communication in Animals](https://doi.org/10.1007/978-981-99-0831-8)** (2023) <br> 👤 *Yoshimasa Seki*

* **[Communication in production animal medicine: modelling a complex interaction with the example of dairy herd health medicine](https://doi.org/10.1186/2046-0481-64-8)** (2011) <br> 👤 *J. L. Kleen, Owen Atkinson, Jos Noordhuizen*

* **[Animal vocal communication: function, structures, and production mechanisms](https://doi.org/10.1093/cz/zox040)** (2017) <br> 👤 *Maxime Garcia, Livio Favaro*

* **[Short communication: Survey of animal-borne pathogens in the farm environment of 13 dairy operations](https://doi.org/10.3168/jds.2012-6499)** (2013) <br> 👤 *John D. Toth, Helen Aceto, Shelley C. Rankin, Zhengxia Dou*

* **[Ten unanswered questions in multimodal communication](https://doi.org/10.1007/s00265-013-1565-y)** (2013) <br> 👤 *Sarah Partan*

* **[Stenotrophomonas maltophilia: an Emerging Global Opportunistic Pathogen](https://doi.org/10.1128/cmr.00019-11)** (2012) <br> 👤 *Joanna S. Brooke*

* **[Acta Zoologica Academiae Scientiarum Hungaricae](https://doi.org/10.17109/actazoo)** (2023) <br> 👤 *Unknown*

* **[The evolutionary origins of ritualized acoustic signals in caterpillars](https://doi.org/10.1038/ncomms1002)** (2010) <br> 👤 *Jaclyn L. Scott, Akito Y. Kawahara, Jeffrey H. Skevington, Shen‐Horn Yen, Abeer Sami, Myron L. Smith, Jayne E. Yack*

* **[The harmonic-dominant signals in animal communication involve the use of new resonant frequencies](https://doi.org/10.1016/j.heliyon.2025.e44424)** (2026) <br> 👤 *Teddy Gaiddon, Thorin Jonsson, Alberto Mittone, France, Fernando Montealegre-Z, Vincent Tournat, TONY ROBILLARD*

* **[The Transition from Animal to Linguistic Communication](https://doi.org/10.1007/s13752-016-0246-2)** (2016) <br> 👤 *Harry Smit*

* **[Humans Mimicking Animals: A Cortical Hierarchy for Human Vocal Communication Sounds](https://doi.org/10.1523/jneurosci.1118-12.2012)** (2012) <br> 👤 *William J. Talkington, Kristina M. Rapuano, L. A. Hitt, Chris Frum, James W. Lewis*

* **[Evolution of Communicative Flexibility - Complexity, Creativity, and Adaptability in Human and Animal Communication. D. Kimbrough Oller and Ulrike Griebel editors.](https://doi.org/10.1093/icb/icp087)** (2009) <br> 👤 *Cory T. Miller, Michael S. Osmanski*

* **[Plasticity of Temporal Pattern Codes for Vocalization Stimuli in Primary Auditory Cortex](https://doi.org/10.1523/jneurosci.4330-05.2006)** (2006) <br> 👤 *Jan W. H. Schnupp, Thomas M. Hall, Rory Kokelaar, Bashir Ahmed*

* **[Distributed Optimization by Ant Colonies](https://openalex.org/W2297395784)** (1992) <br> 👤 *Alberto Colorni, Marco Dorigo, Vittorio Maniezzo, Francisco J. Varela, Paul Bourgine*

* **[Separating noise and function in systems of animal communication: a comparative study of aggressive signaling in crayfish](https://doi.org/10.1101/2020.08.03.234419)** (2020) <br> 👤 *Zackary A. Graham, Michael J. Angilletta*

* **[The Evolution of Animal Communication: Reliability and Deception in Signaling Systems. William A. Searcy and S. Nowicki](https://doi.org/10.1093/icb/icl027)** (2006) <br> 👤 *Scott A. MacDougall‐Shackleton*

* **[Animal Cognition: Multimodal Tactics of Orangutan Communication](https://doi.org/10.1016/j.cub.2007.07.010)** (2007) <br> 👤 *David A. Leavens*

* **[On the mathematical foundations of learning](https://doi.org/10.1090/s0273-0979-01-00923-5)** (2001) <br> 👤 *Felipe Cucker, Steve Smale*

* **[Wild chimpanzees’ use of single and combined vocal and gestural signals](https://doi.org/10.1007/s00265-017-2325-1)** (2017) <br> 👤 *Catherine Hobaiter, Richard W. Byrne, Klaus Zuberbühler*

* **[Cell communication with the neural plate is required for induction of neural markers by BMP inhibition: evidence for homeogenetic induction and implications for Xenopus animal cap and chick explant assays](https://doi.org/10.1016/j.ydbio.2008.12.034)** (2009) <br> 👤 *Claudia Linker, Irene De Almeida, Costis Papanayotou, Matthew J. Stower, Virginie Sabado, Ehsan Ghorani, Andrea Streit, Roberto Mayor, Claudio D. Stern*

* **[Exogenous plant MIR168a specifically targets mammalian LDLRAP1: evidence of cross-kingdom regulation by microRNA](https://doi.org/10.1038/cr.2011.158)** (2011) <br> 👤 *Zhang Lin, Dongxia Hou, Xi Chen, Donghai Li, Lingyun Zhu, Yujing Zhang, Jing Li, Zhen Bian, Xiangying Liang, Xing Fu Cai, Yuan Yin, Cheng Wang, Tianfu Zhang, Dihan Zhu, Dianmu Zhang, Jie Xu, Qun Chen, Yi Ba, Jing Liu, Qiang Wang, Jian‐Qun Chen, Jin Wang, Meng Wang, Qipeng Zhang, Junfeng Zhang, Ke Zen, Chen-Yu Zhang*

* **[Chilean consumers’ perception about animal welfare in dairy production systems: short communication](https://doi.org/10.1071/an14968)** (2015) <br> 👤 *Einar Vargas‐Bello‐Pérez, José Luis Riveros, Claus Köbrich, Pamela Alejandra Álvarez-Melo, Joop Lensink*

* **[Neural correlate of the construction of sentence meaning](https://doi.org/10.1073/pnas.1612132113)** (2016) <br> 👤 *Evelina Fedorenko, Terri L. Scott, Peter Brunner, William G. Coon, Brianna Pritchett, Gerwin Schalk, Nancy Kanwisher*

* **[Electro-Quasistatic Animal Body Communication for Untethered Rodent Biopotential Recording](https://doi.org/10.1038/s41598-021-81108-8)** (2021) <br> 👤 *Shreeya Sriram, Shitij Avlani, Matthew Ward, Shreyas Sen*

* **[Natural pedagogy as evolutionary adaptation](https://doi.org/10.1098/rstb.2010.0319)** (2011) <br> 👤 *Gergely Csibra, György Gergely*

* **[Prevalence of gastrointestinal parasites of stray dogs impounded by the Society for the Prevention of Cruelty to Animals (SPCA), Durban and Coast, South Africa : short communication](https://doi.org/10.4102/jsava.v81i2.124)** (2010) <br> 👤 *Samson Mukaratirwa, V.P. Singh*

* **[Urban noise and the cultural evolution of bird songs](https://doi.org/10.1098/rspb.2009.1571)** (2009) <br> 👤 *David Luther, Luis F. Baptista*

* **[The acoustic features of vowel-like <i>grunt</i> calls in chacma baboons (<i>Papio cyncephalus ursinus</i>): Implications for production processes and functions](https://doi.org/10.1121/1.418523)** (1997) <br> 👤 *Michael J. Owren, Robert M. Seyfarth, Dorothy L. Cheney*

* **[The integrative biology of type 2 diabetes](https://doi.org/10.1038/s41586-019-1797-8)** (2019) <br> 👤 *Michael Roden, Gerald I. Shulman*

* **[<i>SongExplorer</i> : A deep learning workflow for discovery and segmentation of animal acoustic communication signals](https://doi.org/10.1101/2021.03.26.437280)** (2021) <br> 👤 *Benjamin Arthur, Yun Ding, Medhini Sosale, Faduma Khalif, Elizabeth Kim, Peter Waddell, Srinivas C. Turaga, David L. Stern*

* **[Convergent minds: ostension, inference and Grice's third clause](https://doi.org/10.1098/rsfs.2016.0107)** (2017) <br> 👤 *Richard Moore*

* **[Zipf’s Law of Abbreviation and the Principle of Least Effort: Language users optimise a miniature lexicon for efficient communication](https://doi.org/10.1016/j.cognition.2017.05.001)** (2017) <br> 👤 *Jasmeen Kanwal, Kenny Smith, Jennifer Culbertson, Simon Kirby*

* **[The neuroactive potential of the human gut microbiota in quality of life and depression](https://doi.org/10.1038/s41564-018-0337-x)** (2019) <br> 👤 *Mireia Valles‐Colomer, Gwen Falony, Youssef Darzi, Ettje F. Tigchelaar, Jun Wang, Raúl Y. Tito, Carmen Schiweck, Alexander Kurilshikov, Marie Joossens, Cisca Wijmenga, Stephan Claes, Lukas Van Oudenhove, Alexandra Zhernakova, Sara Vieira‐Silva, Jeroen Raes*

* **[Variant Ionotropic Glutamate Receptors as Chemosensory Receptors in Drosophila](https://doi.org/10.1016/j.cell.2008.12.001)** (2009) <br> 👤 *Richard Benton, Kirsten Vannice, Carolina Gomez‐Diaz, Leslie B. Vosshall*

* **[The continuing 2019-nCoV epidemic threat of novel coronaviruses to global health — The latest 2019 novel coronavirus outbreak in Wuhan, China](https://doi.org/10.1016/j.ijid.2020.01.009)** (2020) <br> 👤 *David S.C. Hui, Esam I. Azhar, Tariq A. Madani, Francine Ntoumi, Richard Kock, Osman Dar, Giuseppe Ippolito, Timothy D. McHugh, Ziad A. Memish, Christian Drosten, Alimuddin Zumla, Eskild Petersen*

* **[Small Animal Veterinarians' Communication With Dog Owners From a Motivational Interviewing Perspective](https://doi.org/10.3389/fvets.2021.772589)** (2021) <br> 👤 *Karolina Brunius Enlund, Ebba Jennolf, Ann Pettersson*

* **[Lighting environment predicts the relative abundance of male colour morphs in bluefin killifish (<i>Lucania goodei</i>) populations](https://doi.org/10.1098/rspb.2002.2042)** (2002) <br> 👤 *Rebecca C. Fuller*

* **[The cost of threat displays and the stability of deceptive communication](https://doi.org/10.1006/jtbi.1995.0151)** (1995) <br> 👤 *Eldridge S. Adams, Michael Mesterton‐Gibbons*

* **[Rhythmic Analysis in Animal Communication, Speech, and Music: The Normalized Pairwise Variability Index Is a Summary Statistic of Rhythm Ratios](https://doi.org/10.3390/vibration8020012)** (2025) <br> 👤 *Yannick Jadoul, Francesca D’Orazio, Vesta Eleuteri, Jelle van der Werff, Tommaso Tufarelli, Marco Gamba, Teresa Raimondi, Andrea Ravignani*

* **[Zebra finches identify individuals using vocal signatures unique to each call type](https://doi.org/10.1038/s41467-018-06394-9)** (2018) <br> 👤 *Julie E. Elie, Frédéric E. Theunissen*

* **[Biotremology](https://doi.org/10.1016/j.cub.2016.01.054)** (2016) <br> 👤 *Peggy S. M. Hill, Andreas Wessel*

* **[Recursion, Language, and Starlings](https://doi.org/10.1080/15326900701399947)** (2007) <br> 👤 *Michael C. Corballis*

* **[Social Complexity as a Driver of Communication and Cognition](https://doi.org/10.1093/icb/icv064)** (2015) <br> 👤 *Kendra B. Sewall*

* **[Acoustic masking in marine ecosystems: intuitions, analysis, and implication](https://doi.org/10.3354/meps08402)** (2009) <br> 👤 *CW Clark, W. T. Ellison, BL Southall, L. Dupree Hatch, SM Van Parijs, AS Frankel, Dimitri Ponirakis*

* **[Nonhuman Primate Communication, Pragmatics, and the Origins of Language](https://doi.org/10.1086/679674)** (2015) <br> 👤 *Thomas C. Scott‐Phillips*

* **[Aping Language: Historical Perspectives on the Quest for Semantics, Syntax, and Other Rarefied Properties of Human Language in the Communication of Primates and Other Animals](https://doi.org/10.3389/fpsyg.2021.675172)** (2021) <br> 👤 *Drew Rendall*

* **[Speaking with other animals through intuitive interspecies communication: towards cognitive and interspecies justice](https://doi.org/10.4337/9781788979993.00018)** (2021) <br> 👤 *Morganne Barrett, Viktoria Hinz, Vanessa Wijngaarden, Marie Lovrod*

* **[Hedgehog signaling in animal development: paradigms and principles](https://doi.org/10.1101/gad.938601)** (2001) <br> 👤 *Philip W. Ingham, Andrew P. McMahon*

* **[Male great tits eavesdrop on simulated male–to–male vocal interactions](https://doi.org/10.1098/rspb.2001.1648)** (2001) <br> 👤 *Tom M. Peake, Andrew M. R. Terry, Peter K. McGregor, Torben Dabelsteen*

* **[Wntless, a Conserved Membrane Protein Dedicated to the Secretion of Wnt Proteins from Signaling Cells](https://doi.org/10.1016/j.cell.2006.02.049)** (2006) <br> 👤 *Carla Bänziger, Davide Soldini, Corina Schütt, Peder Zipperlen, George Hausmann, Konrad Basler*

* **[The Variety of Information Transfer in Animal Sonic Communication: Review from a Physics Perspective](https://doi.org/10.3390/e11040888)** (2009) <br> 👤 *Neville H. Fletcher*

* **[The neurobiology of innate, volitional and learned vocalizations in mammals and birds](https://doi.org/10.1098/rstb.2019.0054)** (2019) <br> 👤 *Andreas Nieder, Richard Mooney*

* **[Dishonest signalling in a fiddler crab](https://doi.org/10.1098/rspb.2000.1062)** (2000) <br> 👤 *Patricia R.Y. Backwell, John H. Christy, Steven R. Telford, Michael D. Jennions, Jennions Passmore*

* **[Parameterizing animal sounds and motion with animal-attached tags to study acoustic communication](https://doi.org/10.1007/s00265-022-03154-0)** (2022) <br> 👤 *Marco Casoli, Mark Johnson, Katherine McHugh, Randall S. Wells, Peter L. Tyack*

* **[Transferring the blues: Depression-associated gut microbiota induces neurobehavioural changes in the rat](https://doi.org/10.1016/j.jpsychires.2016.07.019)** (2016) <br> 👤 *John R. Kelly, Yuliya Borre, Ciaran O' Brien, Elaine Patterson, Sahar El Aidy, Jennifer Deane, Paul J. Kennedy, S. Beers, Karen A. Scott, Gerard M. Moloney, Alan E. Hoban, Lucinda V. Scott, Patrick Fitzgerald, R. Paul Ross, Catherine Stanton, Gerard Clarke, John F. Cryan, Timothy G. Dinan*

* **[Animals of Instagram: taxonomic bias in science communication online](https://doi.org/10.22323/2.20040210)** (2021) <br> 👤 *Grace Heathcote*

* **[Mitochondrial Signaling](https://doi.org/10.1016/s1097-2765(04)00179-0)** (2004) <br> 👤 *Ronald A. Butow, Narayan G. Avadhani*

* **[Behavioral Alterations in Rats Prenatally Exposed to Valproic Acid: Animal Model of Autism](https://doi.org/10.1038/sj.npp.1300518)** (2004) <br> 👤 *Tomasz Schneider, Ryszard Przewłocki*

* **[Use of Augmentative Interspecies Communication devices in animal language studies: A review](https://doi.org/10.1002/wcs.1647)** (2023) <br> 👤 *Gabriella E. Smith, Amalia P. M. Bastos, Ashley Evenson, Leo Trottier, Federico Rossano*

* **[The fear gasping face as a threat display in a Melanesian society](https://doi.org/10.1073/pnas.1611622113)** (2016) <br> 👤 *Carlos Crivelli, James A. Russell, Sergio Jarillo, José Miguel Fernández Dols*

* **[Problematising Upstream Technology through Speculative Design](https://doi.org/10.1145/2702123.2702260)** (2015) <br> 👤 *Shaun Lawson, Ben Kirman, Conor Linehan, Tom Feltwell, Lisa Hopkins*

* **[Scratching beneath the surface: intentionality in great ape signal production](https://doi.org/10.1098/rstb.2018.0403)** (2019) <br> 👤 *Kirsty E. Graham, Claudia Wilke, Nicole J. Lahiff, Katie E. Slocombe*

* **[Inferential Communication: Bridging the Gap Between Intentional and Ostensive Communication in Non-human Primates](https://doi.org/10.3389/fpsyg.2021.718251)** (2022) <br> 👤 *Elizabeth Warren, Josep Call*

* **[Seismic signals in a courting male jumping spider (Araneae:Salticidae)](https://doi.org/10.1242/jeb.00634)** (2003) <br> 👤 *Damian O. Elias, Andrew C. Mason, Wayne P. Maddison, Ronald R. Hoy*

* **[Underwater sound from vessel traffic reduces the effective communication range in Atlantic cod and haddock](https://doi.org/10.1038/s41598-017-14743-9)** (2017) <br> 👤 *Jenni A. Stanley, Sofie M. Van Parijs, Leila Hatch*

* **[Short communication: The role of genotypes from animals without phenotypes in single-step genomic evaluations](https://doi.org/10.3168/jds.2017-12734)** (2017) <br> 👤 *Т. Н. Шабалина, E.C.G. Pimentel, Christian Edel, Laura Plieschke, Reiner Emmerling, Kay‐Uwe Götz*

* **[Open Transparent Communication about Animals in Laboratories: Dialog for Multiple Voices and Multiple Audiences](https://doi.org/10.3390/ani11020368)** (2021) <br> 👤 *Larry Carbone*

* **[Animal Source Food Social and Behavior Change Communication Intervention Among Girinka Livestock Transfer Beneficiaries in Rwanda: A Cluster Randomized Evaluation](https://doi.org/10.9745/ghsp-d-21-00082)** (2021) <br> 👤 *Valerie L. Flax, Emily Ouma, L. Izerimana, Mary-Ann Schreiner, Alice Brower, Eugène Niyonzima, Carine Nyilimana, Adeline Ufitinema, Agnes Uwineza*

* **[Dark Matters: Challenges of Nocturnal Communication Between Plants and Animals in Delivery of Pollination Services.](https://openalex.org/W2981041357)** (2018) <br> 👤 *Renee M. Borges*

* **[An Evaluation of Passive Acoustic Monitoring Using Satellite Communication Technology for Near Real-Time Detection of Tagged Animals in a Marine Setting](https://doi.org/10.2174/1874401x01104010010)** (2011) <br> 👤 *Russell W. Bradford*

* **[Flash signal evolution in<i>Photinus</i>fireflies: Character displacement and signal exploitation in a visual communication system](https://doi.org/10.1111/evo.12606)** (2015) <br> 👤 *Kathrin F. Stanger‐Hall, James E. Lloyd*

* **[The impact of environmental noise on song amplitude in a territorial bird](https://doi.org/10.1111/j.0021-8790.2004.00814.x)** (2004) <br> 👤 *Henrik Brumm*

* **[The role of microbiota-gut-brain axis in neuropsychiatric and neurological disorders](https://doi.org/10.1016/j.phrs.2021.105840)** (2021) <br> 👤 *Katarzyna Socała, Urszula Doboszewska, Aleksandra Szopa, Anna Serefko, Marcin Włodarczyk, Anna Zielińska, Ewa Poleszak, Jakub Fichna, Piotr Właź*

* **[Communication during veterinarian-client-patient interactions in companion animal practice](https://openalex.org/W3093755570)** (2008) <br> 👤 *Jason B. Coe*

* **[Cilia-mediated Hedgehog signaling controls form and function in the mammalian larynx](https://doi.org/10.7554/elife.19153)** (2017) <br> 👤 *Jacqueline M. Tabler, Maggie M. Rigney, Gordon J Berman, S. Gopalakrishnan, Églantine Heude, Hadeel Adel Al-Lami, Basil Z Yannakoudakis, Rebecca Fitch, Christopher Carter, Steven A. Vokes, Karen Liu, Shahragim Tajbakhsh, SE Roian Egnor, John B. Wallingford*

* **[How does honest costly signaling work?](https://doi.org/10.1093/beheco/art097)** (2013) <br> 👤 *James P. Higham*

* **[Songbird: a unique animal model for studying the molecular basis of disorders of vocal development and communication](https://doi.org/10.1538/expanim.15-0008)** (2015) <br> 👤 *Chihiro Mori, Kazuhiro Wada*

* **[Social buffering: relief from stress and anxiety](https://doi.org/10.1098/rstb.2006.1941)** (2006) <br> 👤 *Takefumi Kikusui, James Winslow, Yuji Mori*

* **[<i>Severe acute respiratory syndrome-related coronavirus</i> : The species and its viruses – a statement of the Coronavirus Study Group](https://doi.org/10.1101/2020.02.07.937862)** (2020) <br> 👤 *Alexander E. Gorbalenya, Susan C. Baker, Ralph S. Baric, Raoul J. de Groot, Christian Drosten, Anastasia Gulyaeva, Bart L. Haagmans, Chris Lauber, A. M. Leontovich, Benjamin W. Neuman, Dmitry Penzar, Stanley Perlman, Leo L. M. Poon, Dmitry V. Samborskiy, Igor A. Sidorov, Isabel Sola, John Ziebuhr*

* **[Contributions to the mathematical theory of epidemics. II. —The problem of endemicity](https://doi.org/10.1098/rspa.1932.0171)** (1932) <br> 👤 *W. O. Kermack, A. G. McKendrick*

* **[Sustained Perceptual Deficits from Transient Sensory Deprivation](https://doi.org/10.1523/jneurosci.0837-15.2015)** (2015) <br> 👤 *Melissa L. Caras, DH Sanes*

* **[Principles of structure building in music, language and animal song](https://doi.org/10.1098/rstb.2014.0097)** (2015) <br> 👤 *Martin Rohrmeier, Willem Zuidema, Geraínt A. Wiggins, Constance Scharff*

* **[Exploiting Quorum Sensing To Confuse Bacterial Pathogens](https://doi.org/10.1128/mmbr.00046-12)** (2013) <br> 👤 *Breah LaSarre, Michael J. Federle*

* **[An Intracellular Antioxidant Determines the Expression of a Melanin-Based Signal in a Bird](https://doi.org/10.1371/journal.pone.0003335)** (2008) <br> 👤 *Ismael Galván, Carlos Alonso‐Álvarez*

* **[Multimodal signalling in the North American barn swallow: a phenotype network approach](https://doi.org/10.1098/rspb.2015.1574)** (2015) <br> 👤 *Matthew R. Wilkins, Daizaburo Shizuka, Maxwell B. Joseph, Joanna K. Hubbard, Rebecca J. Safran*

* **[Ancient Protostome Origin of Chemosensory Ionotropic Glutamate Receptors and the Evolution of Insect Taste and Olfaction](https://doi.org/10.1371/journal.pgen.1001064)** (2010) <br> 👤 *Vincent Croset, Raphael Rytz, Scott F. Cummins, Aidan Budd, David Brawand, Henrik Kaessmann, Toby J. Gibson, Richard Benton*

* **[Ground squirrel uses ultrasonic alarms](https://doi.org/10.1038/430523a)** (2004) <br> 👤 *David R. Wilson, James F. Hare*

* **[Expression of emotion in voice and music](https://doi.org/10.1016/s0892-1997(05)80231-0)** (1995) <br> 👤 *Klaus R. Scherer*

* **[The blood-brain barrier in systemic inflammation](https://doi.org/10.1016/j.bbi.2016.03.010)** (2016) <br> 👤 *Aravinthan Varatharaj, Ian Galea*

* **[“Bio-communication” Between Mother and Offspring: Lessons From Animals and New Perspectives for Brain Science](https://doi.org/10.1254/jphs.09r01cp)** (2009) <br> 👤 *Yusuke Tozuka, Etsuko Wada, Keiji Wada*

* **[Cytokines in clinical cancer immunotherapy](https://doi.org/10.1038/s41416-018-0328-y)** (2018) <br> 👤 *Pedro Berraondo, Miguel F. Sanmamed, María C. Ochoa, Iñaki Etxeberría, M. Ángela Aznar, José Luis Perez‐Gracia, María E. Rodríguez-Ruiz, Mariano Ponz‐Sarvisé, Eduardo Castañón, Ignacio Melero*

* **[Future directions for soundscape ecology: The importance of ornithological contributions](https://doi.org/10.1642/auk-16-124.1)** (2016) <br> 👤 *Amandine Gasc, Dante Francomano, John B. Dunning, Bryan C. Pijanowski*

* **[Qualitative analysis of small animal veterinarian–perceived barriers to nutrition communication](https://doi.org/10.2460/javma.23.05.0281)** (2023) <br> 👤 *Sophie Wenzel, Jason B. Coe, Tara Long, Sydney Koerner, Morgan Harvey, Megan Shepherd*

* **[From gut dysbiosis to altered brain function and mental illness: mechanisms and pathways](https://doi.org/10.1038/mp.2016.50)** (2016) <br> 👤 *Geraint B. Rogers, Damien J. Keating, Richard L. Young, Ma‐Li Wong, Julio Licínio, Steve Wesselingh*

* **[A survey of animal welfare needs in Soweto : research communication](https://doi.org/10.4102/jsava.v68i4.895)** (1997) <br> 👤 *C.M.E. McCrindle, Sandra Cornelius, Hennie Schoeman*

* **[How Information Communication Technology Can Enhance Evidence-Based Decisions and Farm-to-Fork Animal Traceability for Livestock Farmers](https://doi.org/10.1155/2020/1279569)** (2020) <br> 👤 *G. Mwanga, Ernest R. Mbega, Zaipuna O. Yonah, M.G.G. Chagunda*

* **[Teaching with Animals: The Role of Animal Ambassadors in Improving Presenter Communication Skills](https://doi.org/10.5032/jae.2017.01223)** (2017) <br> 👤 *Nicholas E. Fuhrman, Eric D. Rubenstein*

* **[Communication skills training and assessment of food animal production medicine veterinarians: A component of a voluntary Johne's disease control program](https://doi.org/10.3168/jds.2021-20677)** (2022) <br> 👤 *Karen A. MacDonald-Phillips, Shawn McKenna, Darcy H. Shaw, Greg Keefe, John VanLeeuwen, Elpida Artemiou, Cindy L. Adams*

* **[The nature of the root clock at single cell resolution: Principles of communication and similarities with plant and animal pulsatile and circadian mechanisms](https://doi.org/10.1016/j.ceb.2022.102102)** (2022) <br> 👤 *Pablo Pérez-García, Laura Serrano-Ron, Miguel Á. Moreno-Risueno*

* **[SUCCESSFUL CULTURE <i>IN VITRO</i> OF SHEEP AND CATTLE OVA](https://doi.org/10.1530/jrf.0.0300493)** (1972) <br> 👤 *H.R. Tervit, D. G. Whittingham, L. E. A. ROWSON*

* **[Pointillist structural color in <i>Pollia</i> fruit](https://doi.org/10.1073/pnas.1210105109)** (2012) <br> 👤 *Silvia Vignolini, Paula J. Rudall, Alice Rowland, Alison Reed, Edwige Moyroud, Robert B. Faden, Jeremy J. Baumberg, Beverley J. Glover, Ullrich Steiner*

* **[The contribution of source–filter theory to mammal vocal communication research](https://doi.org/10.1111/j.1469-7998.2009.00661.x)** (2009) <br> 👤 *Angela M. Taylor, David Reby*

* **[Monitoring Animal Behaviour and Environmental Interactions Using Wireless Sensor Networks, GPS Collars and Satellite Remote Sensing](https://doi.org/10.3390/s90503586)** (2009) <br> 👤 *R.N. Handcock, Dave L. Swain, Greg Bishop-Hurley, Kym P. Patison, Tim Wark, Philip Valencia, Peter Corke, C. J. O’Neill*

* **[Stigma and prejudice: One animal or two?](https://doi.org/10.1016/j.socscimed.2008.03.022)** (2008) <br> 👤 *Jo C. Phelan, Bruce G. Link, John F. Dovidio*

* **[Multilevel rhythms in multimodal communication](https://doi.org/10.1098/rstb.2020.0334)** (2021) <br> 👤 *Wim Pouw, Shannon Proksch, Linda Drijvers, Marco Gamba, Judith Holler, Christopher T. Kello, Rebecca Schaefer, Geraínt A. Wiggins*

* **[Anthropomorphism in Human–Animal Interactions: A Pragmatist View](https://doi.org/10.3389/fpsyg.2018.02590)** (2018) <br> 👤 *Véronique Servais*

* **[Communication in farm animal practice 2. Effecting change](https://doi.org/10.1136/inp.c1603)** (2010) <br> 👤 *Owen Atkinson*

* **[Science communication challenges about antimicrobial resistance in animal agriculture: insights from stakeholders](https://doi.org/10.1093/jacamr/dlac032)** (2022) <br> 👤 *Andy J. King, Dara M. Wald, Denise D. Coberley, Michael F. Dahlstrom, Paul J. Plummer*

* **[The Neurotoxicant, Cuprizone, as a Model to Study Demyelination and Remyelination in the Central Nervous System](https://doi.org/10.1111/j.1750-3639.2001.tb00385.x)** (2001) <br> 👤 *Glenn K. Matsushima, Pierre Morell*

* **[Linking Diet to Colorectal Cancer: The Emerging Role of MicroRNA in the Communication between Plant and Animal Kingdoms](https://doi.org/10.3389/fmicb.2017.00597)** (2017) <br> 👤 *Manuela Del Cornò, Gloria Donninelli, Lucia Conti, Sandra Gessani*

* **[Animal vocal sequences: not the Markov chains we thought they were](https://doi.org/10.1098/rspb.2014.1370)** (2014) <br> 👤 *Arik Kershenbaum, Ann E. Bowles, Todd M. Freeberg, Dezhe Z. Jin, Adriano R. Lameira, Kirsten M. Bohn*

* **[Protozoa and plant growth: the microbial loop in soil revisited](https://doi.org/10.1111/j.1469-8137.2004.01066.x)** (2004) <br> 👤 *Michael Bonkowski*

* **[Stress &amp; the gut-brain axis: Regulation by the microbiome](https://doi.org/10.1016/j.ynstr.2017.03.001)** (2017) <br> 👤 *Jane A. Foster, Linda Rinaman, John F. Cryan*

* **[Understanding Communication Barriers: Demographic Variables and Language Needs in the Interaction between English-Speaking Animal Professionals and Spanish-Speaking Animal Caretakers](https://doi.org/10.3390/ani14040624)** (2024) <br> 👤 *Allen Jimena Martinez Aguiriano, Leonor Salazar, Silvana Pietrosemoli, Marcelo Schmidt, Babafela Awosile, Arlene García*

* **[The Evolution of Honest Communication: Integrating Social and Physiological Costs of Ornamentation](https://doi.org/10.1093/icb/icu083)** (2014) <br> 👤 *Elizabeth A. Tibbetts*

* **[The cocktail party problem: What is it? How can it be solved? And why should animal behaviorists study it?](https://doi.org/10.1037/0735-7036.122.3.235)** (2008) <br> 👤 *Mark A. Bee, Christophe Micheyl*

* **[Correction to: MicroRNAs from plants to animals, do they define a new messenger for communication?](https://doi.org/10.1186/s12986-018-0311-x)** (2018) <br> 👤 *Zhiqing Li, Ruodan Xu, Ning Li*

* **[The genome of a songbird](https://doi.org/10.1038/nature08819)** (2010) <br> 👤 *Wesley C. Warren, David F. Clayton, Hans Ellegren, Arthur P. Arnold, LaDeana W. Hillier, Axel Künstner, Steve Searle, Simon White, Albert J. Vilella, Susan Fairley, Andreas Heger, Lesheng Kong, Chris P. Ponting, Erich D. Jarvis, Claudio V. Mello, Pat Minx, Peter V. Lovell, Tarciso Velho, Margaret Ferris, Christopher N. Balakrishnan, Saurabh Sinha, Charles Blatti, Sarah E. London, Yun Li, Ya‐Chi Lin, Julia M. George, Jonathan V. Sweedler, Bruce R. Southey, Preethi H. Gunaratne, Mick Watson, Kiwoong Nam, Niclas Backström, Linnéa Smeds, Benoît Nabholz, Yuichiro Itoh, Osceola Whitney, Andreas R. Pfenning, Jason T. Howard, Martin A. Volker, Benjamin M. Skinner, Darren K. Griffin, Ye Liang, William McLaren, Paul Flicek, Vı́ctor Quesada, Gloria Velasco, Carlos López-Otı́n, Xosé S. Puente, Tsviya Olender, Doron Lancet, Arian F. A. Smit, Robert Hubley, Miriam K. Konkel, Jerilyn A. Walker, Mark A. Batzer, Wanjun Gu, David D. Pollock, Lin Chen, Ze Cheng, Evan E. Eichler, Jessica Stapley, Jon Slate, Robert Ekblom, T. R. Birkhead, Terry Burke, David W. Burt, Constance Scharff, Iris Adam, Hugues Richard, Marc Sultan, А. В. Солдатов, Hans Lehrach, Scott V. Edwards, Shiaw‐Pyng Yang, XiaoChing Li, Tina Graves, Lucinda Fulton, Joanne O. Nelson, Asif Chinwalla, Shunfeng Hou, Elaine R. Mardis, Richard K. Wilson*

* **[Mechanisms of pallidal deep brain stimulation: Alteration of cortico-striatal synaptic communication in a dystonia animal model](https://doi.org/10.1016/j.nbd.2021.105341)** (2021) <br> 👤 *Marco Heerdegen, Monique Zwar, Denise Franz, Max Frederik Hörnschemeyer, Valentin Neubert, Franz Plocksties, Christoph Niemann, Dirk Timmermann, Christian Bahls, Ursula van Rienen, Maria Paap, Stefanie Perl, Anika Lüttig, Angelika Richter, Rüdiger Köhling*

* **[Short communication: detection and molecular characterization of hepatitis E virus in domestic animals of São Tomé and Príncipe](https://doi.org/10.1007/s11250-018-1700-x)** (2018) <br> 👤 *João R. Mesquita, Cláudia Istrate, Nânci Santos-Ferreira, Ana Sofia Ferreira, Joana Abreu-Silva, José Carlos Esteves Veiga, Wim H. M. van der Poel, María Säo José Nascimento*

* **[The fitness and functionality of culturally evolved communication systems](https://doi.org/10.1098/rstb.2008.0130)** (2008) <br> 👤 *Nicolas Fay, Simon Garrod, Leo Roberts*

* **[Mate choice in a polluted world: consequences for individuals, populations and communities](https://doi.org/10.1098/rstb.2018.0055)** (2019) <br> 👤 *Ulrika Candolin, Bob B. M. Wong*

* **[Robot Therapy: A New Approach for Mental Healthcare of the Elderly – A Mini-Review](https://doi.org/10.1159/000319015)** (2010) <br> 👤 *Takanori Shibata, Kazuyoshi Wada*

* **[The relative efficiency of various parts of the spectrum for the heliotropic reactions of animals and plants second communication](https://doi.org/10.1002/jez.1400200207)** (1916) <br> 👤 *Jacques Loeb, Hardolph Wasteneys*

* **[Informational Masking Constrains Vocal Communication in Nonhuman Animals](https://doi.org/10.1101/2022.03.31.486641)** (2022) <br> 👤 *Saumya Gupta, Lata Kalra, Gary J. Rose, Mark A. Bee, Mark A. Bee*

* **[Vocal communication in nonhuman animals: View from the wings](https://doi.org/10.26451/abc.07.02.03.2020)** (2020) <br> 👤 *Irene M. Pepperberg*

* **[The expression of melanin-based plumage is separately modulated by exogenous oxidative stress and a melanocortin](https://doi.org/10.1098/rspb.2009.0774)** (2009) <br> 👤 *Ismael Galván, Carlos Alonso‐Álvarez*

* **[Nutrient Requirements of Ruminants in Developing Countries](https://doi.org/10.26076/6328-a024)** (2021) <br> 👤 *Leonard C. Kearl*

* **[Complexity of mutual communication in animals exemplified by paired dances in the red-crowned crane](https://doi.org/10.2502/janip.68.1.1)** (2017) <br> 👤 *Kohei Takeda, Nobuyuki Kutsukake*

* **[Alarm calls evoke a visual search image of a predator in birds](https://doi.org/10.1073/pnas.1718884115)** (2018) <br> 👤 *Toshitaka N. Suzuki*

* **[Brain–computer interfaces: communication and restoration of movement in paralysis](https://doi.org/10.1113/jphysiol.2006.125633)** (2007) <br> 👤 *Niels Birbaumer, Leonardo G. Cohen*

* **[Efflux-Mediated Drug Resistance in Bacteria](https://doi.org/10.2165/11317030-000000000-00000)** (2009) <br> 👤 *Xian-Zhi Li, Hiroshi Nikaido*

* **[Initiation of DNA Fragmentation during Apoptosis Induces Phosphorylation of H2AX Histone at Serine 139](https://doi.org/10.1074/jbc.275.13.9390)** (2000) <br> 👤 *Emmy P. Rogakou, Wilberto Nieves‐Neira, Chye Boon, ﻿Yves Pommier, William M. Bonner*

* **[<scp>ASVCP</scp>reference interval guidelines: determination of de novo reference intervals in veterinary species and other related topics](https://doi.org/10.1111/vcp.12006)** (2012) <br> 👤 *Kristen R. Friedrichs, Kendal E. Harr, Kathleen P. Freeman, Balázs Szladovits, Raquel M. Walton, Kirstin F. Barnhart, Julia Blanco‐Chavez*

* **[A female songbird out-sings male conspecifics during simulated territorial intrusions](https://doi.org/10.1098/rspb.2008.1445)** (2008) <br> 👤 *Anya E Illes, Laila Yunes-Jimenez*

